# Chapter 7: Functions and Tasks

Functions and tasks are essential constructs in SystemVerilog that allow you to create reusable blocks of code, improving modularity and maintainability. While they serve similar purposes, they have distinct characteristics and use cases.

## Function Declarations and Calls

Functions in SystemVerilog are subroutines that return a value and execute in zero simulation time. They are ideal for computational operations and combinational logic.

## Basic Function Syntax

```systemverilog
function [return_type] function_name ([arguments]);
    // Function body
    return return_value;
endfunction
```

### Simple Function Examples

```systemverilog
module function_examples;
    
    // Function to add two integers
    function int add(int a, int b);
        return a + b;
    endfunction
    
    // Function to find maximum of two values
    function int max(int x, int y);
        if (x > y)
            return x;
        else
            return y;
    endfunction
    
    // Function with bit vector operations
    function logic [7:0] reverse_bits(logic [7:0] data);
        logic [7:0] result;
        for (int i = 0; i < 8; i++) begin
            result[i] = data[7-i];
        end
        return result;
    endfunction
    
    initial begin
        int result1, result2, result3;
        logic [7:0] original = 8'b10110100;
        logic [7:0] reversed;
        
        result1 = add(15, 25);           // Returns 40
        result2 = max(100, 50);          // Returns 100
        reversed = reverse_bits(original); // Returns 8'b00101101
        
        $display("Add result: %d", result1);
        $display("Max result: %d", result2);
        $display("Original: %b, Reversed: %b", original, reversed);
    end
    
endmodule
```

### Functions with Different Return Types

```systemverilog
module function_types;
    
    // Function returning a structure
    typedef struct {
        int quotient;
        int remainder;
    } div_result_t;
    
    function div_result_t divide(int dividend, int divisor);
        div_result_t result;
        result.quotient = dividend / divisor;
        result.remainder = dividend % divisor;
        return result;
    endfunction
    
    // Function returning an array
    function logic [3:0] [7:0] create_pattern(logic [7:0] base);
        logic [3:0] [7:0] pattern;
        for (int i = 0; i < 4; i++) begin
            pattern[i] = base << i;
        end
        return pattern;
    endfunction
    
    initial begin
        div_result_t div_res;
        logic [3:0] [7:0] pattern;
        
        div_res = divide(17, 5);
        pattern = create_pattern(8'b00001111);
        
        $display("17/5 = %d remainder %d", div_res.quotient, div_res.remainder);
        
        for (int i = 0; i < 4; i++) begin
            $display("Pattern[%d]: %b", i, pattern[i]);
        end
    end
    
endmodule
```

## Task Declarations and Calls

Tasks are subroutines that can consume simulation time and don't return values directly. They can have input, output, and inout arguments, making them suitable for complex operations and time-consuming activities.

### Basic Task Syntax

```systemverilog
task task_name ([arguments]);
    // Task body
endtask
```

### Task Examples

```systemverilog
module task_examples;
    
    logic clk = 0;
    logic [7:0] data;
    logic valid;
    
    // Generate clock
    always #5 clk = ~clk;
    
    // Task to wait for a number of clock cycles
    task wait_cycles(int num_cycles);
        repeat(num_cycles) @(posedge clk);
    endtask
    
    // Task to send data with handshaking
    task send_data(input logic [7:0] send_data, output logic done);
        data = send_data;
        valid = 1'b1;
        @(posedge clk);
        valid = 1'b0;
        done = 1'b1;
        @(posedge clk);
        done = 1'b0;
    endtask
    
    // Task with multiple outputs
    task analyze_data(input logic [7:0] input_data, 
                      output int ones_count, 
                      output int zeros_count,
                      output logic parity);
        ones_count = 0;
        zeros_count = 0;
        
        for (int i = 0; i < 8; i++) begin
            if (input_data[i])
                ones_count++;
            else
                zeros_count++;
        end
        
        parity = ^input_data; // XOR reduction for parity
    endtask
    
    initial begin
        logic done;
        int ones, zeros;
        logic par;
        
        // Wait for some cycles
        wait_cycles(3);
        
        // Send some data
        send_data(8'hA5, done);
        $display("Data sent, done = %b", done);
        
        // Analyze data
        analyze_data(8'b11010110, ones, zeros, par);
        $display("Ones: %d, Zeros: %d, Parity: %b", ones, zeros, par);
        
        #100 $finish;
    end
    
endmodule
```

## Automatic vs. Static Lifetime

The lifetime of variables in functions and tasks can be either static (default) or automatic. This affects how variables are allocated and whether they retain values between calls.

### Static Lifetime (Default)

```systemverilog
module static_lifetime;
    
    // Static function - variables retain values between calls
    function int counter();
        int count = 0;  // Initialized only once
        count++;
        return count;
    endfunction
    
    // Static task
    task static_task();
        static int call_count = 0;
        call_count++;
        $display("Static task called %d times", call_count);
    endtask
    
    initial begin
        $display("Counter: %d", counter()); // Prints 1
        $display("Counter: %d", counter()); // Prints 2
        $display("Counter: %d", counter()); // Prints 3
        
        static_task(); // Prints: Static task called 1 times
        static_task(); // Prints: Static task called 2 times
    end
    
endmodule
```

### Automatic Lifetime

```systemverilog
module automatic_lifetime;
    
    // Automatic function - fresh variables for each call
    function automatic int factorial(int n);
        if (n <= 1)
            return 1;
        else
            return n * factorial(n-1); // Recursive call possible
    endfunction
    
    // Automatic task
    task automatic print_sequence(int start, int count);
        for (int i = 0; i < count; i++) begin
            $display("Value: %d", start + i);
            #10; // Can consume time
        end
    endtask
    
    initial begin
        $display("5! = %d", factorial(5)); // Prints 120
        
        // Multiple concurrent task calls
        fork
            print_sequence(10, 3);
            print_sequence(20, 3);
        join
    end
    
endmodule
```

## Pass by Reference

SystemVerilog supports passing arguments by reference using the `ref` keyword, allowing functions and tasks to modify the original variables.

### Pass by Reference Examples

```systemverilog
module pass_by_reference;
    
    // Function with reference arguments
    function automatic void swap(ref int a, ref int b);
        int temp = a;
        a = b;
        b = temp;
    endfunction
    
    // Task to initialize an array by reference
    task automatic init_array(ref int arr[10], input int init_value);
        for (int i = 0; i < 10; i++) begin
            arr[i] = init_value + i;
        end
    endtask
    
    // Function to modify a structure by reference
    typedef struct {
        int x, y;
        string name;
    } point_t;
    
    function automatic void move_point(ref point_t p, int dx, int dy);
        p.x += dx;
        p.y += dy;
    endfunction
    
    initial begin
        int x = 10, y = 20;
        int my_array[10];
        point_t my_point = '{100, 200, "PointA"};
        
        $display("Before swap: x=%d, y=%d", x, y);
        swap(x, y);
        $display("After swap: x=%d, y=%d", x, y);
        
        init_array(my_array, 50);
        $display("Array elements: %p", my_array);
        
        $display("Before move: %s at (%d,%d)", my_point.name, my_point.x, my_point.y);
        move_point(my_point, 15, -25);
        $display("After move: %s at (%d,%d)", my_point.name, my_point.x, my_point.y);
    end
    
endmodule
```

## Return Statements in Functions

Functions must return a value, and the return statement determines what value is returned. You can have multiple return statements in a function.

### Multiple Return Statements

```systemverilog
module return_statements;
    
    // Function with multiple return points
    function automatic string grade_letter(int score);
        if (score >= 90)
            return "A";
        else if (score >= 80)
            return "B"; 
        else if (score >= 70)
            return "C";
        else if (score >= 60)
            return "D";
        else
            return "F";
    endfunction
    
    // Function with early return for error checking
    function automatic real safe_divide(real dividend, real divisor);
        if (divisor == 0.0) begin
            $error("Division by zero attempted");
            return 0.0; // Early return for error case
        end
        return dividend / divisor;
    endfunction
    
    // Function with complex logic and multiple returns
    function automatic int find_first_one(logic [31:0] data);
        for (int i = 0; i < 32; i++) begin
            if (data[i] == 1'b1)
                return i; // Return position of first '1'
        end
        return -1; // Return -1 if no '1' found
    endfunction
    
    initial begin
        string letter;
        real result;
        int position;
        
        letter = grade_letter(85);
        $display("Score 85 gets grade: %s", letter);
        
        result = safe_divide(10.0, 3.0);
        $display("10.0 / 3.0 = %f", result);
        
        result = safe_divide(5.0, 0.0); // Will show error
        
        position = find_first_one(32'h00008000);
        $display("First '1' found at position: %d", position);
        
        position = find_first_one(32'h00000000);
        $display("First '1' found at position: %d", position);
    end
    
endmodule
```

## Void Functions

Void functions don't return a value and are similar to tasks, but they execute in zero simulation time and cannot contain timing control statements.

### Void Function Examples

```systemverilog
module void_functions;
    
    int global_counter = 0;
    logic [7:0] memory [256];
    
    // Void function to increment global counter
    function automatic void increment_counter(int step);
        global_counter += step;
    endfunction
    
    // Void function to initialize memory
    function automatic void init_memory(logic [7:0] pattern);
        for (int i = 0; i < 256; i++) begin
            memory[i] = pattern ^ i[7:0];
        end
    endfunction
    
    // Void function for debug printing
    function automatic void debug_print(string msg, int value);
        $display("[DEBUG %0t] %s: %d", $time, msg, value);
    endfunction
    
    // Void function with reference parameter
    function automatic void reset_array(ref int arr[]);
        foreach(arr[i]) begin
            arr[i] = 0;
        end
    endfunction
    
    initial begin
        int test_array[5] = '{1, 2, 3, 4, 5};
        
        debug_print("Initial counter", global_counter);
        
        increment_counter(5);
        debug_print("After increment", global_counter);
        
        init_memory(8'hAA);
        debug_print("Memory[0]", memory[0]);
        debug_print("Memory[1]", memory[1]);
        debug_print("Memory[255]", memory[255]);
        
        $display("Before reset: %p", test_array);
        reset_array(test_array);
        $display("After reset: %p", test_array);
    end
    
endmodule
```

## Best Practices and Guidelines

### When to Use Functions vs. Tasks

**Use Functions when:**
- You need to return a single value
- The operation is purely combinational
- No timing control is needed
- The operation should complete in zero simulation time

**Use Tasks when:**
- You need multiple outputs
- Timing control is required
- The operation may consume simulation time
- You need to model sequential behavior

### Function and Task Design Guidelines

```systemverilog
module design_guidelines;
    
    // Good: Pure function with clear purpose
    function automatic int absolute_value(int value);
        return (value < 0) ? -value : value;
    endfunction
    
    // Good: Task with clear interface and timing
    task automatic wait_for_ready(ref logic ready_signal, input int timeout_cycles);
        int cycle_count = 0;
        while (!ready_signal && cycle_count < timeout_cycles) begin
            @(posedge clk);
            cycle_count++;
        end
        if (cycle_count >= timeout_cycles) begin
            $error("Timeout waiting for ready signal");
        end
    endtask
    
    // Good: Function with appropriate use of reference
    function automatic void normalize_vector(ref real vector[3]);
        real magnitude = $sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2);
        if (magnitude != 0.0) begin
            vector[0] /= magnitude;
            vector[1] /= magnitude;
            vector[2] /= magnitude;
        end
    endfunction
    
endmodule
```

## Summary

Functions and tasks are powerful constructs in SystemVerilog that enable code reuse and modular design:

- **Functions** return values, execute in zero time, and are ideal for combinational logic
- **Tasks** can have multiple outputs, consume time, and are suited for sequential operations
- **Automatic lifetime** creates fresh variables for each call and enables recursion
- **Static lifetime** (default) preserves variable values between calls
- **Pass by reference** allows modification of original variables
- **Void functions** provide task-like behavior without return values but in zero time

Understanding when and how to use functions and tasks effectively will greatly improve your SystemVerilog code organization and reusability.

# SystemVerilog Functions and Tasks - Example Index

## Function Declarations and Calls

#### Mathematical Functions
Basic arithmetic operations like factorial, power, greatest common divisor

In [2]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_1__mathematical_functions/"
files = ["mathematical_calculator.sv", "mathematical_calculator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// mathematical_calculator.sv
module mathematical_calculator ();               // Mathematical functions design

  // Function to calculate factorial
  function automatic integer calculate_factorial;
    input integer number_input;
    integer factorial_result;
    begin
      if (number_input <= 1)
        factorial_result = 1;
      else
        factorial_result = number_input * calculate_factorial(number_input - 1);
      calculate_factorial = factorial_result;
    end
  endfunction

  // Function to calculate power (base^exponent)
  function automatic integer calculate_power;
    input integer base_value;
    input integer exponent_value;
    integer power_result;
    integer loop_counter;
    begin
      power_result = 1;
      for (loop_counter = 0; loop_counter < exponent_value; loop_counter = loop_counter + 1)
        power_result = power_result * base_value;
      calculate_power = power_result;
    end
  endfunction

  // Function to calculate greatest common divisor using Euclidean algorithm
  function automatic integer calculate_gcd;
    input integer first_number;
    input integer second_number;
    integer temp_remainder;
    begin
      while (second_number != 0) begin
        temp_remainder = first_number % second_number;
        first_number = second_number;
        second_number = temp_remainder;
      end
      calculate_gcd = first_number;
    end
  endfunction

  initial begin
    $display();
    $display("=== Mathematical Functions Demonstration ===");
    
    // Test factorial function
    $display("Factorial Tests:");
    $display("  5! = %0d", calculate_factorial(5));
    $display("  0! = %0d", calculate_factorial(0));
    $display("  3! = %0d", calculate_factorial(3));
    
    // Test power function
    $display("Power Tests:");
    $display("  2^3 = %0d", calculate_power(2, 3));
    $display("  5^2 = %0d", calculate_power(5, 2));
    $display("  10^0 = %0d", calculate_power(10, 0));
    
    // Test GCD function
    $display("Greatest Common Divisor Tests:");
    $display("  GCD(48, 18) = %0d", calculate_gcd(48, 18));
    $display("  GCD(100, 25) = %0d", calculate_gcd(100, 25));
    $display("  GCD(17, 13) = %0d", calculate_gcd(17, 13));
    
    $display();
  end

endmodule
```

```systemverilog
// mathematical_calculator_testbench.sv
module math_calculator_testbench;  // Testbench for mathematical calculator

  mathematical_calculator MATH_CALCULATOR_INSTANCE();  // Instantiate mathematical calculator

  // Test variables
  integer test_factorial_result;
  integer test_power_result;
  integer test_gcd_result;

  initial begin
    // Dump waves for simulation
    $dumpfile("math_calculator_testbench.vcd");
    $dumpvars(0, math_calculator_testbench);
    
    #1;  // Wait for design to initialize
    
    $display();
    $display("=== Testbench Verification ===");
    
    // Verify factorial calculations
    test_factorial_result = MATH_CALCULATOR_INSTANCE.calculate_factorial(4);
    $display("Testbench: 4! = %0d (Expected: 24)", test_factorial_result);
    
    // Verify power calculations  
    test_power_result = MATH_CALCULATOR_INSTANCE.calculate_power(3, 4);
    $display("Testbench: 3^4 = %0d (Expected: 81)", test_power_result);
    
    // Verify GCD calculations
    test_gcd_result = MATH_CALCULATOR_INSTANCE.calculate_gcd(60, 48);
    $display("Testbench: GCD(60, 48) = %0d (Expected: 12)", test_gcd_result);
    
    $display();
    $display("Mathematical functions testbench completed successfully!");
    $display();
    
    #10;  // Wait before finishing
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Mathematical Functions Demonstration ===
Factorial Tests:
  5! = 120
  0! = 1
  3! = 6
Power Tests:
  2^3 = 8
  5^2 = 25
  10^0 = 1
Greatest Common Divisor Tests:
  GCD(48, 18) = 6
  GCD(100, 25) = 25
  GCD(17, 13) = 1


=== Testbench Verification ===
Testbench: 4! = 24 (Expected: 24)
Testbench: 3^4 = 81 (Expected: 81)
Testbench: GCD(60, 48) = 12 (Expected: 12)

Mathematical functions testbench completed successfully!

Process finished with return code: 0
Removing Chapter_7_examples/example_1__mathematical_functions/obj_dir directory...
Chapter_7_examples/example_1__mathematical_functions/obj_dir removed successfully.


0

#### String Manipulation Functions
Functions for string processing, parsing, and formatting operations

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_2__string_manipulation/"
files = ["string_processor.sv", "string_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// string_processor.sv
module string_processor ();
  
  // String manipulation functions demonstration
  function automatic string format_name(string first_name, string last_name);
    return $sformatf("%s %s", first_name, last_name);
  endfunction
  
  function automatic string extract_extension(string filename);
    int dot_position;
    dot_position = filename.len() - 1;
    
    // Find the last dot in filename
    while (dot_position >= 0 && filename[dot_position] != ".") begin
      dot_position--;
    end
    
    if (dot_position >= 0)
      return filename.substr(dot_position + 1, filename.len() - 1);
    else
      return "no_extension";
  endfunction
  
  function automatic string reverse_string(string input_text);
    string reversed_text = "";
    for (int i = input_text.len() - 1; i >= 0; i--) begin
      reversed_text = {reversed_text, input_text[i]};
    end
    return reversed_text;
  endfunction
  
  function automatic int count_vowels(string text);
    int vowel_count = 0;
    string lowercase_text;
    
    lowercase_text = text.tolower();
    
    for (int i = 0; i < lowercase_text.len(); i++) begin
      case (lowercase_text[i])
        "a", "e", "i", "o", "u": vowel_count++;
        default: ; // Non-vowel characters, do nothing
      endcase
    end
    return vowel_count;
  endfunction
  
  initial begin
    string first_name = "Alice";
    string last_name = "Johnson";
    string full_name;
    string filename = "report.txt";
    string file_extension;
    string sample_text = "SystemVerilog";
    string reversed_text;
    int vowel_count;
    
    $display("=== String Manipulation Functions Demo ===");
    $display();
    
    // Format name demonstration
    full_name = format_name(first_name, last_name);
    $display("Name formatting:");
    $display("  First: %s, Last: %s", first_name, last_name);
    $display("  Full name: %s", full_name);
    $display();
    
    // File extension extraction
    file_extension = extract_extension(filename);
    $display("File extension extraction:");
    $display("  Filename: %s", filename);
    $display("  Extension: %s", file_extension);
    $display();
    
    // String reversal
    reversed_text = reverse_string(sample_text);
    $display("String reversal:");
    $display("  Original: %s", sample_text);
    $display("  Reversed: %s", reversed_text);
    $display();
    
    // Vowel counting
    vowel_count = count_vowels(sample_text);
    $display("Vowel counting:");
    $display("  Text: %s", sample_text);
    $display("  Vowel count: %0d", vowel_count);
    $display();
  end

endmodule
```

```systemverilog
// string_processor_testbench.sv
module string_processor_testbench;
  
  // Instantiate the string processor design
  string_processor STRING_PROCESSOR_INSTANCE();
  
  // Additional testbench verification
  task automatic verify_string_functions();
    string test_first = "Bob";
    string test_last = "Smith";
    string expected_full = "Bob Smith";
    string actual_full;
    
    string test_filename = "data.csv";
    string expected_ext = "csv";
    string actual_ext;
    
    string test_word = "hello";
    string expected_reverse = "olleh";
    string actual_reverse;
    
    int expected_vowels = 2; // "e" and "o" in "hello"
    int actual_vowels;
    
    $display("=== Testbench Verification ===");
    $display();
    
    // Test name formatting
    actual_full = STRING_PROCESSOR_INSTANCE.format_name(test_first, test_last);
    $display("Name Format Test:");
    $display("  Expected: %s", expected_full);
    $display("  Actual:   %s", actual_full);
    $display("  Result:   %s", (actual_full == expected_full) ? "PASS" : "FAIL");
    $display();
    
    // Test file extension extraction
    actual_ext = STRING_PROCESSOR_INSTANCE.extract_extension(test_filename);
    $display("Extension Test:");
    $display("  Expected: %s", expected_ext);
    $display("  Actual:   %s", actual_ext);
    $display("  Result:   %s", (actual_ext == expected_ext) ? "PASS" : "FAIL");
    $display();
    
    // Test string reversal
    actual_reverse = STRING_PROCESSOR_INSTANCE.reverse_string(test_word);
    $display("Reverse Test:");
    $display("  Expected: %s", expected_reverse);
    $display("  Actual:   %s", actual_reverse);
    $display("  Result:   %s", (actual_reverse == expected_reverse) ? "PASS" : "FAIL");
    $display();
    
    // Test vowel counting
    actual_vowels = STRING_PROCESSOR_INSTANCE.count_vowels(test_word);
    $display("Vowel Count Test:");
    $display("  Expected: %0d", expected_vowels);
    $display("  Actual:   %0d", actual_vowels);
    $display("  Result:   %s", (actual_vowels == expected_vowels) ? "PASS" : "FAIL");
    $display();
    
  endtask
  
  initial begin
    // Dump waves for analysis
    $dumpfile("string_processor_testbench.vcd");
    $dumpvars(0, string_processor_testbench);
    
    #1; // Wait for design to initialize
    
    $display("Hello from string processor testbench!");
    $display();
    
    // Run verification tests
    verify_string_functions();
    
    $display("=== Test Summary ===");
    $display("String manipulation functions tested successfully!");
    $display();
    
    #5; // Additional time before finish
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== String Manipulation Functions Demo ===

Name formatting:
  First: Alice, Last: Johnson
  Full name: Alice Johnson

File extension extraction:
  Filename: report.txt
  Extension: txt

String reversal:
  Original: SystemVerilog
  Reversed: golireVmetsyS

Vowel counting:
  Text: SystemVerilog
  Vowel count: 4

Hello from string processor testbench!

=== Testbench Verification ===

Name Format Test:
  Expected: Bob Smith
  Actual:   Bob Smith
  Result:   PASS

Extension Test:
  Expected: csv
  Actual:   csv
  Result:   PASS

Reverse Test:
  Expected: olleh
  Actual:   olleh
  Result:   PASS

Vowel Count Test:
  Expected: 2
  Actual:   2
  Result:   PASS

=== Test Summary ===
String manipulation functions tested successfully!

Process finished with return code: 0
Removing Chapter_7_examples/example_2__string_manipulation/obj_dir directory...
Chapter_7_examples/example_2__string_manipulation/obj_dir removed successfully.


0

#### Bit Manipulation Functions
Functions for bit operations, masking, shifting, and bit counting

In [3]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_3__bit_manipulation_functions/"
files = [
    "bit_manipulation_functions.sv",
    "bit_manipulation_functions_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bit_manipulation_functions.sv
module bit_manipulation_functions ();

  // Function to count number of set bits (population count)
  function automatic int count_ones(input logic [31:0] data_word);
    int bit_count = 0;
    for (int bit_index = 0; bit_index < 32; bit_index++) begin
      if (data_word[bit_index]) bit_count++;
    end
    return bit_count;
  endfunction

  // Function to reverse bits in a byte
  function automatic logic [7:0] reverse_byte_bits(input logic [7:0] byte_in);
    logic [7:0] reversed_byte;
    for (int bit_pos = 0; bit_pos < 8; bit_pos++) begin
      reversed_byte[7-bit_pos] = byte_in[bit_pos];
    end
    return reversed_byte;
  endfunction

  // Function to create bit mask with specified width and position
  function automatic logic [31:0] create_bit_mask(input int mask_width, 
                                                   input int start_position);
    logic [31:0] generated_mask = 0;
    for (int mask_bit = 0; mask_bit < mask_width; mask_bit++) begin
      generated_mask[start_position + mask_bit] = 1'b1;
    end
    return generated_mask;
  endfunction

  // Function to extract bit field from word
  function automatic logic [31:0] extract_bit_field(input logic [31:0] source_word,
                                                     input int field_width,
                                                     input int field_position);
    logic [31:0] extraction_mask;
    extraction_mask = create_bit_mask(field_width, field_position);
    return (source_word & extraction_mask) >> field_position;
  endfunction

  // Function to perform circular left shift
  function automatic logic [15:0] circular_left_shift(input logic [15:0] shift_data,
                                                       input int shift_amount);
    int effective_shift = shift_amount % 16;
    return (shift_data << effective_shift) | (shift_data >> (16 - effective_shift));
  endfunction

  initial begin
    logic [31:0] test_word = 32'hA5C3_F0E1;
    logic [7:0] test_byte = 8'b1010_0011;
    logic [15:0] rotation_data = 16'hBEEF;
    
    $display("=== Bit Manipulation Functions Demo ===");
    $display();
    
    // Test count_ones function
    $display("Count ones in 0x%08h: %0d bits", test_word, count_ones(test_word));
    
    // Test reverse_byte_bits function  
    $display("Reverse bits in 0x%02h: 0x%02h", test_byte, 
             reverse_byte_bits(test_byte));
    
    // Test create_bit_mask function
    $display("Create 4-bit mask at position 8: 0x%08h", 
             create_bit_mask(4, 8));
    
    // Test extract_bit_field function
    $display("Extract 8 bits from position 16 of 0x%08h: 0x%02h", 
             test_word, extract_bit_field(test_word, 8, 16));
    
    // Test circular_left_shift function
    $display("Circular left shift 0x%04h by 4 positions: 0x%04h", 
             rotation_data, circular_left_shift(rotation_data, 4));
    
    $display();
  end

endmodule
```

```systemverilog
// bit_manipulation_functions_testbench.sv
module bit_manipulation_functions_testbench;

  // Instantiate design under test
  bit_manipulation_functions BIT_MANIP_DUT();

  // Test variables
  logic [31:0] test_data_word;
  logic [7:0] test_input_byte;
  logic [15:0] shift_test_data;
  int expected_ones_count;
  logic [7:0] expected_reversed_byte;
  logic [31:0] expected_mask_value;
  logic [31:0] expected_extracted_field;
  logic [15:0] expected_shifted_result;

  initial begin
    // Setup waveform dump
    $dumpfile("bit_manipulation_functions_testbench.vcd");
    $dumpvars(0, bit_manipulation_functions_testbench);
    
    $display("=== Comprehensive Bit Manipulation Testing ===");
    $display();
    
    // Test Case 1: Count ones function verification
    test_data_word = 32'hFFFF_0000;  // 16 ones, 16 zeros
    expected_ones_count = 16;
    #1;
    if (BIT_MANIP_DUT.count_ones(test_data_word) == expected_ones_count) begin
      $display("PASS: count_ones(0x%08h) = %0d", 
               test_data_word, BIT_MANIP_DUT.count_ones(test_data_word));
    end else begin
      $display("FAIL: count_ones(0x%08h) = %0d, expected %0d", 
               test_data_word, BIT_MANIP_DUT.count_ones(test_data_word), 
               expected_ones_count);
    end
    
    // Test Case 2: Byte bit reversal verification
    test_input_byte = 8'b1100_0011;  // 0xC3
    expected_reversed_byte = 8'b1100_0011;  // Same when palindromic
    #1;
    if (BIT_MANIP_DUT.reverse_byte_bits(test_input_byte) == expected_reversed_byte) begin
      $display("PASS: reverse_byte_bits(0x%02h) = 0x%02h", 
               test_input_byte, BIT_MANIP_DUT.reverse_byte_bits(test_input_byte));
    end else begin
      $display("FAIL: reverse_byte_bits(0x%02h) = 0x%02h, expected 0x%02h", 
               test_input_byte, BIT_MANIP_DUT.reverse_byte_bits(test_input_byte),
               expected_reversed_byte);
    end
    
    // Test Case 3: Bit mask creation verification
    expected_mask_value = 32'h0000_0F00;  // 4 bits at position 8
    #1;
    if (BIT_MANIP_DUT.create_bit_mask(4, 8) == expected_mask_value) begin
      $display("PASS: create_bit_mask(4, 8) = 0x%08h", 
               BIT_MANIP_DUT.create_bit_mask(4, 8));
    end else begin
      $display("FAIL: create_bit_mask(4, 8) = 0x%08h, expected 0x%08h", 
               BIT_MANIP_DUT.create_bit_mask(4, 8), expected_mask_value);
    end
    
    // Test Case 4: Bit field extraction verification
    test_data_word = 32'hABCD_EF12;
    expected_extracted_field = 32'h0000_00CD;  // Bits [15:8] = 0xCD
    #1;
    if (BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8) == 
        expected_extracted_field) begin
      $display("PASS: extract_bit_field(0x%08h, 8, 8) = 0x%08h", 
               test_data_word, BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8));
    end else begin
      $display("FAIL: extract_bit_field(0x%08h, 8, 8) = 0x%08h, expected 0x%08h", 
               test_data_word, BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8),
               expected_extracted_field);
    end
    
    // Test Case 5: Circular shift verification
    shift_test_data = 16'hF000;  // 1111_0000_0000_0000
    expected_shifted_result = 16'h000F;  // After 4-bit left shift
    #1;
    if (BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4) == 
        expected_shifted_result) begin
      $display("PASS: circular_left_shift(0x%04h, 4) = 0x%04h", 
               shift_test_data, 
               BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4));
    end else begin
      $display("FAIL: circular_left_shift(0x%04h, 4) = 0x%04h, expected 0x%04h", 
               shift_test_data, 
               BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4),
               expected_shifted_result);
    end
    
    $display();
    $display("=== Bit Manipulation Testing Complete ===");
    
    #1;
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Bit Manipulation Functions Demo ===

Count ones in 0xa5c3f0e1: 16 bits
Reverse bits in 0xa3: 0xc5
Create 4-bit mask at position 8: 0x00000f00
Extract 8 bits from position 16 of 0xa5c3f0e1: 0xc3
Circular left shift 0xbeef by 4 positions: 0xeefb

=== Comprehensive Bit Manipulation Testing ===

PASS: count_ones(0xffff0000) = 16
PASS: reverse_byte_bits(0xc3) = 0xc3
PASS: create_bit_mask(4, 8) = 0x00000f00
FAIL: extract_bit_field(0xabcdef12, 8, 8) = 0x000000ef, expected 0x000000cd
PASS: circular_left_shift(0xf000, 4) = 0x000f

=== Bit Manipulation Testing Complete ===
Process finished with return code: 0
Removing Chapter_7_examples/example_3__bit_manipulation_functions/obj_dir directory...
Chapter_7_examples/example_3__bit_manipulation_functions/obj_dir removed successfully.


0

#### CRC Calculator Function
Function to calculate cyclic redundancy check for data integrity

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_4__crc_calculator/"
files = [
    "crc_calculator_design.sv",
    "crc_calculator_design_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// crc_calculator_design.sv
module crc_calculator_design_module (
  input  logic        clock_signal,
  input  logic        reset_active_low,
  input  logic        data_valid_input,
  input  logic [7:0]  data_byte_input,
  output logic [7:0]  crc_checksum_output,
  output logic        crc_calculation_complete
);

  // CRC-8 polynomial: x^8 + x^2 + x^1 + 1 (0x07)
  parameter logic [7:0] CRC_POLYNOMIAL_CONSTANT = 8'h07;
  
  // Internal registers for CRC calculation
  logic [7:0] crc_register_current;
  logic [7:0] crc_register_next;
  logic       calculation_in_progress;

  // CRC calculation logic
  always_comb begin
    crc_register_next = crc_register_current;
    
    if (data_valid_input) begin
      // XOR input data with current CRC
      crc_register_next = crc_register_current ^ data_byte_input;
      
      // Process each bit through the polynomial
      for (int bit_position = 0; bit_position < 8; bit_position++) begin
        if (crc_register_next[7]) begin
          crc_register_next = (crc_register_next << 1) ^ CRC_POLYNOMIAL_CONSTANT;
        end else begin
          crc_register_next = crc_register_next << 1;
        end
      end
    end
  end

  // Sequential logic for state updates
  always_ff @(posedge clock_signal or negedge reset_active_low) begin
    if (!reset_active_low) begin
      crc_register_current     <= 8'h00;
      calculation_in_progress  <= 1'b0;
      crc_calculation_complete <= 1'b0;
    end else begin
      crc_register_current <= crc_register_next;
      
      if (data_valid_input) begin
        calculation_in_progress  <= 1'b1;
        crc_calculation_complete <= 1'b0;
      end else if (calculation_in_progress) begin
        calculation_in_progress  <= 1'b0;
        crc_calculation_complete <= 1'b1;
      end else begin
        crc_calculation_complete <= 1'b0;
      end
    end
  end

  // Output assignment
  assign crc_checksum_output = crc_register_current;

  // Display messages for debugging
  initial $display("CRC Calculator Design Module Initialized");
  
  always @(posedge crc_calculation_complete) begin
    $display("CRC calculation complete! Checksum: 0x%02h", 
             crc_checksum_output);
  end

endmodule
```

```systemverilog
// crc_calculator_design_testbench.sv
module crc_calculator_testbench_module;

  // Testbench signals with descriptive names
  logic        clock_signal_tb;
  logic        reset_active_low_tb;
  logic        data_valid_input_tb;
  logic [7:0]  data_byte_input_tb;
  logic [7:0]  crc_checksum_output_tb;
  logic        crc_calculation_complete_tb;

  // Test data array for CRC calculation
  logic [7:0] test_data_array [4] = '{8'hAB, 8'hCD, 8'hEF, 8'h12};
  integer     data_index_counter;

  // Instantiate the CRC calculator design under test
  crc_calculator_design_module CRC_CALCULATOR_INSTANCE (
    .clock_signal              (clock_signal_tb),
    .reset_active_low          (reset_active_low_tb),
    .data_valid_input          (data_valid_input_tb),
    .data_byte_input           (data_byte_input_tb),
    .crc_checksum_output       (crc_checksum_output_tb),
    .crc_calculation_complete  (crc_calculation_complete_tb)
  );

  // Clock generation - 10ns period
  always #5 clock_signal_tb = ~clock_signal_tb;

  // Main test sequence
  initial begin
    // Initialize waveform dump
    $dumpfile("crc_calculator_testbench_module.vcd");
    $dumpvars(0, crc_calculator_testbench_module);
    
    // Display test start message
    $display();
    $display("=== CRC Calculator Testbench Starting ===");
    $display();

    // Initialize all signals
    clock_signal_tb      = 1'b0;
    reset_active_low_tb  = 1'b1;
    data_valid_input_tb  = 1'b0;
    data_byte_input_tb   = 8'h00;
    data_index_counter   = 0;

    // Apply reset sequence
    $display("Applying reset sequence...");
    #10 reset_active_low_tb = 1'b0;  // Assert reset
    #20 reset_active_low_tb = 1'b1;  // Release reset
    #10;

    // Test CRC calculation with sample data
    $display("Starting CRC calculation with test data:");
    for (data_index_counter = 0; data_index_counter < 4; 
         data_index_counter++) begin
      
      $display("Processing byte %0d: 0x%02h", 
               data_index_counter, test_data_array[data_index_counter]);
      
      // Present data to CRC calculator
      data_byte_input_tb  = test_data_array[data_index_counter];
      data_valid_input_tb = 1'b1;
      #10;
      
      // Remove data valid signal
      data_valid_input_tb = 1'b0;
      #10;
      
      // Wait for calculation completion
      wait (crc_calculation_complete_tb);
      #5;
    end

    // Final CRC result display
    $display();
    $display("Final CRC checksum result: 0x%02h", crc_checksum_output_tb);
    $display();

    // Test reset functionality
    $display("Testing reset functionality...");
    #10 reset_active_low_tb = 1'b0;
    #10 reset_active_low_tb = 1'b1;
    #10;
    
    $display("CRC after reset: 0x%02h", crc_checksum_output_tb);
    $display();

    // Complete testbench execution
    $display("=== CRC Calculator Testbench Complete ===");
    $display();
    #50;
    $finish;
  end

  // Monitor for important signal changes
  initial begin
    forever begin
      @(posedge crc_calculation_complete_tb);
      $display("Time %0t: CRC calculation completed, result = 0x%02h", 
               $time, crc_checksum_output_tb);
    end
  end

endmodule
```

Verilator Simulation Output:
CRC Calculator Design Module Initialized

=== CRC Calculator Testbench Starting ===

Applying reset sequence...
Starting CRC calculation with test data:
Processing byte 0: 0xab
Time 55: CRC calculation completed, result = 0x58
CRC calculation complete! Checksum: 0x58
Processing byte 1: 0xcd
Time 75: CRC calculation completed, result = 0xe2
CRC calculation complete! Checksum: 0xe2
Processing byte 2: 0xef
Time 105: CRC calculation completed, result = 0x23
CRC calculation complete! Checksum: 0x23
Processing byte 3: 0x12
Time 125: CRC calculation completed, result = 0x97
CRC calculation complete! Checksum: 0x97

Final CRC checksum result: 0x97

Testing reset functionality...
CRC after reset: 0x00

=== CRC Calculator Testbench Complete ===

Process finished with return code: 0
Removing Chapter_7_examples/example_4__crc_calculator/obj_dir directory...
Chapter_7_examples/example_4__crc_calculator/obj_dir removed successfully.


0

#### Lookup Table Function
Function implementing ROM or lookup table functionality

In [2]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_5__lookup_table_function/"
files = ["lookup_table_rom.sv", "lookup_table_rom_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// lookup_table_rom.sv
module lookup_table_rom (
    input  logic [2:0] address_input,           // 3-bit address for 8 entries
    output logic [7:0] data_output              // 8-bit data output
);

    // ROM lookup table with 8 entries of 8-bit data
    logic [7:0] rom_memory [0:7];

    // Initialize ROM with predefined values
    initial begin
        rom_memory[0] = 8'h10;  // Address 0: 0x10
        rom_memory[1] = 8'h25;  // Address 1: 0x25
        rom_memory[2] = 8'h3A;  // Address 2: 0x3A
        rom_memory[3] = 8'h47;  // Address 3: 0x47
        rom_memory[4] = 8'h5C;  // Address 4: 0x5C
        rom_memory[5] = 8'h69;  // Address 5: 0x69
        rom_memory[6] = 8'h7E;  // Address 6: 0x7E
        rom_memory[7] = 8'h83;  // Address 7: 0x83
    end

    // Combinational lookup - output changes immediately with address
    assign data_output = rom_memory[address_input];

endmodule
```

```systemverilog
// lookup_table_rom_testbench.sv
module lookup_table_rom_testbench;
    
    // Testbench signals
    logic [2:0] test_address;                   // Address input signal
    logic [7:0] expected_data;                  // Expected data output
    logic [7:0] actual_data;                    // Actual data from ROM

    // Instantiate the design under test
    lookup_table_rom ROM_INSTANCE (
        .address_input(test_address),
        .data_output(actual_data)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("lookup_table_rom_testbench.vcd");
        $dumpvars(0, lookup_table_rom_testbench);
        
        $display();
        $display("=== ROM Lookup Table Test Started ===");
        $display();

        // Test all ROM addresses
        for (int addr = 0; addr < 8; addr++) begin
            test_address = addr[2:0];           // Set address
            #1;                                 // Wait for combinational delay
            
            // Display results
            $display("Address: %0d (0x%h) -> Data: 0x%h", 
                     test_address, test_address, actual_data);
        end

        $display();
        $display("=== Verification Test ===");
        
        // Verify specific values
        test_address = 3'b000; #1;
        if (actual_data !== 8'h10) 
            $display(
                "ERROR: Address 0 expected 0x10, got 0x%h", actual_data);
        
        test_address = 3'b011; #1;
        if (actual_data !== 8'h47) 
            $display(
                "ERROR: Address 3 expected 0x47, got 0x%h", actual_data);
        
        test_address = 3'b111; #1;
        if (actual_data !== 8'h83) 
            $display(
                "ERROR: Address 7 expected 0x83, got 0x%h", actual_data);

        $display();
        $display("=== ROM Lookup Table Test Completed ===");
        $display();
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:

=== ROM Lookup Table Test Started ===

Address: 0 (0x0) -> Data: 0x10
Address: 1 (0x1) -> Data: 0x25
Address: 2 (0x2) -> Data: 0x3a
Address: 3 (0x3) -> Data: 0x47
Address: 4 (0x4) -> Data: 0x5c
Address: 5 (0x5) -> Data: 0x69
Address: 6 (0x6) -> Data: 0x7e
Address: 7 (0x7) -> Data: 0x83

=== Verification Test ===

=== ROM Lookup Table Test Completed ===

Process finished with return code: 0
Removing Chapter_7_examples/example_5__lookup_table_function/obj_dir directory...
Chapter_7_examples/example_5__lookup_table_function/obj_dir removed successfully.


0

#### Data Conversion Functions
Functions for converting between different data types and formats

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_6__data_conversion_functions/"
files = ["data_converter_unit.sv", "data_converter_unit_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_converter_unit.sv
module data_converter_unit();                     // Data conversion functions

  // Function to convert binary to BCD (Binary Coded Decimal)
  function automatic [7:0] binary_to_bcd(input [7:0] binary_value);
    reg [7:0] bcd_result;
    reg [7:0] temp_value;
    integer digit_index;
    begin
      bcd_result = 8'b0;
      temp_value = binary_value;
      
      // Convert using double dabble algorithm (simplified for 8-bit)
      for (digit_index = 0; digit_index < 8; digit_index++) begin
        // Add 3 to BCD digits >= 5 before shifting
        if (bcd_result[3:0] >= 5) bcd_result[3:0] = bcd_result[3:0] + 3;
        if (bcd_result[7:4] >= 5) bcd_result[7:4] = bcd_result[7:4] + 3;
        
        // Shift left and bring in next binary bit
        bcd_result = {bcd_result[6:0], temp_value[7]};
        temp_value = temp_value << 1;
      end
      
      binary_to_bcd = bcd_result;
    end
  endfunction

  // Function to convert signed to unsigned with overflow detection
  function automatic [8:0] signed_to_unsigned_safe(input signed [7:0] 
                                                   signed_input);
    reg overflow_flag;
    reg [7:0] unsigned_result;
    begin
      if (signed_input < 0) begin
        overflow_flag = 1'b1;
        unsigned_result = 8'b0;  // Clamp to zero for negative values
      end else begin
        overflow_flag = 1'b0;
        unsigned_result = signed_input;
      end
      
      // Return {overflow_flag, unsigned_result}
      signed_to_unsigned_safe = {overflow_flag, unsigned_result};
    end
  endfunction

  // Function to convert temperature from Celsius to Fahrenheit
  function automatic [15:0] celsius_to_fahrenheit(input signed [7:0] 
                                                  celsius_temp);
    reg signed [15:0] fahrenheit_result;
    begin
      // F = (C * 9/5) + 32, using fixed point arithmetic
      fahrenheit_result = (celsius_temp * 9) / 5 + 32;
      celsius_to_fahrenheit = fahrenheit_result;
    end
  endfunction

  // Function to pack RGB values into single word
  function automatic [23:0] pack_rgb_color(input [7:0] red_channel,
                                           input [7:0] green_channel,
                                           input [7:0] blue_channel);
    begin
      pack_rgb_color = {red_channel, green_channel, blue_channel};
    end
  endfunction

  // Function to extract RGB components from packed color
  function automatic [23:0] unpack_rgb_color(input [23:0] packed_color,
                                             input [1:0] channel_select);
    begin
      case (channel_select)
        2'b00: unpack_rgb_color = {16'b0, packed_color[23:16]}; // Red
        2'b01: unpack_rgb_color = {16'b0, packed_color[15:8]};  // Green
        2'b10: unpack_rgb_color = {16'b0, packed_color[7:0]};   // Blue
        2'b11: unpack_rgb_color = packed_color;                 // All
      endcase
    end
  endfunction

  initial begin
    $display();
    $display("Data Conversion Functions Demonstration");
    $display("=========================================");
    $display();
  end

endmodule
```

```systemverilog
// data_converter_unit_testbench.sv
module data_conversion_testbench;             // Data conversion testbench

  // Instantiate the data converter unit
  data_converter_unit CONVERTER_INSTANCE();

  // Test variables
  reg [7:0] test_binary_value;
  reg signed [7:0] test_signed_value;
  reg signed [7:0] test_celsius_temp;
  reg [7:0] test_red, test_green, test_blue;
  reg [23:0] test_packed_color;
  integer test_channel_select;

  // Result variables
  reg [7:0] bcd_result;
  reg [8:0] unsigned_result;
  reg [15:0] fahrenheit_result;
  reg [23:0] packed_result;
  reg [23:0] unpacked_result;

  initial begin
    // Setup waveform dumping
    $dumpfile("data_conversion_testbench.vcd");
    $dumpvars(0, data_conversion_testbench);
    
    #1;  // Wait for initialization
    
    $display("Testing Binary to BCD Conversion:");
    $display("================================");
    
    // Test binary to BCD conversion
    test_binary_value = 8'd42;
    bcd_result = CONVERTER_INSTANCE.binary_to_bcd(test_binary_value);
    $display(
        "Binary %d converts to BCD 0x%h", test_binary_value, bcd_result);
    
    test_binary_value = 8'd99;
    bcd_result = CONVERTER_INSTANCE.binary_to_bcd(test_binary_value);
    $display(
        "Binary %d converts to BCD 0x%h", test_binary_value, bcd_result);
    
    $display();
    $display("Testing Signed to Unsigned Conversion:");
    $display("=====================================");
    
    // Test signed to unsigned conversion
    test_signed_value = -25;
    unsigned_result = CONVERTER_INSTANCE.signed_to_unsigned_safe(
                      test_signed_value);
    $display(
        "Signed %d converts to unsigned %d (overflow: %b)", 
        test_signed_value, unsigned_result[7:0], unsigned_result[8]);
    
    test_signed_value = 100;
    unsigned_result = CONVERTER_INSTANCE.signed_to_unsigned_safe(
                      test_signed_value);
    $display(
        "Signed %d converts to unsigned %d (overflow: %b)", 
        test_signed_value, unsigned_result[7:0], unsigned_result[8]);
    
    $display();
    $display("Testing Temperature Conversion:");
    $display("==============================");
    
    // Test temperature conversion
    test_celsius_temp = 0;
    fahrenheit_result = CONVERTER_INSTANCE.celsius_to_fahrenheit(
                        test_celsius_temp);
    $display(
        "Temperature %d Celsius converts to %d Fahrenheit", 
        test_celsius_temp, fahrenheit_result);
    
    test_celsius_temp = 25;
    fahrenheit_result = CONVERTER_INSTANCE.celsius_to_fahrenheit(
                        test_celsius_temp);
    $display(
        "Temperature %d Celsius converts to %d Fahrenheit", 
        test_celsius_temp, fahrenheit_result);
    
    $display();
    $display("Testing RGB Color Packing/Unpacking:");
    $display("===================================");
    
    // Test RGB color packing
    test_red = 8'hFF;    // Maximum red
    test_green = 8'h80;  // Medium green
    test_blue = 8'h40;   // Low blue
    
    packed_result = CONVERTER_INSTANCE.pack_rgb_color(test_red, test_green, 
                                                      test_blue);
    $display("RGB(%h, %h, %h) packs to 0x%h", test_red, test_green, 
             test_blue, packed_result);
    
    // Test RGB color unpacking
    test_packed_color = 24'hFF8040;
    
    for (test_channel_select = 0; test_channel_select < 4; 
         test_channel_select = test_channel_select + 1) begin
      unpacked_result = CONVERTER_INSTANCE.unpack_rgb_color(
                        test_packed_color, test_channel_select[1:0]);
      case (test_channel_select[1:0])
        2'b00: $display("Red channel: 0x%h", unpacked_result[7:0]);
        2'b01: $display("Green channel: 0x%h", unpacked_result[7:0]);
        2'b10: $display("Blue channel: 0x%h", unpacked_result[7:0]);
        2'b11: $display("All channels: 0x%h", unpacked_result);
      endcase
    end
    
    $display();
    $display("Data conversion function testing completed successfully!");
    $display();

  end

endmodule
```

Verilator Simulation Output:

Data Conversion Functions Demonstration

Testing Binary to BCD Conversion:
Binary  42 converts to BCD 0x42
Binary  99 converts to BCD 0x99

Testing Signed to Unsigned Conversion:
Signed  -25 converts to unsigned   0 (overflow: 1)
Signed  100 converts to unsigned 100 (overflow: 0)

Testing Temperature Conversion:
Temperature    0 Celsius converts to    32 Fahrenheit
Temperature   25 Celsius converts to    77 Fahrenheit

Testing RGB Color Packing/Unpacking:
RGB(ff, 80, 40) packs to 0xff8040
Red channel: 0xff
Green channel: 0x80
Blue channel: 0x40
All channels: 0xff8040

Data conversion function testing completed successfully!
Process finished with return code: 0
Removing Chapter_7_examples/example_6__data_conversion_functions/obj_dir directory...
Chapter_7_examples/example_6__data_conversion_functions/obj_dir removed successfully.


0

## Functions with Different Return Types

#### Complex Number Operations
Functions returning structured data for complex arithmetic operations

In [9]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_7__complex_number_operations/"
files = [
    "complex_arithmetic_processor.sv",
    "complex_arithmetic_processor_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// complex_arithmetic_processor.sv
module complex_arithmetic_processor ();

  // Define complex number structure
  typedef struct packed {
    logic signed [15:0] real_component;
    logic signed [15:0] imaginary_component;
  } complex_number_type;

  // Function to add two complex numbers and return structured result
  function complex_number_type add_complex_numbers(
    complex_number_type first_complex_operand,
    complex_number_type second_complex_operand
  );
    complex_number_type addition_result;
    addition_result.real_component = first_complex_operand.real_component + 
                                     second_complex_operand.real_component;
    addition_result.imaginary_component = 
      first_complex_operand.imaginary_component + 
      second_complex_operand.imaginary_component;
    return addition_result;
  endfunction

  // Function to multiply two complex numbers and return structured result
  function complex_number_type multiply_complex_numbers(
    complex_number_type first_complex_operand,
    complex_number_type second_complex_operand
  );
    complex_number_type multiplication_result;
    // (a + bi) * (c + di) = (ac - bd) + (ad + bc)i
    multiplication_result.real_component = 
      (first_complex_operand.real_component * 
       second_complex_operand.real_component) -
      (first_complex_operand.imaginary_component * 
       second_complex_operand.imaginary_component);
    
    multiplication_result.imaginary_component = 
      (first_complex_operand.real_component * 
       second_complex_operand.imaginary_component) +
      (first_complex_operand.imaginary_component * 
       second_complex_operand.real_component);
    
    return multiplication_result;
  endfunction

  // Function to calculate magnitude squared (avoids square root)
  function logic [31:0] calculate_magnitude_squared(
    complex_number_type input_complex_number
  );
    return (input_complex_number.real_component * 
            input_complex_number.real_component) +
           (input_complex_number.imaginary_component * 
            input_complex_number.imaginary_component);
  endfunction

  initial begin
    complex_number_type first_number = '{real_component: 16'd3, 
                                          imaginary_component: 16'd4};
    complex_number_type second_number = '{real_component: 16'd1, 
                                           imaginary_component: 16'd2};
    complex_number_type addition_result;
    complex_number_type multiplication_result;
    logic [31:0] magnitude_squared_result;

    $display();
    $display("Complex Number Arithmetic Operations Demo");
    $display("==========================================");
    
    // Display input numbers
    $display("First Number:  %0d + %0di", 
             first_number.real_component, 
             first_number.imaginary_component);
    $display("Second Number: %0d + %0di", 
             second_number.real_component, 
             second_number.imaginary_component);
    $display();

    // Perform addition
    addition_result = add_complex_numbers(first_number, second_number);
    $display("Addition Result: %0d + %0di", 
             addition_result.real_component, 
             addition_result.imaginary_component);

    // Perform multiplication
    multiplication_result = multiply_complex_numbers(first_number, 
                                                     second_number);
    $display("Multiplication Result: %0d + %0di", 
             multiplication_result.real_component, 
             multiplication_result.imaginary_component);

    // Calculate magnitude squared of first number
    magnitude_squared_result = calculate_magnitude_squared(first_number);
    $display("Magnitude Squared of First Number: %0d", 
             magnitude_squared_result);
    $display();
  end

endmodule
```

```systemverilog
// complex_arithmetic_processor_testbench.sv
module complex_arithmetic_testbench;

  // Instantiate the design under test
  complex_arithmetic_processor complex_processor_instance();

  initial begin
    // Configure wave dumping for analysis
    $dumpfile("complex_arithmetic_testbench.vcd");
    $dumpvars(0, complex_arithmetic_testbench);
    
    #1;  // Allow design to execute
    
    $display("Testbench: Complex arithmetic operations completed");
    $display("Testbench: Check VCD file for signal analysis");
    $display();
    
    $finish;  // End simulation
  end

endmodule
```

Verilator Simulation Output:

Complex Number Arithmetic Operations Demo
First Number:  3 + 4i
Second Number: 1 + 2i

Addition Result: 4 + 6i
Multiplication Result: -5 + 10i
Magnitude Squared of First Number: 25

Testbench: Complex arithmetic operations completed
Testbench: Check VCD file for signal analysis

Process finished with return code: 0
Removing Chapter_7_examples/example_7__complex_number_operations/obj_dir directory...
Chapter_7_examples/example_7__complex_number_operations/obj_dir removed successfully.


0

#### Statistical Analysis Functions
Functions returning multiple statistical measures (mean, variance, etc.)

In [11]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_8__statistical_analysis_functions/"
files = ["statistical_analyzer.sv", "statistical_analyzer_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// statistical_analyzer.sv
module statistical_data_processor ();        // Statistical analysis module

  // Function returning multiple statistical measures
  function automatic void calculate_statistics(
    input  real data_samples[5],              // Input data array
    output real mean_value,                   // Mean output
    output real variance_value,               // Variance output
    output real std_deviation,                // Standard deviation output
    output real min_value,                    // Minimum value output
    output real max_value                     // Maximum value output
  );
    real sum_of_values;
    real sum_of_squares;
    int  sample_count;
    
    // Initialize values
    sum_of_values = 0.0;
    sum_of_squares = 0.0;
    sample_count = 5;
    min_value = data_samples[0];
    max_value = data_samples[0];
    
    // Calculate sum and find min/max
    foreach (data_samples[i]) begin
      sum_of_values += data_samples[i];
      sum_of_squares += (data_samples[i] * data_samples[i]);
      if (data_samples[i] < min_value) min_value = data_samples[i];
      if (data_samples[i] > max_value) max_value = data_samples[i];
    end
    
    // Calculate mean
    mean_value = sum_of_values / sample_count;
    
    // Calculate variance
    variance_value = (sum_of_squares / sample_count) - 
                    (mean_value * mean_value);
    
    // Calculate standard deviation
    std_deviation = $sqrt(variance_value);
    
  endfunction

  // Function to analyze data quality
  function automatic void quality_metrics(
    input  real mean_val,
    input  real std_dev,
    output string quality_rating,
    output bit   is_consistent_data
  );
    real coefficient_of_variation;
    
    coefficient_of_variation = (std_dev / mean_val) * 100.0;
    
    if (coefficient_of_variation < 10.0) begin
      quality_rating = "EXCELLENT";
      is_consistent_data = 1'b1;
    end else if (coefficient_of_variation < 25.0) begin
      quality_rating = "GOOD";
      is_consistent_data = 1'b1;
    end else begin
      quality_rating = "POOR";
      is_consistent_data = 1'b0;
    end
    
  endfunction

  initial begin
    $display();
    $display("Statistical Data Processor Ready");
  end

endmodule
```

```systemverilog
// statistical_analyzer_testbench.sv
module statistical_analyzer_testbench;       // Statistical analyzer testbench

  // Instantiate the statistical data processor
  statistical_data_processor STATS_PROCESSOR_INSTANCE();

  // Test data and result variables
  real sample_data_set[5];                   // Input data samples
  real calculated_mean;                      // Mean result
  real calculated_variance;                  // Variance result  
  real calculated_std_dev;                   // Standard deviation result
  real minimum_sample;                       // Minimum value result
  real maximum_sample;                       // Maximum value result
  string data_quality;                       // Quality assessment
  bit data_consistency_flag;                 // Consistency indicator

  initial begin
    // Configure wave dumping
    $dumpfile("statistical_analyzer_testbench.vcd");
    $dumpvars(0, statistical_analyzer_testbench);
    
    $display();
    $display("=== Statistical Analysis Functions Test ===");
    $display();
    
    // Test Case 1: Consistent data set
    sample_data_set = '{10.5, 10.2, 10.8, 10.1, 10.4};
    
    $display("Test Case 1: Analyzing consistent data samples");
    $display("Input samples: %0.1f, %0.1f, %0.1f, %0.1f, %0.1f",
             sample_data_set[0], sample_data_set[1], sample_data_set[2],
             sample_data_set[3], sample_data_set[4]);
    
    // Call statistical analysis function
    STATS_PROCESSOR_INSTANCE.calculate_statistics(
      sample_data_set,
      calculated_mean,
      calculated_variance,
      calculated_std_dev,
      minimum_sample,
      maximum_sample
    );
    
    // Display statistical results
    $display("  Mean value:        %0.3f", calculated_mean);
    $display("  Variance:          %0.6f", calculated_variance);
    $display("  Standard deviation: %0.6f", calculated_std_dev);
    $display("  Minimum sample:    %0.1f", minimum_sample);
    $display("  Maximum sample:    %0.1f", maximum_sample);
    
    // Assess data quality
    STATS_PROCESSOR_INSTANCE.quality_metrics(
      calculated_mean,
      calculated_std_dev,
      data_quality,
      data_consistency_flag
    );
    
    $display("  Quality rating:    %s", data_quality);
    $display("  Data consistent:   %s", 
             data_consistency_flag ? "YES" : "NO");
    $display();
    
    #5;  // Wait between test cases
    
    // Test Case 2: Variable data set
    sample_data_set = '{5.0, 15.0, 8.0, 20.0, 12.0};
    
    $display("Test Case 2: Analyzing variable data samples");
    $display("Input samples: %0.1f, %0.1f, %0.1f, %0.1f, %0.1f",
             sample_data_set[0], sample_data_set[1], sample_data_set[2],
             sample_data_set[3], sample_data_set[4]);
    
    // Rerun analysis with new data
    STATS_PROCESSOR_INSTANCE.calculate_statistics(
      sample_data_set,
      calculated_mean,
      calculated_variance,
      calculated_std_dev,
      minimum_sample,
      maximum_sample
    );
    
    $display("  Mean value:        %0.3f", calculated_mean);
    $display("  Variance:          %0.6f", calculated_variance);
    $display("  Standard deviation: %0.6f", calculated_std_dev);
    $display("  Minimum sample:    %0.1f", minimum_sample);
    $display("  Maximum sample:    %0.1f", maximum_sample);
    
    STATS_PROCESSOR_INSTANCE.quality_metrics(
      calculated_mean,
      calculated_std_dev,
      data_quality,
      data_consistency_flag
    );
    
    $display("  Quality rating:    %s", data_quality);
    $display("  Data consistent:   %s", 
             data_consistency_flag ? "YES" : "NO");
    $display();
    
    $display("Statistical analysis testing completed!");
    $display();
    
  end

endmodule
```

Verilator Simulation Output:

Statistical Data Processor Ready

=== Statistical Analysis Functions Test ===

Test Case 1: Analyzing consistent data samples
Input samples: 10.5, 10.2, 10.8, 10.1, 10.4
  Mean value:        10.400
  Variance:          0.060000
  Standard deviation: 0.244949
  Minimum sample:    10.1
  Maximum sample:    10.8
  Quality rating:    EXCELLENT
  Data consistent:   YES

Test Case 2: Analyzing variable data samples
Input samples: 5.0, 15.0, 8.0, 20.0, 12.0
  Mean value:        12.000
  Variance:          27.600000
  Standard deviation: 5.253570
  Minimum sample:    5.0
  Maximum sample:    20.0
  Quality rating:    POOR
  Data consistent:    NO

Statistical analysis testing completed!
Process finished with return code: 0
Removing Chapter_7_examples/example_8__statistical_analysis_functions/obj_dir directory...
Chapter_7_examples/example_8__statistical_analysis_functions/obj_dir removed successfully.


0

#### Vector Operations
Functions for vector mathematics returning arrays or packed structures

In [13]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_9__vector_operations/"
files = ["vector_math_operations.sv", "vector_math_operations_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// vector_math_operations.sv
module vector_math_calculator();

  // Define packed structure for 3D vector
  typedef struct packed {
    logic signed [15:0] x_component;
    logic signed [15:0] y_component;
    logic signed [15:0] z_component;
  } vector_3d_packed_t;

  // Define array type for vector operations
  typedef logic signed [15:0] vector_array_t [2:0];

  // Function: Add two 3D vectors returning packed structure
  function vector_3d_packed_t add_vectors_packed(
    input vector_3d_packed_t first_vector,
    input vector_3d_packed_t second_vector
  );
    vector_3d_packed_t result_vector;
    result_vector.x_component = first_vector.x_component + 
                               second_vector.x_component;
    result_vector.y_component = first_vector.y_component + 
                               second_vector.y_component;
    result_vector.z_component = first_vector.z_component + 
                               second_vector.z_component;
    return result_vector;
  endfunction

  // Function: Multiply vector by scalar returning array
  function vector_array_t scale_vector_array(
    input vector_array_t input_vector,
    input logic signed [7:0] scale_factor
  );
    vector_array_t scaled_result;
    scaled_result[0] = input_vector[0] * scale_factor;
    scaled_result[1] = input_vector[1] * scale_factor;
    scaled_result[2] = input_vector[2] * scale_factor;
    return scaled_result;
  endfunction

  // Function: Calculate dot product returning single value
  function logic signed [31:0] compute_dot_product(
    input vector_array_t vector_a,
    input vector_array_t vector_b
  );
    logic signed [31:0] dot_product_result;
    dot_product_result = (vector_a[0] * vector_b[0]) +
                        (vector_a[1] * vector_b[1]) +
                        (vector_a[2] * vector_b[2]);
    return dot_product_result;
  endfunction

  // Function: Convert packed to array format
  function vector_array_t packed_to_array_converter(
    input vector_3d_packed_t packed_vector
  );
    vector_array_t array_result;
    array_result[0] = packed_vector.x_component;
    array_result[1] = packed_vector.y_component;
    array_result[2] = packed_vector.z_component;
    return array_result;
  endfunction

  initial begin
    $display("Vector Operations Functions Module Loaded");
    $display("Supports: Addition, Scaling, Dot Product, Conversion");
  end

endmodule
```

```systemverilog
// vector_math_operations_testbench.sv
module vector_math_testbench;

  // Instantiate design under test
  vector_math_calculator VECTOR_CALC_INSTANCE();

  // Import types from design module
  typedef struct packed {
    logic signed [15:0] x_component;
    logic signed [15:0] y_component;
    logic signed [15:0] z_component;
  } vector_3d_packed_t;

  typedef logic signed [15:0] vector_array_t [2:0];

  // Test variables
  vector_3d_packed_t first_test_vector, second_test_vector;
  vector_3d_packed_t addition_result;
  vector_array_t array_vector_a, array_vector_b;
  vector_array_t scaling_result;
  logic signed [31:0] dot_product_value;
  logic signed [7:0] multiplication_factor;

  initial begin
    // Setup VCD dumping
    $dumpfile("vector_math_testbench.vcd");
    $dumpvars(0, vector_math_testbench);

    $display("=== Vector Mathematics Operations Testbench ===");
    $display();

    // Test 1: Vector addition with packed structures
    $display("Test 1: Adding packed vectors");
    first_test_vector.x_component = 16'd10;
    first_test_vector.y_component = 16'd20;
    first_test_vector.z_component = 16'd30;
    
    second_test_vector.x_component = 16'd5;
    second_test_vector.y_component = 16'd15;
    second_test_vector.z_component = 16'd25;

    addition_result = VECTOR_CALC_INSTANCE.add_vectors_packed(
      first_test_vector, second_test_vector);
    
    $display("Vector A: (%0d, %0d, %0d)", 
             first_test_vector.x_component,
             first_test_vector.y_component,
             first_test_vector.z_component);
    $display("Vector B: (%0d, %0d, %0d)", 
             second_test_vector.x_component,
             second_test_vector.y_component,
             second_test_vector.z_component);
    $display("Sum Result: (%0d, %0d, %0d)", 
             addition_result.x_component,
             addition_result.y_component,
             addition_result.z_component);
    $display();

    // Test 2: Vector scaling with arrays
    $display("Test 2: Scaling vector using arrays");
    array_vector_a[0] = 16'd4;
    array_vector_a[1] = 16'd6;
    array_vector_a[2] = 16'd8;
    multiplication_factor = 8'd3;

    scaling_result = VECTOR_CALC_INSTANCE.scale_vector_array(
      array_vector_a, multiplication_factor);

    $display("Original Vector: [%0d, %0d, %0d]", 
             array_vector_a[0], array_vector_a[1], array_vector_a[2]);
    $display("Scale Factor: %0d", multiplication_factor);
    $display("Scaled Result: [%0d, %0d, %0d]", 
             scaling_result[0], scaling_result[1], scaling_result[2]);
    $display();

    // Test 3: Dot product calculation
    $display("Test 3: Computing dot product");
    array_vector_b[0] = 16'd2;
    array_vector_b[1] = 16'd3;
    array_vector_b[2] = 16'd4;

    dot_product_value = VECTOR_CALC_INSTANCE.compute_dot_product(
      array_vector_a, array_vector_b);

    $display("Vector A: [%0d, %0d, %0d]", 
             array_vector_a[0], array_vector_a[1], array_vector_a[2]);
    $display("Vector B: [%0d, %0d, %0d]", 
             array_vector_b[0], array_vector_b[1], array_vector_b[2]);
    $display("Dot Product: %0d", dot_product_value);
    $display();

    // Test 4: Format conversion
    $display("Test 4: Converting packed to array format");
    array_vector_a = VECTOR_CALC_INSTANCE.packed_to_array_converter(
      first_test_vector);
    
    $display("Packed Vector: (%0d, %0d, %0d)", 
             first_test_vector.x_component,
             first_test_vector.y_component,
             first_test_vector.z_component);
    $display("Converted Array: [%0d, %0d, %0d]", 
             array_vector_a[0], array_vector_a[1], array_vector_a[2]);

    $display();
    $display("=== All Vector Operations Tests Completed ===");

    #10 $finish;
  end

endmodule
```

Verilator Simulation Output:
Vector Operations Functions Module Loaded
Supports: Addition, Scaling, Dot Product, Conversion
=== Vector Mathematics Operations Testbench ===

Test 1: Adding packed vectors
Vector A: (10, 20, 30)
Vector B: (5, 15, 25)
Sum Result: (15, 35, 55)

Test 2: Scaling vector using arrays
Original Vector: [4, 6, 8]
Scale Factor: 3
Scaled Result: [12, 18, 24]

Test 3: Computing dot product
Vector A: [4, 6, 8]
Vector B: [2, 3, 4]
Dot Product: 58

Test 4: Converting packed to array format
Packed Vector: (10, 20, 30)
Converted Array: [10, 20, 30]

=== All Vector Operations Tests Completed ===
Process finished with return code: 0
Removing Chapter_7_examples/example_9__vector_operations/obj_dir directory...
Chapter_7_examples/example_9__vector_operations/obj_dir removed successfully.


0

#### Packet Parser Function
Function parsing network packets and returning structured information

In [15]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_10__packet_parser_function/"
files = ["packet_parser_module.sv", "packet_parser_module_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// packet_parser_module.sv
module packet_parser_module ();
  
  // Packet header structure
  typedef struct packed {
    logic [7:0]  protocol_type;
    logic [15:0] source_port;
    logic [15:0] destination_port;
    logic [15:0] packet_length;
    logic [7:0]  checksum;
  } packet_header_t;
  
  // Function to parse network packet and extract header information
  function packet_header_t parse_network_packet(input [63:0] raw_packet_data);
    packet_header_t parsed_header;
    
    // Extract fields from raw packet data
    parsed_header.protocol_type    = raw_packet_data[63:56];
    parsed_header.source_port      = raw_packet_data[55:40];
    parsed_header.destination_port = raw_packet_data[39:24];
    parsed_header.packet_length    = raw_packet_data[23:8];
    parsed_header.checksum         = raw_packet_data[7:0];
    
    return parsed_header;
  endfunction
  
  initial begin
    logic [63:0] incoming_packet;
    packet_header_t extracted_header;
    
    $display("=== Network Packet Parser Function Demo ===");
    $display();
    
    // Simulate incoming network packet
    incoming_packet = 64'h06_1F90_0050_0400_A5;
    
    // Parse the packet using the function
    extracted_header = parse_network_packet(incoming_packet);
    
    // Display parsed results
    $display("Raw packet data:     0x%016h", incoming_packet);
    $display("Protocol type:       0x%02h", extracted_header.protocol_type);
    $display("Source port:         %0d", extracted_header.source_port);
    $display("Destination port:    %0d", extracted_header.destination_port);
    $display("Packet length:       %0d bytes", extracted_header.packet_length);
    $display("Checksum:            0x%02h", extracted_header.checksum);
    $display();
  end
  
endmodule
```

```systemverilog
// packet_parser_module_testbench.sv
module packet_parser_testbench;
  
  // Instantiate the packet parser module
  packet_parser_module PACKET_PARSER_INSTANCE();
  
  // Additional testbench variables
  logic [63:0] test_packets [];
  
  initial begin
    // Setup VCD file for waveform viewing
    $dumpfile("packet_parser_testbench.vcd");
    $dumpvars(0, packet_parser_testbench);
    
    $display("=== Packet Parser Function Testbench ===");
    $display();
    
    // Initialize test packet array
    test_packets = new[3];
    test_packets[0] = 64'h11_1F40_0050_0200_3C;  // UDP packet
    test_packets[1] = 64'h06_0050_1F90_0800_7F;  // TCP packet  
    test_packets[2] = 64'h01_0000_0000_001C_FF;  // ICMP packet
    
    // Test multiple packets
    for (int packet_index = 0; packet_index < 3; packet_index++) begin
      $display("--- Testing Packet %0d ---", packet_index + 1);
      test_packet_parsing(test_packets[packet_index]);
      $display();
      #10;  // Wait between tests
    end
    
    $display("Packet parser function testing completed!");
    $display();
    $finish;
  end
  
  // Task to test packet parsing with different inputs
  task test_packet_parsing(input [63:0] test_packet_data);
    // Call the parser function from the design module
    $display("Testing packet: 0x%016h", test_packet_data);
    
    // Note: In a real testbench, we would instantiate the design
    // and call its functions. For this simple example, we demonstrate
    // the concept of testing the packet parsing functionality.
    $display("Packet successfully processed by parser function");
  endtask
  
endmodule
```

Verilator Simulation Output:
=== Network Packet Parser Function Demo ===

Raw packet data:     0x061f9000500400a5
Protocol type:       0x06
Source port:         8080
Destination port:    80
Packet length:       1024 bytes
Checksum:            0xa5

=== Packet Parser Function Testbench ===

--- Testing Packet 1 ---
Testing packet: 0x111f40005002003c
Packet successfully processed by parser function

--- Testing Packet 2 ---
Testing packet: 0x0600501f9008007f
Packet successfully processed by parser function

--- Testing Packet 3 ---
Testing packet: 0x0100000000001cff
Packet successfully processed by parser function

Packet parser function testing completed!

Process finished with return code: 0
Removing Chapter_7_examples/example_10__packet_parser_function/obj_dir directory...
Chapter_7_examples/example_10__packet_parser_function/obj_dir removed successfully.


0

#### Color Space Converter
Function converting between RGB, HSV, and other color representations

In [19]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_11__color_space_converter/"
files = ["color_space_converter.sv", "color_space_converter_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// color_space_converter.sv
module rgb_to_hsv_converter (
    input  logic        clk,
    input  logic        reset_n,
    input  logic        convert_enable,
    input  logic [7:0]  red_channel,
    input  logic [7:0]  green_channel,
    input  logic [7:0]  blue_channel,
    output logic [8:0]  hue_output,        // 0-359 degrees
    output logic [7:0]  saturation_output, // 0-255 (0-100%)
    output logic [7:0]  value_output,      // 0-255 (0-100%)
    output logic        conversion_ready
);

    // Internal registers for RGB values
    logic [7:0] rgb_max, rgb_min, rgb_delta;
    logic [7:0] red_reg, green_reg, blue_reg;
    
    // State machine for conversion process
    typedef enum logic [1:0] {
        IDLE = 2'b00,
        CALCULATE = 2'b01,
        READY = 2'b10
    } converter_state_t;
    
    converter_state_t current_state, next_state;

    // Find maximum and minimum RGB values
    always_comb begin
        rgb_max = (red_reg >= green_reg) ? 
                  ((red_reg >= blue_reg) ? red_reg : blue_reg) :
                  ((green_reg >= blue_reg) ? green_reg : blue_reg);
        
        rgb_min = (red_reg <= green_reg) ? 
                  ((red_reg <= blue_reg) ? red_reg : blue_reg) :
                  ((green_reg <= blue_reg) ? green_reg : blue_reg);
        
        rgb_delta = rgb_max - rgb_min;
    end

    // State machine sequential logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            current_state <= IDLE;
            red_reg <= 8'h0;
            green_reg <= 8'h0;
            blue_reg <= 8'h0;
        end else begin
            current_state <= next_state;
            if (convert_enable && current_state == IDLE) begin
                red_reg <= red_channel;
                green_reg <= green_channel;
                blue_reg <= blue_channel;
            end
        end
    end

    // State machine combinational logic
    always_comb begin
        next_state = current_state;
        case (current_state)
            IDLE: begin
                if (convert_enable)
                    next_state = CALCULATE;
            end
            CALCULATE: begin
                next_state = READY;
            end
            READY: begin
                if (!convert_enable)
                    next_state = IDLE;
            end
            default: begin
                next_state = IDLE;
            end
        endcase
    end

    // HSV calculation (simplified for demonstration)
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            hue_output <= 9'h0;
            saturation_output <= 8'h0;
            value_output <= 8'h0;
            conversion_ready <= 1'b0;
        end else begin
            case (current_state)
                CALCULATE: begin
                    // Value is simply the maximum RGB component
                    value_output <= rgb_max;
                    
                    // Saturation calculation
                    if (rgb_max == 0)
                        saturation_output <= 8'h0;
                    else
                        saturation_output <= (rgb_delta * 8'd255) / rgb_max;
                    
                    // Simplified hue calculation (demonstration only)
                    if (rgb_delta == 0) begin
                        hue_output <= 9'h0;  // Undefined, set to 0
                    end else if (rgb_max == red_reg) begin
                        hue_output <= 9'd60;  // Red dominant - simplified
                    end else if (rgb_max == green_reg) begin
                        hue_output <= 9'd120; // Green dominant - simplified
                    end else begin
                        hue_output <= 9'd240; // Blue dominant - simplified
                    end
                    
                    conversion_ready <= 1'b1;
                end
                IDLE: begin
                    conversion_ready <= 1'b0;
                end
                READY: begin
                    // Maintain current values
                end
                default: begin
                    hue_output <= 9'h0;
                    saturation_output <= 8'h0;
                    value_output <= 8'h0;
                    conversion_ready <= 1'b0;
                end
            endcase
        end
    end

endmodule
```

```systemverilog
// color_space_converter_testbench.sv
module color_converter_testbench;

    // Clock and reset signals
    logic        system_clock;
    logic        reset_signal;
    
    // Input signals to design under test
    logic        conversion_trigger;
    logic [7:0]  input_red_value;
    logic [7:0]  input_green_value;
    logic [7:0]  input_blue_value;
    
    // Output signals from design under test
    logic [8:0]  output_hue_degrees;
    logic [7:0]  output_saturation_percent;
    logic [7:0]  output_value_brightness;
    logic        output_conversion_complete;

    // Instantiate the RGB to HSV converter
    rgb_to_hsv_converter COLOR_CONVERTER_INSTANCE (
        .clk(system_clock),
        .reset_n(reset_signal),
        .convert_enable(conversion_trigger),
        .red_channel(input_red_value),
        .green_channel(input_green_value),
        .blue_channel(input_blue_value),
        .hue_output(output_hue_degrees),
        .saturation_output(output_saturation_percent),
        .value_output(output_value_brightness),
        .conversion_ready(output_conversion_complete)
    );

    // Clock generation
    initial begin
        system_clock = 1'b0;
        forever #5 system_clock = ~system_clock; // 100MHz clock
    end

    // Test stimulus
    initial begin
        // Initialize VCD dump
        $dumpfile("color_converter_testbench.vcd");
        $dumpvars(0, color_converter_testbench);
        
        // Initialize signals
        reset_signal = 1'b0;
        conversion_trigger = 1'b0;
        input_red_value = 8'h0;
        input_green_value = 8'h0;
        input_blue_value = 8'h0;
        
        // Display test header
        $display();
        $display("=== RGB to HSV Color Space Converter Test ===");
        $display();
        
        // Reset sequence
        #10 reset_signal = 1'b1;
        #20;
        
        // Test Case 1: Pure Red (255, 0, 0)
        $display("Test 1: Converting Pure Red RGB(255, 0, 0)");
        input_red_value = 8'd255;
        input_green_value = 8'd0;
        input_blue_value = 8'd0;
        conversion_trigger = 1'b1;
        #10;
        
        // Wait for conversion to complete
        wait(output_conversion_complete);
        #10;
        $display("  Result: H=%d, S=%d, V=%d", 
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness);
        
        conversion_trigger = 1'b0;
        #20;
        
        // Test Case 2: Pure Green (0, 255, 0)
        $display("Test 2: Converting Pure Green RGB(0, 255, 0)");
        input_red_value = 8'd0;
        input_green_value = 8'd255;
        input_blue_value = 8'd0;
        conversion_trigger = 1'b1;
        #10;
        
        wait(output_conversion_complete);
        #10;
        $display("  Result: H=%d, S=%d, V=%d", 
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness);
        
        conversion_trigger = 1'b0;
        #20;
        
        // Test Case 3: Pure Blue (0, 0, 255)
        $display("Test 3: Converting Pure Blue RGB(0, 0, 255)");
        input_red_value = 8'd0;
        input_green_value = 8'd0;
        input_blue_value = 8'd255;
        conversion_trigger = 1'b1;
        #10;
        
        wait(output_conversion_complete);
        #10;
        $display("  Result: H=%d, S=%d, V=%d", 
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness);
        
        conversion_trigger = 1'b0;
        #20;
        
        // Test Case 4: White (255, 255, 255)
        $display("Test 4: Converting White RGB(255, 255, 255)");
        input_red_value = 8'd255;
        input_green_value = 8'd255;
        input_blue_value = 8'd255;
        conversion_trigger = 1'b1;
        #10;
        
        wait(output_conversion_complete);
        #10;
        $display("  Result: H=%d, S=%d, V=%d", 
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness);
        
        conversion_trigger = 1'b0;
        #20;
        
        // Test Case 5: Gray (128, 128, 128)
        $display("Test 5: Converting Gray RGB(128, 128, 128)");
        input_red_value = 8'd128;
        input_green_value = 8'd128;
        input_blue_value = 8'd128;
        conversion_trigger = 1'b1;
        #10;
        
        wait(output_conversion_complete);
        #10;
        $display("  Result: H=%d, S=%d, V=%d", 
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness);
        
        conversion_trigger = 1'b0;
        #20;
        
        $display();
        $display("=== Color Space Conversion Tests Complete ===");
        $display();
        
        $finish;
    end

    // Monitor for debugging
    initial begin
        $monitor("Time=%0t: RGB(%d,%d,%d) -> HSV(%d,%d,%d) Ready=%b", 
                $time, input_red_value, input_green_value, input_blue_value,
                output_hue_degrees, output_saturation_percent, 
                output_value_brightness, output_conversion_complete);
    end

endmodule
```

Verilator Simulation Output:

=== RGB to HSV Color Space Converter Test ===

Time=0: RGB(  0,  0,  0) -> HSV(  0,  0,  0) Ready=0
Test 1: Converting Pure Red RGB(255, 0, 0)
Time=30: RGB(255,  0,  0) -> HSV(  0,  0,  0) Ready=0
Time=45: RGB(255,  0,  0) -> HSV( 60,  0,255) Ready=1
  Result: H= 60, S=  0, V=255
Time=65: RGB(255,  0,  0) -> HSV( 60,  0,255) Ready=0
Test 2: Converting Pure Green RGB(0, 255, 0)
Time=75: RGB(  0,255,  0) -> HSV( 60,  0,255) Ready=0
Time=85: RGB(  0,255,  0) -> HSV(120,  0,255) Ready=1
  Result: H=120, S=  0, V=255
Time=105: RGB(  0,255,  0) -> HSV(120,  0,255) Ready=0
Test 3: Converting Pure Blue RGB(0, 0, 255)
Time=115: RGB(  0,  0,255) -> HSV(120,  0,255) Ready=0
Time=125: RGB(  0,  0,255) -> HSV(240,  0,255) Ready=1
  Result: H=240, S=  0, V=255
Time=145: RGB(  0,  0,255) -> HSV(240,  0,255) Ready=0
Test 4: Converting White RGB(255, 255, 255)
Time=155: RGB(255,255,255) -> HSV(240,  0,255) Ready=0
Time=165: RGB(255,255,255) -> HSV(  0,  0,255) Ready=1
  Re

0

## Task Declarations and Calls

#### Bus Transaction Tasks
Tasks for performing read/write operations on system buses

In [4]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_12__bus_transaction_tasks/"
files = ["simple_bus_controller.sv", "simple_bus_controller_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_bus_controller.sv
module simple_bus_controller (
  input  logic        clock,
  input  logic        reset_n,
  input  logic        bus_enable,
  input  logic        write_enable,
  input  logic [7:0]  address_bus,
  input  logic [31:0] write_data_bus,
  output logic [31:0] read_data_bus,
  output logic        bus_ready
);

  // Simple memory array for demonstration
  logic [31:0] memory_array [0:255];
  logic        operation_active;

  // Bus operation state machine
  always_ff @(posedge clock or negedge reset_n) begin
    if (!reset_n) begin
      read_data_bus    <= 32'h0;
      bus_ready        <= 1'b1;
      operation_active <= 1'b0;
      // Initialize memory with test pattern
      for (int i = 0; i < 256; i++) begin
        memory_array[i] <= 32'hDEAD_0000 + i;
      end
    end else begin
      if (bus_enable && bus_ready) begin
        operation_active <= 1'b1;
        bus_ready        <= 1'b0;
        
        if (write_enable) begin
          // Write operation
          memory_array[address_bus] <= write_data_bus;
          $display("BUS WRITE: addr=0x%02h, data=0x%08h", 
                   address_bus, write_data_bus);
        end else begin
          // Read operation  
          read_data_bus <= memory_array[address_bus];
          $display("BUS READ:  addr=0x%02h, data=0x%08h",
                   address_bus, memory_array[address_bus]);
        end
      end else if (operation_active) begin
        // Complete operation after one cycle
        operation_active <= 1'b0;
        bus_ready        <= 1'b1;
      end
    end
  end

endmodule
```

```systemverilog
// simple_bus_controller_testbench.sv
module bus_transaction_testbench;

  // Clock and reset signals
  logic        system_clock;
  logic        system_reset_n;
  
  // Bus interface signals
  logic        bus_enable;
  logic        write_enable;
  logic [7:0]  address_bus;
  logic [31:0] write_data_bus;
  logic [31:0] read_data_bus;
  logic        bus_ready;
  
  // Test variables
  logic [31:0] read_result;

  // Instantiate design under test
  simple_bus_controller BUS_CONTROLLER_INSTANCE (
    .clock(system_clock),
    .reset_n(system_reset_n),
    .bus_enable(bus_enable),
    .write_enable(write_enable),
    .address_bus(address_bus),
    .write_data_bus(write_data_bus),
    .read_data_bus(read_data_bus),
    .bus_ready(bus_ready)
  );

  // Clock generation
  initial begin
    system_clock = 0;
    forever #5 system_clock = ~system_clock;
  end

  // Bus Write Transaction Task
  task automatic perform_bus_write_transaction(
    input logic [7:0]  write_address,
    input logic [31:0] write_data
  );
    begin
      $display("\n=== Starting Bus Write Transaction ===");
      $display("Target Address: 0x%02h, Write Data: 0x%08h", 
               write_address, write_data);
      
      // Wait for bus ready
      wait(bus_ready == 1'b1);
      
      // Setup write transaction
      @(posedge system_clock);
      address_bus     = write_address;
      write_data_bus  = write_data;
      write_enable    = 1'b1;
      bus_enable      = 1'b1;
      
      // Wait for operation to complete
      @(posedge system_clock);
      bus_enable      = 1'b0;
      write_enable    = 1'b0;
      
      wait(bus_ready == 1'b1);
      $display("=== Bus Write Transaction Complete ===\n");
    end
  endtask

  // Bus Read Transaction Task
  task automatic perform_bus_read_transaction(
    input  logic [7:0]  read_address,
    output logic [31:0] read_data
  );
    begin
      $display("\n=== Starting Bus Read Transaction ===");
      $display("Target Address: 0x%02h", read_address);
      
      // Wait for bus ready
      wait(bus_ready == 1'b1);
      
      // Setup read transaction
      @(posedge system_clock);
      address_bus     = read_address;
      write_enable    = 1'b0;
      bus_enable      = 1'b1;
      
      // Wait for operation to complete
      @(posedge system_clock);
      bus_enable      = 1'b0;
      
      wait(bus_ready == 1'b1);
      @(posedge system_clock);
      read_data = read_data_bus;
      
      $display("Read Data: 0x%08h", read_data);
      $display("=== Bus Read Transaction Complete ===\n");
    end
  endtask

  // Bus Read-Modify-Write Transaction Task
  task automatic perform_bus_read_modify_write_transaction(
    input logic [7:0]  target_address,
    input logic [31:0] modify_mask,
    input logic [31:0] modify_value
  );
    logic [31:0] current_data;
    logic [31:0] modified_data;
    begin
      $display("\n=== Starting Read-Modify-Write Transaction ===");
      $display("Address: 0x%02h, Mask: 0x%08h, Value: 0x%08h", 
               target_address, modify_mask, modify_value);
      
      // Read current value
      perform_bus_read_transaction(target_address, current_data);
      
      // Modify data
      modified_data = (current_data & ~modify_mask) | 
                      (modify_value & modify_mask);
      $display("Original: 0x%08h -> Modified: 0x%08h", 
               current_data, modified_data);
      
      // Write modified value back
      perform_bus_write_transaction(target_address, modified_data);
      
      $display("=== Read-Modify-Write Transaction Complete ===\n");
    end
  endtask

  // Test stimulus
  initial begin
    // Dump waves for analysis
    $dumpfile("bus_transaction_testbench.vcd");
    $dumpvars(0, bus_transaction_testbench);
    
    // Initialize signals
    system_reset_n  = 0;
    bus_enable      = 0;
    write_enable    = 0;
    address_bus     = 8'h00;
    write_data_bus  = 32'h0000_0000;
    
    $display("\n=== Bus Transaction Tasks Example ===");
    $display("Testing various bus operations using tasks\n");
    
    // Release reset
    #20;
    system_reset_n = 1;
    #10;
    
    // Test 1: Simple write operations
    $display("TEST 1: Simple Write Operations");
    perform_bus_write_transaction(8'h10, 32'hCAFE_BABE);
    perform_bus_write_transaction(8'h20, 32'h1234_5678);
    perform_bus_write_transaction(8'h30, 32'hFEED_FACE);
    
    // Test 2: Read back written data
    $display("TEST 2: Read Back Written Data");
    perform_bus_read_transaction(8'h10, read_result);
    perform_bus_read_transaction(8'h20, read_result);
    perform_bus_read_transaction(8'h30, read_result);
    
    // Test 3: Read-modify-write operations
    $display("TEST 3: Read-Modify-Write Operations");
    perform_bus_read_modify_write_transaction(8'h10, 32'h0000_FFFF, 
                                              32'h0000_DEAD);
    perform_bus_read_modify_write_transaction(8'h20, 32'hFFFF_0000, 
                                              32'hBEEF_0000);
    
    // Test 4: Verify final results
    $display("TEST 4: Final Verification");
    perform_bus_read_transaction(8'h10, read_result);
    perform_bus_read_transaction(8'h20, read_result);
    
    $display("\n=== All Bus Transaction Tests Complete ===");
    #50;
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Bus Transaction Tasks Example ===
Testing various bus operations using tasks

TEST 1: Simple Write Operations

=== Starting Bus Write Transaction ===
Target Address: 0x10, Write Data: 0xcafebabe
BUS WRITE: addr=0x10, data=0xcafebabe
=== Bus Write Transaction Complete ===


=== Starting Bus Write Transaction ===
Target Address: 0x20, Write Data: 0x12345678
BUS WRITE: addr=0x20, data=0x12345678
=== Bus Write Transaction Complete ===


=== Starting Bus Write Transaction ===
Target Address: 0x30, Write Data: 0xfeedface
BUS WRITE: addr=0x30, data=0xfeedface
=== Bus Write Transaction Complete ===

TEST 2: Read Back Written Data

=== Starting Bus Read Transaction ===
Target Address: 0x10
BUS READ:  addr=0x10, data=0xcafebabe
Read Data: 0xcafebabe
=== Bus Read Transaction Complete ===


=== Starting Bus Read Transaction ===
Target Address: 0x20
BUS READ:  addr=0x20, data=0x12345678
Read Data: 0x12345678
=== Bus Read Transaction Complete ===


=== Starting Bus 

0

#### Protocol Handler Tasks
Tasks implementing communication protocol sequences

In [8]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_13__protocol_handler_tasks/"
files = ["simple_spi_controller.sv", "simple_spi_controller_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_spi_controller.sv
module simple_spi_controller (
  input  logic       clock_signal,
  input  logic       reset_signal,
  input  logic       start_transaction,
  input  logic [7:0] transmit_data,
  output logic       spi_clock,
  output logic       spi_chip_select,
  output logic       spi_data_out,
  input  logic       spi_data_in,
  output logic [7:0] received_data,
  output logic       transaction_complete
);

  typedef enum logic [1:0] {
    IDLE_STATE,
    ACTIVE_STATE,
    COMPLETE_STATE
  } protocol_state_type;

  protocol_state_type current_state, next_state;
  logic [3:0] bit_counter;
  logic [7:0] shift_register_tx, shift_register_rx;

  // State machine for SPI protocol
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      current_state <= IDLE_STATE;
      bit_counter <= 0;
      shift_register_tx <= 0;
      shift_register_rx <= 0;
      spi_clock <= 0;
      spi_chip_select <= 1;
      spi_data_out <= 0;
      received_data <= 0;
      transaction_complete <= 0;
    end else begin
      current_state <= next_state;
      
      case (current_state)
        IDLE_STATE: begin
          spi_chip_select <= 1;
          spi_clock <= 0;
          transaction_complete <= 0;
          if (start_transaction) begin
            shift_register_tx <= transmit_data;
            bit_counter <= 0;
          end
        end
        
        ACTIVE_STATE: begin
          spi_chip_select <= 0;
          spi_clock <= ~spi_clock;
          
          if (spi_clock) begin  // Rising edge of SPI clock
            shift_register_rx <= {shift_register_rx[6:0], spi_data_in};
            bit_counter <= bit_counter + 1;
          end else begin        // Falling edge of SPI clock
            spi_data_out <= shift_register_tx[7];
            shift_register_tx <= {shift_register_tx[6:0], 1'b0};
          end
        end
        
        COMPLETE_STATE: begin
          spi_chip_select <= 1;
          spi_clock <= 0;
          received_data <= shift_register_rx;
          transaction_complete <= 1;
        end
        
        default: begin
          spi_chip_select <= 1;
          spi_clock <= 0;
          transaction_complete <= 0;
        end
      endcase
    end
  end

  // Next state logic
  always_comb begin
    case (current_state)
      IDLE_STATE: 
        next_state = start_transaction ? ACTIVE_STATE : IDLE_STATE;
      ACTIVE_STATE: 
        next_state = (bit_counter == 8) ? COMPLETE_STATE : ACTIVE_STATE;
      COMPLETE_STATE: 
        next_state = IDLE_STATE;
      default: 
        next_state = IDLE_STATE;
    endcase
  end

endmodule
```

```systemverilog
// simple_spi_controller_testbench.sv
module spi_controller_testbench;

  // Clock and reset signals
  logic system_clock;
  logic system_reset;
  
  // DUT interface signals
  logic       initiate_transfer;
  logic [7:0] data_to_send;
  logic       spi_serial_clock;
  logic       spi_chip_select_n;
  logic       spi_master_out;
  logic       spi_master_in;
  logic [7:0] data_received;
  logic       transfer_done;

  // Instantiate the design under test
  simple_spi_controller SPI_CONTROLLER_INSTANCE (
    .clock_signal(system_clock),
    .reset_signal(system_reset),
    .start_transaction(initiate_transfer),
    .transmit_data(data_to_send),
    .spi_clock(spi_serial_clock),
    .spi_chip_select(spi_chip_select_n),
    .spi_data_out(spi_master_out),
    .spi_data_in(spi_master_in),
    .received_data(data_received),
    .transaction_complete(transfer_done)
  );

  // Clock generation
  initial begin
    system_clock = 0;
    forever #5 system_clock = ~system_clock;  // 10ns period (100MHz)
  end

  // Task: Protocol handler for SPI write transaction
  task automatic spi_write_transaction(input logic [7:0] write_data);
    begin
      $display("Starting SPI write transaction with data: 0x%02X", 
               write_data);
      
      data_to_send = write_data;
      initiate_transfer = 1;
      @(posedge system_clock);
      initiate_transfer = 0;
      
      // Wait for transaction to complete
      wait(transfer_done);
      @(posedge system_clock);
      
      $display("SPI write transaction completed");
      $display("Data sent: 0x%02X, Data received: 0x%02X", 
               write_data, data_received);
    end
  endtask

  // Task: Protocol handler for SPI read transaction
  task automatic spi_read_transaction(output logic [7:0] read_data);
    begin
      $display("Starting SPI read transaction");
      
      data_to_send = 8'h00;  // Send dummy data for read
      initiate_transfer = 1;
      @(posedge system_clock);
      initiate_transfer = 0;
      
      // Wait for transaction to complete
      wait(transfer_done);
      @(posedge system_clock);
      
      read_data = data_received;
      $display("SPI read transaction completed");
      $display("Data received: 0x%02X", read_data);
    end
  endtask

  // Task: Protocol handler for full duplex SPI transaction
  task automatic spi_full_duplex_transaction(
    input  logic [7:0] tx_data,
    output logic [7:0] rx_data
  );
    begin
      $display("Starting SPI full-duplex transaction");
      $display("Transmitting: 0x%02X", tx_data);
      
      data_to_send = tx_data;
      initiate_transfer = 1;
      @(posedge system_clock);
      initiate_transfer = 0;
      
      // Wait for transaction to complete
      wait(transfer_done);
      @(posedge system_clock);
      
      rx_data = data_received;
      $display("Full-duplex transaction completed");
      $display("Sent: 0x%02X, Received: 0x%02X", tx_data, rx_data);
    end
  endtask

  // Task: Protocol handler for system reset sequence
  task automatic system_reset_sequence();
    begin
      $display("Executing system reset sequence");
      system_reset = 1;
      initiate_transfer = 0;
      data_to_send = 8'h00;
      repeat(3) @(posedge system_clock);
      system_reset = 0;
      @(posedge system_clock);
      $display("System reset sequence completed");
    end
  endtask

  // Test variables
  logic [7:0] received_byte;
  logic [7:0] duplex_received;

  // Simulate SPI slave response
  always @(posedge spi_serial_clock) begin
    if (!spi_chip_select_n) begin
      // Echo back inverted data (simple slave behavior)
      spi_master_in <= ~spi_master_out;
    end
  end

  // Main test sequence
  initial begin
    // Initialize waveform dump
    $dumpfile("spi_controller_testbench.vcd");
    $dumpvars(0, spi_controller_testbench);
    
    $display("=== SPI Protocol Handler Tasks Test ===");
    $display();
    
    // Initialize signals
    initiate_transfer = 0;
    data_to_send = 8'h00;
    spi_master_in = 0;
    
    // Execute reset sequence
    system_reset_sequence();
    
    $display();
    $display("--- Testing Protocol Handler Tasks ---");
    
    // Test write transaction
    spi_write_transaction(8'hA5);
    
    #100;  // Wait between transactions
    
    // Test read transaction
    spi_read_transaction(received_byte);
    
    #100;  // Wait between transactions
    
    // Test full-duplex transaction
    spi_full_duplex_transaction(8'h3C, duplex_received);
    
    #100;  // Wait between transactions
    
    // Test multiple consecutive transactions
    $display();
    $display("--- Testing Multiple Consecutive Transactions ---");
    spi_write_transaction(8'h55);
    spi_write_transaction(8'hAA);
    spi_write_transaction(8'hFF);
    
    #200;  // Final wait
    
    $display();
    $display("=== SPI Protocol Handler Tasks Test Complete ===");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== SPI Protocol Handler Tasks Test ===

Executing system reset sequence
System reset sequence completed

--- Testing Protocol Handler Tasks ---
Starting SPI write transaction with data: 0xa5
SPI write transaction completed
Data sent: 0xa5, Data received: 0x5a
Starting SPI read transaction
SPI read transaction completed
Data received: 0xff
Starting SPI full-duplex transaction
Transmitting: 0x3c
Full-duplex transaction completed
Sent: 0x3c, Received: 0xc3

--- Testing Multiple Consecutive Transactions ---
Starting SPI write transaction with data: 0x55
SPI write transaction completed
Data sent: 0x55, Data received: 0xaa
Starting SPI write transaction with data: 0xaa
SPI write transaction completed
Data sent: 0xaa, Data received: 0x55
Starting SPI write transaction with data: 0xff
SPI write transaction completed
Data sent: 0xff, Data received: 0x00

=== SPI Protocol Handler Tasks Test Complete ===
Process finished with return code: 0
Removing Chapter_7_example

0

#### Test Pattern Generator Task
Task generating various test patterns for verification

In [12]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_14__test_pattern_generator/"
files = [
    "pattern_generator_design.sv",
    "pattern_generator_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// pattern_generator_design.sv
module pattern_generator_design (
  input  logic       clock_signal,
  input  logic       reset_signal,
  input  logic [1:0] pattern_select,
  output logic [7:0] test_pattern_output
);

  logic [7:0] counter_value;
  
  // Counter for pattern generation
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      counter_value <= 8'h00;
    end else begin
      counter_value <= counter_value + 1;
    end
  end
  
  // Pattern generation task
  task automatic generate_test_patterns(
    input  logic [1:0] select_input,
    input  logic [7:0] count_input,
    output logic [7:0] pattern_output
  );
    case (select_input)
      2'b00: pattern_output = count_input;              // Incrementing
      2'b01: pattern_output = ~count_input;             // Inverted
      2'b10: pattern_output = {count_input[0], 
                               count_input[1], 
                               count_input[2], 
                               count_input[3],
                               count_input[4], 
                               count_input[5], 
                               count_input[6], 
                               count_input[7]};         // Bit-reversed
      2'b11: pattern_output = count_input ^ 8'hAA;     // XOR pattern
    endcase
  endtask
  
  // Apply pattern generation
  always_comb begin
    generate_test_patterns(pattern_select, counter_value, 
                          test_pattern_output);
  end

endmodule
```

```systemverilog
// pattern_generator_testbench.sv
module pattern_generator_testbench;

  // Testbench signals
  logic       clock_stimulus;
  logic       reset_stimulus;
  logic [1:0] pattern_select_stimulus;
  logic [7:0] test_pattern_result;
  
  // Clock generation
  initial begin
    clock_stimulus = 0;
    forever #5 clock_stimulus = ~clock_stimulus;
  end
  
  // Design under test instantiation
  pattern_generator_design PATTERN_GEN_INSTANCE (
    .clock_signal(clock_stimulus),
    .reset_signal(reset_stimulus),
    .pattern_select(pattern_select_stimulus),
    .test_pattern_output(test_pattern_result)
  );
  
  // Test pattern verification task
  task automatic verify_pattern_output(
    input logic [1:0] expected_select,
    input logic [7:0] expected_count,
    input logic [7:0] actual_output
  );
    logic [7:0] expected_pattern;
    
    case (expected_select)
      2'b00: expected_pattern = expected_count;
      2'b01: expected_pattern = ~expected_count;
      2'b10: expected_pattern = {expected_count[0], expected_count[1], 
                                expected_count[2], expected_count[3],
                                expected_count[4], expected_count[5], 
                                expected_count[6], expected_count[7]};
      2'b11: expected_pattern = expected_count ^ 8'hAA;
    endcase
    
    if (actual_output === expected_pattern) begin
      $display("PASS: Pattern %0d, Count %02h, Output %02h", 
               expected_select, expected_count, actual_output);
    end else begin
      $display("FAIL: Pattern %0d, Count %02h, Expected %02h, Got %02h", 
               expected_select, expected_count, expected_pattern, 
               actual_output);
    end
  endtask
  
  // Test stimulus generation task
  task automatic run_pattern_test(input logic [1:0] pattern_type);
    integer test_cycles;
    logic [7:0] observed_count;
    
    $display("Testing pattern type %0d:", pattern_type);
    pattern_select_stimulus = pattern_type;
    
    for (test_cycles = 0; test_cycles < 8; test_cycles++) begin
      @(posedge clock_stimulus);
      #1; // Small delay for signal propagation
      // Get the actual counter value from the design
      observed_count = PATTERN_GEN_INSTANCE.counter_value;
      verify_pattern_output(pattern_type, observed_count, 
                           test_pattern_result);
    end
    $display();
  endtask

  // Main test sequence
  initial begin
    // Dump waves
    $dumpfile("pattern_generator_testbench.vcd");
    $dumpvars(0, pattern_generator_testbench);
    
    // Initialize signals
    reset_stimulus = 1;
    pattern_select_stimulus = 2'b00;
    
    // Display test header
    $display("=== Test Pattern Generator Verification ===");
    $display();
    
    // Apply reset
    repeat(2) @(posedge clock_stimulus);
    reset_stimulus = 0;
    
    // Test all pattern types
    run_pattern_test(2'b00);  // Incrementing pattern
    run_pattern_test(2'b01);  // Inverted pattern
    run_pattern_test(2'b10);  // Bit-reversed pattern
    run_pattern_test(2'b11);  // XOR pattern
    
    // Reset and test pattern switching
    $display("Testing pattern switching:");
    reset_stimulus = 1;
    repeat(2) @(posedge clock_stimulus);
    reset_stimulus = 0;
    
    pattern_select_stimulus = 2'b00;
    repeat(3) @(posedge clock_stimulus);
    pattern_select_stimulus = 2'b11;
    repeat(3) @(posedge clock_stimulus);
    pattern_select_stimulus = 2'b01;
    repeat(3) @(posedge clock_stimulus);
    
    $display("=== Test Pattern Generator Complete ===");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Test Pattern Generator Verification ===

Testing pattern type 0:
PASS: Pattern 0, Count 02, Output 02
PASS: Pattern 0, Count 03, Output 03
PASS: Pattern 0, Count 04, Output 04
PASS: Pattern 0, Count 05, Output 05
PASS: Pattern 0, Count 06, Output 06
PASS: Pattern 0, Count 07, Output 07
PASS: Pattern 0, Count 08, Output 08
PASS: Pattern 0, Count 09, Output 09

Testing pattern type 1:
PASS: Pattern 1, Count 0a, Output f5
PASS: Pattern 1, Count 0b, Output f4
PASS: Pattern 1, Count 0c, Output f3
PASS: Pattern 1, Count 0d, Output f2
PASS: Pattern 1, Count 0e, Output f1
PASS: Pattern 1, Count 0f, Output f0
PASS: Pattern 1, Count 10, Output ef
PASS: Pattern 1, Count 11, Output ee

Testing pattern type 2:
PASS: Pattern 2, Count 12, Output 48
PASS: Pattern 2, Count 13, Output c8
PASS: Pattern 2, Count 14, Output 28
PASS: Pattern 2, Count 15, Output a8
PASS: Pattern 2, Count 16, Output 68
PASS: Pattern 2, Count 17, Output e8
PASS: Pattern 2, Count 18, Output 18
P

0

#### Memory Test Tasks
Tasks for comprehensive memory testing with different patterns

In [15]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_15__memory_test_tasks/"
files = [
    "memory_array_design.sv",
    "memory_array_design_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// memory_array_design.sv
module simple_memory_array_module (
  input  logic        clock_signal,
  input  logic        reset_signal,
  input  logic        write_enable_signal,
  input  logic [7:0]  address_bus,
  input  logic [15:0] write_data_bus,
  output logic [15:0] read_data_bus
);

  // Memory array - 256 words of 16 bits each
  logic [15:0] memory_storage_array [0:255];
  
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      read_data_bus <= 16'h0000;
    end else begin
      if (write_enable_signal) begin
        memory_storage_array[address_bus] <= write_data_bus;
        $display("WRITE: Addr[%02h] = %04h", address_bus, write_data_bus);
      end
      read_data_bus <= memory_storage_array[address_bus];
    end
  end

endmodule
```

```systemverilog
// memory_array_design_testbench.sv
module memory_test_pattern_testbench;

  // Clock and reset signals
  logic        clock_signal;
  logic        reset_signal;
  logic        write_enable_signal;
  logic [7:0]  address_bus;
  logic [15:0] write_data_bus;
  logic [15:0] read_data_bus;

  // Instantiate memory design under test
  simple_memory_array_module MEMORY_DESIGN_INSTANCE (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .write_enable_signal(write_enable_signal),
    .address_bus(address_bus),
    .write_data_bus(write_data_bus),
    .read_data_bus(read_data_bus)
  );

  // Clock generation
  initial begin
    clock_signal = 0;
    forever #5 clock_signal = ~clock_signal;  // 10ns period
  end

  // Task: Walking ones pattern test
  task automatic walking_ones_pattern_test();
    logic [15:0] walking_pattern;
    $display("\n=== WALKING ONES PATTERN TEST ===");
    
    for (int address_index = 0; address_index < 16; address_index++) begin
      walking_pattern = 16'h0001 << address_index;
      write_memory_location(address_index[7:0], walking_pattern);
      verify_memory_location(address_index[7:0], walking_pattern);
    end
  endtask

  // Task: Walking zeros pattern test  
  task automatic walking_zeros_pattern_test();
    logic [15:0] walking_pattern;
    $display("\n=== WALKING ZEROS PATTERN TEST ===");
    
    for (int address_index = 0; address_index < 16; address_index++) begin
      walking_pattern = ~(16'h0001 << address_index);
      write_memory_location(address_index[7:0], walking_pattern);
      verify_memory_location(address_index[7:0], walking_pattern);
    end
  endtask

  // Task: Checkerboard pattern test
  task automatic checkerboard_pattern_test();
    logic [15:0] checkerboard_pattern_a = 16'hAAAA;
    logic [15:0] checkerboard_pattern_b = 16'h5555;
    $display("\n=== CHECKERBOARD PATTERN TEST ===");
    
    // Write checkerboard pattern A to even addresses
    for (int address_index = 0; address_index < 32; address_index += 2) begin
      write_memory_location(address_index[7:0], checkerboard_pattern_a);
    end
    
    // Write checkerboard pattern B to odd addresses  
    for (int address_index = 1; address_index < 32; address_index += 2) begin
      write_memory_location(address_index[7:0], checkerboard_pattern_b);
    end
    
    // Verify all patterns
    for (int address_index = 0; address_index < 32; address_index++) begin
      if (address_index % 2 == 0)
        verify_memory_location(address_index[7:0], checkerboard_pattern_a);
      else
        verify_memory_location(address_index[7:0], checkerboard_pattern_b);
    end
  endtask

  // Task: Address-in-data pattern test
  task automatic address_in_data_pattern_test();
    $display("\n=== ADDRESS-IN-DATA PATTERN TEST ===");
    
    for (int address_index = 0; address_index < 64; address_index++) begin
      write_memory_location(address_index[7:0], {8'h00, address_index[7:0]});
      verify_memory_location(address_index[7:0], {8'h00, address_index[7:0]});
    end
  endtask

  // Helper task: Write to memory location
  task automatic write_memory_location(input [7:0] addr, input [15:0] data);
    @(posedge clock_signal);
    address_bus = addr;
    write_data_bus = data;
    write_enable_signal = 1'b1;
    @(posedge clock_signal);
    write_enable_signal = 1'b0;
  endtask

  // Helper task: Verify memory location
  task automatic verify_memory_location(input [7:0] addr, 
                                       input [15:0] expected_data);
    @(posedge clock_signal);
    address_bus = addr;
    write_enable_signal = 1'b0;
    @(posedge clock_signal);
    
    if (read_data_bus === expected_data) begin
      $display("PASS: Addr[%02h] = %04h (Expected: %04h)", 
               addr, read_data_bus, expected_data);
    end else begin
      $display("FAIL: Addr[%02h] = %04h (Expected: %04h)", 
               addr, read_data_bus, expected_data);
    end
  endtask

  // Main test sequence
  initial begin
    // Setup waveform dump
    $dumpfile("memory_test_pattern_testbench.vcd");
    $dumpvars(0, memory_test_pattern_testbench);

    // Initialize signals
    reset_signal = 1'b1;
    write_enable_signal = 1'b0;
    address_bus = 8'h00;
    write_data_bus = 16'h0000;

    // Release reset
    repeat(3) @(posedge clock_signal);
    reset_signal = 1'b0;
    $display("Memory test patterns starting...\n");

    // Execute comprehensive memory test patterns
    walking_ones_pattern_test();
    walking_zeros_pattern_test();
    checkerboard_pattern_test();
    address_in_data_pattern_test();

    $display("\n=== MEMORY PATTERN TESTING COMPLETE ===");
    $display();
    
    // Finish simulation
    repeat(5) @(posedge clock_signal);
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Memory test patterns starting...


=== WALKING ONES PATTERN TEST ===
WRITE: Addr[00] = 0001
PASS: Addr[00] = 0001 (Expected: 0001)
WRITE: Addr[01] = 0002
PASS: Addr[01] = 0002 (Expected: 0002)
WRITE: Addr[02] = 0004
PASS: Addr[02] = 0004 (Expected: 0004)
WRITE: Addr[03] = 0008
PASS: Addr[03] = 0008 (Expected: 0008)
WRITE: Addr[04] = 0010
PASS: Addr[04] = 0010 (Expected: 0010)
WRITE: Addr[05] = 0020
PASS: Addr[05] = 0020 (Expected: 0020)
WRITE: Addr[06] = 0040
PASS: Addr[06] = 0040 (Expected: 0040)
WRITE: Addr[07] = 0080
PASS: Addr[07] = 0080 (Expected: 0080)
WRITE: Addr[08] = 0100
PASS: Addr[08] = 0100 (Expected: 0100)
WRITE: Addr[09] = 0200
PASS: Addr[09] = 0200 (Expected: 0200)
WRITE: Addr[0a] = 0400
PASS: Addr[0a] = 0400 (Expected: 0400)
WRITE: Addr[0b] = 0800
PASS: Addr[0b] = 0800 (Expected: 0800)
WRITE: Addr[0c] = 1000
PASS: Addr[0c] = 1000 (Expected: 1000)
WRITE: Addr[0d] = 2000
PASS: Addr[0d] = 2000 (Expected: 2000)
WRITE: Addr[0e] = 4000
PASS: Addr[

0

#### Waveform Generator Task
Task producing different types of waveforms for testing

In [17]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_16__waveform_generator_task/"
files = [
    "waveform_generator_design.sv",
    "waveform_generator_design_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// waveform_generator_design.sv
module waveform_generator_module (
  input  logic       clock_signal,
  input  logic       reset_signal,
  input  logic [1:0] waveform_type_select,
  output logic       generated_waveform_output
);

  logic [3:0] internal_counter;
  
  // Counter for waveform generation
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      internal_counter <= 4'h0;
    end else begin
      internal_counter <= internal_counter + 1'b1;
    end
  end
  
  // Waveform generation based on type selection
  always_comb begin
    case (waveform_type_select)
      2'b00: // Square wave (50% duty cycle)
        generated_waveform_output = internal_counter[3];
      2'b01: // Triangle wave approximation (using counter bits)
        generated_waveform_output = internal_counter[2] ^ internal_counter[3];
      2'b10: // Sawtooth wave (using MSB)
        generated_waveform_output = internal_counter[3];
      2'b11: // Pulse wave (25% duty cycle)
        generated_waveform_output = (internal_counter < 4'h4);
      default:
        generated_waveform_output = 1'b0;
    endcase
  end

endmodule
```

```systemverilog
// waveform_generator_design_testbench.sv
module waveform_generator_testbench;
  
  // Testbench signals
  logic       clock_signal;
  logic       reset_signal;
  logic [1:0] waveform_type_select;
  logic       generated_waveform_output;
  
  // Instantiate design under test
  waveform_generator_module WAVEFORM_GENERATOR_INSTANCE (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .waveform_type_select(waveform_type_select),
    .generated_waveform_output(generated_waveform_output)
  );
  
  // Clock generation
  initial begin
    clock_signal = 1'b0;
    forever #5 clock_signal = ~clock_signal;
  end
  
  // Task to generate different waveform types
  task automatic generate_waveform_pattern(
    input [1:0] selected_waveform_type,
    input int   pattern_duration_cycles
  );
    $display("Generating waveform type: %0d for %0d cycles", 
             selected_waveform_type, pattern_duration_cycles);
    waveform_type_select = selected_waveform_type;
    repeat(pattern_duration_cycles) @(posedge clock_signal);
    $display("Waveform pattern generation completed");
    $display();
  endtask
  
  // Task to apply reset sequence
  task automatic apply_system_reset(input int reset_duration_cycles);
    $display("Applying system reset for %0d cycles", reset_duration_cycles);
    reset_signal = 1'b1;
    repeat(reset_duration_cycles) @(posedge clock_signal);
    reset_signal = 1'b0;
    $display("System reset released");
    $display();
  endtask
  
  // Main test sequence
  initial begin
    // Setup waveform dumping
    $dumpfile("waveform_generator_testbench.vcd");
    $dumpvars(0, waveform_generator_testbench);
    
    // Initialize signals
    reset_signal = 1'b0;
    waveform_type_select = 2'b00;
    
    $display();
    $display("=== Waveform Generator Task Test ===");
    $display();
    
    // Apply reset
    apply_system_reset(3);
    
    // Test different waveform types using tasks
    generate_waveform_pattern(2'b00, 16);  // Square wave
    generate_waveform_pattern(2'b01, 16);  // Triangle wave
    generate_waveform_pattern(2'b10, 16);  // Sawtooth wave
    generate_waveform_pattern(2'b11, 16);  // Pulse wave
    
    $display("All waveform patterns tested successfully!");
    $display();
    
    #50;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

=== Waveform Generator Task Test ===

Applying system reset for 3 cycles
System reset released

Generating waveform type: 0 for 16 cycles
Waveform pattern generation completed

Generating waveform type: 1 for 16 cycles
Waveform pattern generation completed

Generating waveform type: 2 for 16 cycles
Waveform pattern generation completed

Generating waveform type: 3 for 16 cycles
Waveform pattern generation completed

All waveform patterns tested successfully!

Process finished with return code: 0
Removing Chapter_7_examples/example_16__waveform_generator_task/obj_dir directory...
Chapter_7_examples/example_16__waveform_generator_task/obj_dir removed successfully.


0

#### File I/O Tasks
Tasks for reading from and writing to files during simulation

In [19]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_17__file_io_tasks/"
files = ["file_processor.sv", "file_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// file_processor.sv
module file_data_processor #(
  parameter DATA_WIDTH = 8
)(
  input  logic                    clock,
  input  logic                    reset_n,
  input  logic [DATA_WIDTH-1:0]   input_data,
  input  logic                    data_valid,
  output logic [DATA_WIDTH-1:0]   processed_data,
  output logic                    output_valid
);

  // Simple data processing: increment by 1
  always_ff @(posedge clock or negedge reset_n) begin
    if (!reset_n) begin
      processed_data <= '0;
      output_valid   <= 1'b0;
    end else begin
      if (data_valid) begin
        processed_data <= input_data + 1'b1;  // Simple increment
        output_valid   <= 1'b1;
      end else begin
        output_valid   <= 1'b0;
      end
    end
  end

endmodule
```

```systemverilog
// file_processor_testbench.sv
module file_processor_testbench;

  // Testbench signals
  logic       tb_clock;
  logic       tb_reset_n;
  logic [7:0] tb_input_data;
  logic       tb_data_valid;
  logic [7:0] tb_processed_data;
  logic       tb_output_valid;

  // File handles
  integer input_file_handle;
  integer output_file_handle;
  integer log_file_handle;

  // Test data
  logic [7:0] read_data;
  integer     scan_result;

  // DUT instantiation
  file_data_processor #(
    .DATA_WIDTH(8)
  ) dut_file_processor (
    .clock(tb_clock),
    .reset_n(tb_reset_n),
    .input_data(tb_input_data),
    .data_valid(tb_data_valid),
    .processed_data(tb_processed_data),
    .output_valid(tb_output_valid)
  );

  // Clock generation
  initial begin
    tb_clock = 1'b0;
    forever #5 tb_clock = ~tb_clock;  // 10ns period
  end

  // Main test sequence
  initial begin
    // Setup waveform dumping
    $dumpfile("file_processor_testbench.vcd");
    $dumpvars(0, file_processor_testbench);

    // Open files
    input_file_handle = $fopen("input_test_data.txt", "r");
    output_file_handle = $fopen("output_results.txt", "w");
    log_file_handle = $fopen("simulation_log.txt", "w");

    // Check if files opened successfully
    if (input_file_handle == 0) begin
      $display("ERROR: Could not open input_test_data.txt");
      $finish;
    end

    // Write header to log file
    $fwrite(log_file_handle, "=== File I/O Test Simulation Log ===\n");
    $fwrite(log_file_handle, "Time\tInput\tOutput\tValid\n");
    $fwrite(log_file_handle, "----\t-----\t------\t-----\n");

    // Initialize signals
    tb_reset_n = 1'b0;
    tb_input_data = 8'h00;
    tb_data_valid = 1'b0;

    // Reset sequence
    repeat(3) @(posedge tb_clock);
    tb_reset_n = 1'b1;
    @(posedge tb_clock);

    $display("Starting file I/O test...");

    // Read data from file and process
    while (!$feof(input_file_handle)) begin
      // Read data from input file
      scan_result = $fscanf(input_file_handle, "%h\n", read_data);
      
      if (scan_result == 1) begin
        // Apply data to DUT
        tb_input_data = read_data;
        tb_data_valid = 1'b1;
        @(posedge tb_clock);
        
        // Wait for output
        @(posedge tb_clock);
        
        if (tb_output_valid) begin
          // Write results to output file
          $fwrite(output_file_handle, "%02h -> %02h\n", 
                  read_data, tb_processed_data);
          
          // Write to log file
          $fwrite(log_file_handle, "%0t\t%02h\t%02h\t%b\n", 
                  $time, read_data, tb_processed_data, tb_output_valid);
          
          $display("Processed: 0x%02h -> 0x%02h", 
                   read_data, tb_processed_data);
        end
        
        tb_data_valid = 1'b0;
        @(posedge tb_clock);
      end
    end

    // Write test summary
    $fwrite(log_file_handle, "\n=== Test Completed Successfully ===\n");
    $fwrite(output_file_handle, "\n# Test completed at time %0t\n", $time);

    // Close all files
    $fclose(input_file_handle);
    $fclose(output_file_handle);
    $fclose(log_file_handle);

    $display("File I/O test completed!");
    $display("Check output_results.txt and simulation_log.txt");
    
    #20;
    $finish;
  end

  // Create input test data file
  initial begin
    integer temp_file;
    temp_file = $fopen("input_test_data.txt", "w");
    
    // Write test data (hex values)
    $fwrite(temp_file, "0A\n");
    $fwrite(temp_file, "1F\n");
    $fwrite(temp_file, "33\n");
    $fwrite(temp_file, "7E\n");
    $fwrite(temp_file, "FF\n");
    
    $fclose(temp_file);
  end

endmodule
```

Verilator Simulation Output:
Starting file I/O test...
Processed: 0x0a -> 0x0b
Processed: 0x1f -> 0x20
Processed: 0x33 -> 0x34
Processed: 0x7e -> 0x7f
Processed: 0xff -> 0x00
File I/O test completed!
Check output_results.txt and simulation_log.txt
Process finished with return code: 0
Removing Chapter_7_examples/example_17__file_io_tasks/obj_dir directory...
Chapter_7_examples/example_17__file_io_tasks/obj_dir removed successfully.


0

## Automatic vs. Static Lifetime

#### Recursive Algorithm Examples
Automatic functions implementing recursive algorithms like tree traversal

In [21]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_18__recursive_factorial/"
files = [
    "recursive_factorial_calculator.sv",
    "recursive_factorial_calculator_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// recursive_factorial_calculator.sv
module recursive_factorial_calculator_module;

  // Automatic function for recursive factorial calculation
  function automatic int unsigned calculate_factorial(int unsigned input_number);
    if (input_number <= 1) begin
      return 1;  // Base case: factorial of 0 or 1 is 1
    end else begin
      // Recursive case: n! = n * (n-1)!
      return input_number * calculate_factorial(input_number - 1);
    end
  endfunction

  // Function to demonstrate tree-like traversal depth
  function automatic int unsigned calculate_tree_depth(int unsigned tree_nodes);
    if (tree_nodes <= 1) begin
      return 0;  // Base case: single node has depth 0
    end else begin
      // Recursive case: depth increases with each level
      return 1 + calculate_tree_depth(tree_nodes / 2);
    end
  endfunction

  initial begin
    int unsigned factorial_result;
    int unsigned tree_depth_result;
    
    $display("=== Recursive Factorial Calculator ===");
    $display();
    
    // Test factorial calculations
    for (int i = 0; i <= 6; i++) begin
      factorial_result = calculate_factorial(i);
      $display("Factorial of %0d = %0d", i, factorial_result);
    end
    
    $display();
    $display("=== Tree Depth Calculator ===");
    
    // Test tree depth calculations
    for (int nodes = 1; nodes <= 16; nodes *= 2) begin
      tree_depth_result = calculate_tree_depth(nodes);
      $display("Tree with %0d nodes has depth %0d", nodes, tree_depth_result);
    end
    
    $display();
  end

endmodule
```

```systemverilog
// recursive_factorial_calculator_testbench.sv
module factorial_testbench_module;

  // Instantiate the recursive calculator design
  recursive_factorial_calculator_module RECURSIVE_CALC_INSTANCE();

  // Local automatic function for verification
  function automatic int unsigned verify_factorial_calculation(int unsigned num);
    if (num <= 1) begin
      return 1;
    end else begin
      return num * verify_factorial_calculation(num - 1);
    end
  endfunction

  initial begin
    int unsigned expected_result;
    int unsigned actual_result;
    bit all_tests_passed;
    
    // Dump waves for analysis
    $dumpfile("factorial_testbench_module.vcd");
    $dumpvars(0, factorial_testbench_module);
    
    #1;  // Wait for design to initialize
    
    $display("=== Testbench: Recursive Algorithm Verification ===");
    $display();
    
    all_tests_passed = 1'b1;
    
    // Verify factorial calculations
    for (int test_value = 0; test_value <= 5; test_value++) begin
      expected_result = verify_factorial_calculation(test_value);
      actual_result = RECURSIVE_CALC_INSTANCE.calculate_factorial(test_value);
      
      if (expected_result == actual_result) begin
        $display("Test PASSED: factorial(%0d) = %0d", 
                 test_value, actual_result);
      end else begin
        $display("Test FAILED: factorial(%0d) expected %0d, got %0d", 
                 test_value, expected_result, actual_result);
        all_tests_passed = 1'b0;
      end
    end
    
    $display();
    
    // Test boundary conditions
    $display("=== Boundary Condition Tests ===");
    
    // Test edge cases
    if (RECURSIVE_CALC_INSTANCE.calculate_factorial(0) == 1) begin
      $display("Boundary test PASSED: factorial(0) = 1");
    end else begin
      $display("Boundary test FAILED: factorial(0) should be 1");
      all_tests_passed = 1'b0;
    end
    
    if (RECURSIVE_CALC_INSTANCE.calculate_factorial(1) == 1) begin
      $display("Boundary test PASSED: factorial(1) = 1");
    end else begin
      $display("Boundary test FAILED: factorial(1) should be 1");
      all_tests_passed = 1'b0;
    end
    
    $display();
    
    // Final test summary
    if (all_tests_passed) begin
      $display("ALL RECURSIVE ALGORITHM TESTS PASSED!");
    end else begin
      $display("SOME TESTS FAILED - CHECK IMPLEMENTATION");
    end
    
    $display();
    $display("Testbench completed successfully!");
    $display();
    
    #10;  // Additional time for wave viewing
  end

endmodule
```

Verilator Simulation Output:
=== Recursive Factorial Calculator ===

Factorial of 0 = 1
Factorial of 1 = 1
Factorial of 2 = 2
Factorial of 3 = 6
Factorial of 4 = 24
Factorial of 5 = 120
Factorial of 6 = 720

=== Tree Depth Calculator ===
Tree with 1 nodes has depth 0
Tree with 2 nodes has depth 1
Tree with 4 nodes has depth 2
Tree with 8 nodes has depth 3
Tree with 16 nodes has depth 4

=== Testbench: Recursive Algorithm Verification ===

Test PASSED: factorial(0) = 1
Test PASSED: factorial(1) = 1
Test PASSED: factorial(2) = 2
Test PASSED: factorial(3) = 6
Test PASSED: factorial(4) = 24
Test PASSED: factorial(5) = 120

=== Boundary Condition Tests ===
Boundary test PASSED: factorial(0) = 1
Boundary test PASSED: factorial(1) = 1

ALL RECURSIVE ALGORITHM TESTS PASSED!

Testbench completed successfully!
Process finished with return code: 0
Removing Chapter_7_examples/example_18__recursive_factorial/obj_dir directory...
Chapter_7_examples/example_18__recursive_factorial/obj_dir removed suc

0

#### Call Counter Examples
Static functions maintaining call statistics and usage counters

In [26]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_19__call_counter/"
files = ["call_counter_utility.sv", "call_counter_utility_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// call_counter_utility.sv
// Utility class demonstrating static functions with call counters

class statistics_tracker_class;
  
  // Static function to track total function calls across all instances
  static function void increment_total_calls();
    static int total_call_counter = 0;
    total_call_counter++;
    $display("Total calls made: %0d", total_call_counter);
  endfunction
  
  // Static function to track specific operation calls
  static function void track_operation_usage(string operation_name);
    static int operation_counter[string];
    
    if (operation_counter.exists(operation_name) == 0) begin
      operation_counter[operation_name] = 0;
    end
    
    operation_counter[operation_name]++;
    $display("Operation '%s' called %0d times", 
             operation_name, operation_counter[operation_name]);
  endfunction
  
  // Static function to get usage statistics report
  static function void print_usage_report();
    $display("=== Usage Statistics Report ===");
    $display("This report shows function call statistics");
  endfunction
  
endclass

// Simple module to demonstrate the utility
module call_counter_utility_module;
  
  initial begin
    $display("Call Counter Utility Demo");
    $display("==========================");
    
    // Demonstrate static function call counting
    statistics_tracker_class::increment_total_calls();
    statistics_tracker_class::track_operation_usage("read");
    statistics_tracker_class::track_operation_usage("write");
    statistics_tracker_class::track_operation_usage("read");
    statistics_tracker_class::increment_total_calls();
    
    $display();
    statistics_tracker_class::print_usage_report();
  end
  
endmodule
```

```systemverilog
// call_counter_utility_testbench.sv
// Testbench for call counter utility demonstration

module call_counter_testbench_module;
  
  // Instantiate the design under test
  call_counter_utility_module COUNTER_UTILITY_INSTANCE();
  
  initial begin
    // Dump waves for simulation
    $dumpfile("call_counter_testbench_module.vcd");
    $dumpvars(0, call_counter_testbench_module);
    
    $display("Testbench: Call Counter Utility Test");
    $display("=====================================");
    
    // Wait for design to complete
    #10;
    
    $display();
    $display("Testbench: Additional counter tests");
    
    // Test multiple calls to demonstrate persistent counters
    statistics_tracker_class::track_operation_usage("delete");
    statistics_tracker_class::track_operation_usage("write");
    statistics_tracker_class::track_operation_usage("delete");
    statistics_tracker_class::increment_total_calls();
    
    $display();
    $display("Testbench: Final statistics");
    statistics_tracker_class::print_usage_report();
    
    // End simulation
    #5;
    $display();
    $display("Testbench: Simulation completed successfully");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
Call Counter Utility Demo
Total calls made: 1
Operation 'read' called 1 times
Operation 'write' called 1 times
Operation 'read' called 2 times
Total calls made: 2

=== Usage Statistics Report ===
This report shows function call statistics
Testbench: Call Counter Utility Test

Testbench: Additional counter tests
Operation 'delete' called 1 times
Operation 'write' called 2 times
Operation 'delete' called 2 times
Total calls made: 3

Testbench: Final statistics
=== Usage Statistics Report ===
This report shows function call statistics

Testbench: Simulation completed successfully
Process finished with return code: 0
Removing Chapter_7_examples/example_19__call_counter/obj_dir directory...
Chapter_7_examples/example_19__call_counter/obj_dir removed successfully.


0

#### Stack Implementation
Automatic functions for stack operations with proper variable scoping

In [35]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_20__stack_automatic_functions/"
files = ["stack_processor.sv", "stack_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// stack_processor.sv
module stack_processor;                    // Stack implementation module

  // Stack parameters
  parameter int STACK_DEPTH = 8;
  parameter int DATA_WIDTH = 16;
  
  // Stack storage and pointer
  logic [DATA_WIDTH-1:0] memory_stack [STACK_DEPTH-1:0];
  int stack_pointer;
  
  // Initialize stack
  initial begin
    stack_pointer = 0;
    $display("Stack Processor: Initialized with depth=%0d, width=%0d",
             STACK_DEPTH, DATA_WIDTH);
  end

  // Automatic function to push data onto stack
  function automatic logic push_data_onto_stack(
    input logic [DATA_WIDTH-1:0] push_value
  );
    // Local variables with automatic scope
    logic operation_success;
    int current_depth;
    
    current_depth = stack_pointer;  // Capture current state
    
    if (current_depth < STACK_DEPTH) begin
      memory_stack[current_depth] = push_value;
      stack_pointer++;
      operation_success = 1'b1;
      $display("  PUSH: 0x%04h at position %0d (depth now %0d)",
               push_value, current_depth, stack_pointer);
    end else begin
      operation_success = 1'b0;
      $display("  PUSH FAILED: Stack overflow! Depth=%0d", current_depth);
    end
    
    return operation_success;
  endfunction

  // Automatic function to pop data from stack
  function automatic logic [DATA_WIDTH-1:0] pop_data_from_stack(
    output logic pop_success
  );
    // Local variables with automatic scope
    logic [DATA_WIDTH-1:0] popped_value;
    int current_depth;
    
    current_depth = stack_pointer;  // Capture current state
    
    if (current_depth > 0) begin
      stack_pointer--;
      popped_value = memory_stack[stack_pointer];
      pop_success = 1'b1;
      $display("  POP:  0x%04h from position %0d (depth now %0d)",
               popped_value, stack_pointer, stack_pointer);
    end else begin
      popped_value = '0;
      pop_success = 1'b0;
      $display("  POP FAILED: Stack underflow! Depth=%0d", current_depth);
    end
    
    return popped_value;
  endfunction

  // Automatic function to peek at top of stack
  function automatic logic [DATA_WIDTH-1:0] peek_stack_top(
    output logic peek_valid
  );
    // Local variables with automatic scope
    logic [DATA_WIDTH-1:0] top_value;
    int current_depth;
    
    current_depth = stack_pointer;  // Capture current state
    
    if (current_depth > 0) begin
      top_value = memory_stack[current_depth - 1];
      peek_valid = 1'b1;
      $display("  PEEK: 0x%04h at top (position %0d)",
               top_value, current_depth - 1);
    end else begin
      top_value = '0;
      peek_valid = 1'b0;
      $display("  PEEK FAILED: Stack empty! Depth=%0d", current_depth);
    end
    
    return top_value;
  endfunction

  // Function to display current stack status
  function automatic void display_stack_status();
    // Local variables with automatic scope
    string status_message;
    int items_count;
    
    items_count = stack_pointer;
    
    if (items_count == 0) begin
      status_message = "EMPTY";
    end else if (items_count == STACK_DEPTH) begin
      status_message = "FULL";
    end else begin
      status_message = $sformatf("%0d/%0d", items_count, STACK_DEPTH);
    end
    
    $display("Stack Status: %s (pointer=%0d)", status_message, items_count);
  endfunction

endmodule
```

```systemverilog
// stack_processor_testbench.sv
module stack_processor_testbench;         // Testbench for stack processor

  // Instantiate the stack processor
  stack_processor STACK_PROCESSOR_INSTANCE();

  // Test variables
  logic [15:0] test_data_values [0:9] = {
    16'hABCD, 16'h1234, 16'hDEAD, 16'hBEEF, 16'hCAFE,
    16'hFACE, 16'hFEED, 16'hC0DE, 16'hDA7A, 16'hF00D
  };
  
  logic push_result, pop_result, peek_result;
  logic [15:0] popped_value, peeked_value;
  int test_iteration;

  initial begin
    // Setup waveform dumping
    $dumpfile("stack_processor_testbench.vcd");
    $dumpvars(0, stack_processor_testbench);
    
    $display();
    $display("=== Stack Processor Testbench Started ===");
    $display();
    
    // Wait for initialization
    #1;
    
    // Display initial stack status
    STACK_PROCESSOR_INSTANCE.display_stack_status();
    $display();
    
    // Test 1: Fill stack with data
    $display("TEST 1: Filling stack with test data");
    for (test_iteration = 0; test_iteration < 10; test_iteration++) begin
      push_result = STACK_PROCESSOR_INSTANCE.push_data_onto_stack(
        test_data_values[test_iteration]
      );
      #1;
    end
    STACK_PROCESSOR_INSTANCE.display_stack_status();
    $display();
    
    // Test 2: Peek at stack top
    $display("TEST 2: Peeking at stack top");
    peeked_value = STACK_PROCESSOR_INSTANCE.peek_stack_top(peek_result);
    if (peek_result) 
      $display("Successfully peeked value: 0x%04h", peeked_value);
    $display();
    
    // Test 3: Pop some values
    $display("TEST 3: Popping values from stack");
    for (test_iteration = 0; test_iteration < 5; test_iteration++) begin
      popped_value = STACK_PROCESSOR_INSTANCE.pop_data_from_stack(pop_result);
      if (pop_result)
        $display("Test iteration %0d: Successfully popped 0x%04h", 
                 test_iteration, popped_value);
      #1;
    end
    STACK_PROCESSOR_INSTANCE.display_stack_status();
    $display();
    
    // Test 4: Push more data
    $display("TEST 4: Pushing additional data");
    push_result = STACK_PROCESSOR_INSTANCE.push_data_onto_stack(16'h9999);
    push_result = STACK_PROCESSOR_INSTANCE.push_data_onto_stack(16'h8888);
    push_result = STACK_PROCESSOR_INSTANCE.push_data_onto_stack(16'h7777);
    STACK_PROCESSOR_INSTANCE.display_stack_status();
    $display();
    
    // Test 5: Empty the stack
    $display("TEST 5: Emptying the entire stack");
    test_iteration = 0;
    while (STACK_PROCESSOR_INSTANCE.stack_pointer > 0) begin
      popped_value = STACK_PROCESSOR_INSTANCE.pop_data_from_stack(pop_result);
      test_iteration++;
      #1;
    end
    STACK_PROCESSOR_INSTANCE.display_stack_status();
    $display();
    
    // Test 6: Try operations on empty stack
    $display("TEST 6: Testing operations on empty stack");
    popped_value = STACK_PROCESSOR_INSTANCE.pop_data_from_stack(pop_result);
    peeked_value = STACK_PROCESSOR_INSTANCE.peek_stack_top(peek_result);
    $display();
    
    $display("=== All Stack Tests Completed Successfully ===");
    $display();
    
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Stack Processor: Initialized with depth=8, width=16

=== Stack Processor Testbench Started ===

Stack Status: EMPTY (pointer=0)

TEST 1: Filling stack with test data
  PUSH: 0xabcd at position 0 (depth now 1)
  PUSH: 0x1234 at position 1 (depth now 2)
  PUSH: 0xdead at position 2 (depth now 3)
  PUSH: 0xbeef at position 3 (depth now 4)
  PUSH: 0xcafe at position 4 (depth now 5)
  PUSH: 0xface at position 5 (depth now 6)
  PUSH: 0xfeed at position 6 (depth now 7)
  PUSH: 0xc0de at position 7 (depth now 8)
  PUSH FAILED: Stack overflow! Depth=8
  PUSH FAILED: Stack overflow! Depth=8
Stack Status: FULL (pointer=8)

TEST 2: Peeking at stack top
  PEEK: 0xc0de at top (position 7)
Successfully peeked value: 0xc0de

TEST 3: Popping values from stack
  POP:  0xc0de from position 7 (depth now 7)
Test iteration 0: Successfully popped 0xc0de
  POP:  0xfeed from position 6 (depth now 6)
Test iteration 1: Successfully popped 0xfeed
  POP:  0xface from position 5 (depth 

0

#### Random Number Generator
Static function maintaining seed state between calls

In [ ]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_21__random_number_generator/"
files = ["random_number_generator.sv", "random_number_generator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


Verilator Simulation Output:

Random Number Generator with Persistent State

Testing Static Function Random Generator
Random number: 11
Random number: 62
Random number: 61
Random number: 56
Random number: 31

Static function maintains seed between calls
Process finished with return code: 0
Removing Chapter_7_examples/example_21__random_number_generator/obj_dir directory...
Chapter_7_examples/example_21__random_number_generator/obj_dir removed successfully.


0

#### Nested Function Calls
Examples showing variable lifetime in nested function scenarios

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_22__nested_function_calls/"
files = [
    "nested_function_lifetime_demo.sv",
    "nested_function_lifetime_demo_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// nested_function_lifetime_demo.sv
module nested_function_lifetime_demo;
  
  // Function with automatic variables (default)
  function automatic int calculate_outer_sum(input int base_value, 
                                           input int multiplier);
    int outer_accumulator;  // Automatic variable - new instance each call
    
    $display("Outer function called: base=%0d, mult=%0d", 
             base_value, multiplier);
    
    outer_accumulator = base_value;
    
    // Nested function call
    return outer_accumulator + calculate_inner_product(base_value, 
                                                      multiplier);
  endfunction
  
  // Nested function with static variable
  function automatic int calculate_inner_product(input int value, 
                                               input int factor);
    static int call_counter = 0;  // Static - persists across calls
    int inner_result;             // Automatic - new each call
    
    call_counter++;  // Increment static counter
    inner_result = value * factor;
    
    $display("  Inner function call #%0d: %0d * %0d = %0d", 
             call_counter, value, factor, inner_result);
    
    return inner_result;
  endfunction
  
  // Function demonstrating variable scope in nested calls
  function automatic int recursive_factorial_helper(input int number);
    int local_temp;  // Each recursion level has its own copy
    
    if (number <= 1) begin
      $display("    Base case reached: number=%0d", number);
      return 1;
    end
    else begin
      local_temp = number;
      $display("    Recursive call: number=%0d", number);
      return local_temp * recursive_factorial_helper(number - 1);
    end
  endfunction
  
  initial begin
    int first_result, second_result, third_result;
    int factorial_result;
    
    $display("=== Nested Function Call Variable Lifetime Demo ===");
    $display();
    
    // Demonstrate automatic variable behavior in nested calls
    $display("Test 1: First nested function call sequence");
    first_result = calculate_outer_sum(10, 3);  
    $display("First result: %0d", first_result);
    $display();
    
    $display("Test 2: Second nested function call sequence");
    second_result = calculate_outer_sum(5, 4);  
    $display("Second result: %0d", second_result);
    $display();
    
    $display("Test 3: Third nested function call sequence");
    third_result = calculate_outer_sum(7, 2);   
    $display("Third result: %0d", third_result);
    $display();
    
    // Demonstrate recursive function with automatic variables
    $display("Test 4: Recursive function with automatic variables");
    factorial_result = recursive_factorial_helper(4);
    $display("Factorial of 4: %0d", factorial_result);
    $display();
    
  end
  
endmodule
```

```systemverilog
// nested_function_lifetime_demo_testbench.sv
module nested_function_lifetime_testbench;
  
  // Instantiate the design under test
  nested_function_lifetime_demo DESIGN_UNDER_TEST_INSTANCE();
  
  // Additional testbench function to demonstrate cross-module calls
  function automatic int testbench_wrapper_function(input int test_value);
    int wrapper_local;  // Automatic variable in testbench
    
    $display("Testbench wrapper called with value: %0d", test_value);
    wrapper_local = test_value + 100;
    
    // This would call design functions if they were accessible
    // For demonstration, we'll just return a calculated value
    return wrapper_local * 2;
  endfunction
  
  initial begin
    int testbench_result;
    
    // Dump waves for simulation
    $dumpfile("nested_function_lifetime_testbench.vcd");
    $dumpvars(0, nested_function_lifetime_testbench);
    
    #1;  // Wait for design to complete
    
    $display("=== Testbench Additional Tests ===");
    $display();
    
    // Test variable lifetime in testbench functions
    $display("Testing testbench function variable lifetime:");
    testbench_result = testbench_wrapper_function(25);
    $display("Testbench result: %0d", testbench_result);
    $display();
    
    testbench_result = testbench_wrapper_function(50);
    $display("Testbench result: %0d", testbench_result);
    $display();
    
    $display("=== Key Observations ===");
    $display("1. Automatic variables create new instances for each call");
    $display("2. Static variables persist across all function calls");
    $display("3. Nested calls maintain separate variable scopes");
    $display("4. Recursive calls each have their own variable copies");
    $display();
    
    $display("Simulation completed successfully!");
    
  end
  
endmodule
```

Verilator Simulation Output:
=== Nested Function Call Variable Lifetime Demo ===

Test 1: First nested function call sequence
Outer function called: base=10, mult=3
  Inner function call #1: 10 * 3 = 30
First result: 40

Test 2: Second nested function call sequence
Outer function called: base=5, mult=4
  Inner function call #2: 5 * 4 = 20
Second result: 25

Test 3: Third nested function call sequence
Outer function called: base=7, mult=2
  Inner function call #3: 7 * 2 = 14
Third result: 21

Test 4: Recursive function with automatic variables
    Recursive call: number=4
    Recursive call: number=3
    Recursive call: number=2
    Base case reached: number=1
Factorial of 4: 24

=== Testbench Additional Tests ===

Testing testbench function variable lifetime:
Testbench wrapper called with value: 25
Testbench result: 250

Testbench wrapper called with value: 50
Testbench result: 300

=== Key Observations ===
1. Automatic variables create new instances for each call
2. Static variables p

0

## Pass by Reference

#### Array Sorting Tasks
Tasks that sort arrays in-place using pass by reference

In [13]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_23__array_sorting_tasks/"
files = ["array_sorting_design.sv", "array_sorting_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// array_sorting_design.sv
module array_sorting_engine();
  
  // Task to sort integer array in ascending order using bubble sort
  // Uses pass by reference (ref keyword) to modify array in-place
  task automatic sort_ascending_numbers(ref int unsorted_data_array[8]);
    int temp_swap_value;
    
    $display("Starting ascending sort of 8 elements");
    
    // Bubble sort implementation
    for (int outer_pass = 0; outer_pass < 7; outer_pass++) begin
      for (int inner_index = 0; inner_index < 7 - outer_pass; 
           inner_index++) begin
        if (unsorted_data_array[inner_index] > 
            unsorted_data_array[inner_index + 1]) begin
          // Swap elements
          temp_swap_value = unsorted_data_array[inner_index];
          unsorted_data_array[inner_index] = 
            unsorted_data_array[inner_index + 1];
          unsorted_data_array[inner_index + 1] = temp_swap_value;
        end
      end
    end
    $display("Ascending sort completed successfully");
  endtask

  // Task to sort integer array in descending order
  // Uses pass by reference to modify original array
  task automatic sort_descending_numbers(ref int unsorted_data_array[6]);
    int temp_swap_value;
    
    $display("Starting descending sort of 6 elements");
    
    // Bubble sort for descending order
    for (int outer_pass = 0; outer_pass < 5; outer_pass++) begin
      for (int inner_index = 0; inner_index < 5 - outer_pass; 
           inner_index++) begin
        if (unsorted_data_array[inner_index] < 
            unsorted_data_array[inner_index + 1]) begin
          // Swap elements
          temp_swap_value = unsorted_data_array[inner_index];
          unsorted_data_array[inner_index] = 
            unsorted_data_array[inner_index + 1];
          unsorted_data_array[inner_index + 1] = temp_swap_value;
        end
      end
    end
    $display("Descending sort completed successfully");
  endtask

  // Utility task to display 8-element array contents
  task display_array_8_contents(input int data_array[8], 
                                input string array_description);
    $write("%s: [", array_description);
    for (int element_index = 0; element_index < 8; element_index++) begin
      if (element_index > 0) $write(", ");
      $write("%0d", data_array[element_index]);
    end
    $display("]");
  endtask

  // Utility task to display 6-element array contents
  task display_array_6_contents(input int data_array[6], 
                                input string array_description);
    $write("%s: [", array_description);
    for (int element_index = 0; element_index < 6; element_index++) begin
      if (element_index > 0) $write(", ");
      $write("%0d", data_array[element_index]);
    end
    $display("]");
  endtask

  initial begin
    $display();
    $display("=== Array Sorting Tasks Demonstration ===");
  end

endmodule
```

```systemverilog
// array_sorting_design_testbench.sv
module array_sorting_testbench;
  
  // Instantiate the design under test
  array_sorting_engine SORTING_ENGINE_INSTANCE();

  // Declare test arrays at module level
  int random_numbers[8];
  int mixed_values[6];
  int duplicate_numbers[8];

  initial begin
    // Configure wave dumping
    $dumpfile("array_sorting_testbench.vcd");
    $dumpvars(0, array_sorting_testbench);
    
    // Initialize test data arrays
    random_numbers = '{64, 34, 25, 12, 22, 11, 90, 5};
    mixed_values = '{100, -50, 0, 75, -25, 200};
    duplicate_numbers = '{5, 2, 8, 2, 9, 1, 5, 5};
    
    $display();
    $display("=== Testing Array Sorting Tasks ===");
    $display();
    
    // Test 1: Sort random numbers in ascending order
    $display("--- Test 1: Ascending Sort ---");
    SORTING_ENGINE_INSTANCE.display_array_8_contents(random_numbers, 
                                                     "Original array");
    SORTING_ENGINE_INSTANCE.sort_ascending_numbers(random_numbers);
    SORTING_ENGINE_INSTANCE.display_array_8_contents(random_numbers, 
                                                     "Sorted ascending");
    $display();
    
    // Test 2: Sort mixed values in descending order
    $display("--- Test 2: Descending Sort ---");
    SORTING_ENGINE_INSTANCE.display_array_6_contents(mixed_values, 
                                                     "Original array");
    SORTING_ENGINE_INSTANCE.sort_descending_numbers(mixed_values);
    SORTING_ENGINE_INSTANCE.display_array_6_contents(mixed_values, 
                                                     "Sorted descending");
    $display();
    
    // Test 3: Sort array with duplicates in ascending order
    $display("--- Test 3: Duplicates Ascending Sort ---");
    SORTING_ENGINE_INSTANCE.display_array_8_contents(duplicate_numbers, 
                                                     "Original array");
    SORTING_ENGINE_INSTANCE.sort_ascending_numbers(duplicate_numbers);
    SORTING_ENGINE_INSTANCE.display_array_8_contents(duplicate_numbers, 
                                                     "Sorted ascending");
    $display();
    
    $display("All sorting tests completed successfully!");
    $display();
    
    #10; // Wait before finishing
  end

endmodule
```

Verilator Simulation Output:

=== Array Sorting Tasks Demonstration ===

=== Testing Array Sorting Tasks ===

--- Test 1: Ascending Sort ---
Original array: [64, 34, 25, 12, 22, 11, 90, 5]
Starting ascending sort of 8 elements
Ascending sort completed successfully
Sorted ascending: [5, 11, 12, 22, 25, 34, 64, 90]

--- Test 2: Descending Sort ---
Original array: [100, -50, 0, 75, -25, 200]
Starting descending sort of 6 elements
Descending sort completed successfully
Sorted descending: [200, 100, 75, 0, -25, -50]

--- Test 3: Duplicates Ascending Sort ---
Original array: [5, 2, 8, 2, 9, 1, 5, 5]
Starting ascending sort of 8 elements
Ascending sort completed successfully
Sorted ascending: [1, 2, 2, 5, 5, 5, 8, 9]

All sorting tests completed successfully!
Process finished with return code: 0
Removing Chapter_7_examples/example_23__array_sorting_tasks/obj_dir directory...
Chapter_7_examples/example_23__array_sorting_tasks/obj_dir removed successfully.


0

#### Matrix Operations
Functions performing matrix operations on large data structures

In [19]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_24__matrix_operations/"
files = ["matrix_adder_design.sv", "matrix_adder_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// matrix_adder_design.sv
module matrix_adder_processor #(
  parameter MATRIX_SIZE = 4,
  parameter DATA_WIDTH  = 8
)(
  input  logic                                    clock_signal,
  input  logic                                    reset_signal,
  input  logic                                    start_operation,
  input  logic [DATA_WIDTH-1:0]                  matrix_a_input 
                                                  [MATRIX_SIZE-1:0]
                                                  [MATRIX_SIZE-1:0],
  input  logic [DATA_WIDTH-1:0]                  matrix_b_input 
                                                  [MATRIX_SIZE-1:0]
                                                  [MATRIX_SIZE-1:0],
  output logic [DATA_WIDTH-1:0]                  matrix_result_output 
                                                  [MATRIX_SIZE-1:0]
                                                  [MATRIX_SIZE-1:0],
  output logic                                    operation_complete
);

  // Internal matrix storage
  logic [DATA_WIDTH-1:0] matrix_a_storage [MATRIX_SIZE-1:0]
                                          [MATRIX_SIZE-1:0];
  logic [DATA_WIDTH-1:0] matrix_b_storage [MATRIX_SIZE-1:0]
                                          [MATRIX_SIZE-1:0];
  logic [DATA_WIDTH-1:0] result_storage   [MATRIX_SIZE-1:0]
                                          [MATRIX_SIZE-1:0];

  // Control signals
  logic operation_in_progress;
  logic [3:0] row_counter;
  logic [3:0] column_counter;

  // Matrix addition function
  function automatic void perform_matrix_addition();
    for (int row_index = 0; row_index < MATRIX_SIZE; row_index++) begin
      for (int col_index = 0; col_index < MATRIX_SIZE; col_index++) begin
        result_storage[row_index][col_index] = 
          matrix_a_storage[row_index][col_index] + 
          matrix_b_storage[row_index][col_index];
      end
    end
  endfunction

  // Sequential logic for matrix operations
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      operation_complete <= 1'b0;
      operation_in_progress <= 1'b0;
      row_counter <= 4'b0;
      column_counter <= 4'b0;
      
      // Clear result matrix
      for (int i = 0; i < MATRIX_SIZE; i++) begin
        for (int j = 0; j < MATRIX_SIZE; j++) begin
          result_storage[i][j] <= '0;
        end
      end
      
    end else begin
      if (start_operation && !operation_in_progress) begin
        // Load input matrices and start operation
        matrix_a_storage <= matrix_a_input;
        matrix_b_storage <= matrix_b_input;
        operation_in_progress <= 1'b1;
        operation_complete <= 1'b0;
        
      end else if (operation_in_progress && !operation_complete) begin
        // Perform matrix addition in the next cycle after loading
        perform_matrix_addition();
        operation_complete <= 1'b1;
        operation_in_progress <= 1'b0;
        
      end else if (operation_complete) begin
        operation_complete <= 1'b0;
      end
    end
  end

  // Output assignment
  assign matrix_result_output = result_storage;

  // Display function for debugging
  function automatic void display_matrix_operation();
    $display("Matrix A:");
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      $write("  ");
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        $write("%3d ", matrix_a_storage[row][col]);
      end
      $display();
    end
    
    $display("Matrix B:");
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      $write("  ");
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        $write("%3d ", matrix_b_storage[row][col]);
      end
      $display();
    end
    
    $display("Result Matrix (A + B):");
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      $write("  ");
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        $write("%3d ", result_storage[row][col]);
      end
      $display();
    end
    $display();
  endfunction

endmodule
```

```systemverilog
// matrix_adder_design_testbench.sv
module matrix_adder_testbench;

  // Parameters matching the design
  parameter MATRIX_SIZE = 4;
  parameter DATA_WIDTH  = 8;
  parameter CLOCK_PERIOD = 10;

  // Testbench signals
  logic clock_signal;
  logic reset_signal;
  logic start_operation;
  logic [DATA_WIDTH-1:0] test_matrix_a [MATRIX_SIZE-1:0] [MATRIX_SIZE-1:0];
  logic [DATA_WIDTH-1:0] test_matrix_b [MATRIX_SIZE-1:0] [MATRIX_SIZE-1:0];
  logic [DATA_WIDTH-1:0] expected_result [MATRIX_SIZE-1:0] [MATRIX_SIZE-1:0];
  logic [DATA_WIDTH-1:0] actual_result [MATRIX_SIZE-1:0] [MATRIX_SIZE-1:0];
  logic operation_complete;

  // Test result tracking
  int successful_tests = 0;
  int total_tests = 0;

  // Instantiate the design under test
  matrix_adder_processor #(
    .MATRIX_SIZE(MATRIX_SIZE),
    .DATA_WIDTH(DATA_WIDTH)
  ) matrix_processor_instance (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .start_operation(start_operation),
    .matrix_a_input(test_matrix_a),
    .matrix_b_input(test_matrix_b),
    .matrix_result_output(actual_result),
    .operation_complete(operation_complete)
  );

  // Clock generation
  always #(CLOCK_PERIOD/2) clock_signal = ~clock_signal;

  // Task to initialize test matrices
  task automatic initialize_test_matrices();
    // Initialize Matrix A with incremental values
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        test_matrix_a[row][col] = DATA_WIDTH'(row * MATRIX_SIZE + col + 1);
      end
    end
    
    // Initialize Matrix B with different pattern
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        test_matrix_b[row][col] = DATA_WIDTH'((row + col) * 2);
      end
    end
  endtask

  // Task to calculate expected results
  task automatic calculate_expected_result();
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        expected_result[row][col] = test_matrix_a[row][col] + 
                                   test_matrix_b[row][col];
      end
    end
  endtask

  // Task to perform matrix addition test
  task automatic perform_matrix_addition_test();
    total_tests++;
    
    $display("=== Test %0d: Matrix Addition ===", total_tests);
    
    // Initialize matrices
    initialize_test_matrices();
    calculate_expected_result();
    
    // Start operation
    start_operation = 1'b1;
    @(posedge clock_signal);
    start_operation = 1'b0;
    
    // Wait for completion
    wait(operation_complete);
    @(posedge clock_signal);
    
    // Display matrices
    matrix_processor_instance.display_matrix_operation();
    
    // Verify results
    if (verify_matrix_results()) begin
      $display("Matrix addition test PASSED");
      successful_tests++;
    end else begin
      $display("Matrix addition test FAILED");
    end
    
    $display();
  endtask

  // Function to verify matrix results
  function automatic bit verify_matrix_results();
    bit test_passed = 1'b1;
    
    for (int row = 0; row < MATRIX_SIZE; row++) begin
      for (int col = 0; col < MATRIX_SIZE; col++) begin
        if (actual_result[row][col] !== expected_result[row][col]) begin
          $display("Mismatch at [%0d][%0d]: Expected=%0d, Got=%0d",
                  row, col, expected_result[row][col], 
                  actual_result[row][col]);
          test_passed = 1'b0;
        end
      end
    end
    
    return test_passed;
  endfunction

  // Task to perform system reset
  task automatic perform_system_reset();
    $display("Performing system reset...");
    reset_signal = 1'b1;
    repeat(3) @(posedge clock_signal);
    reset_signal = 1'b0;
    @(posedge clock_signal);
    $display("Reset complete");
    $display();
  endtask

  // Main test sequence
  initial begin
    // Initialize signals
    clock_signal = 1'b0;
    reset_signal = 1'b0;
    start_operation = 1'b0;
    
    // Setup waveform dumping
    $dumpfile("matrix_adder_testbench.vcd");
    $dumpvars(0, matrix_adder_testbench);
    
    $display("Starting Matrix Operations Testbench");
    $display("=====================================");
    $display();
    
    // Perform reset
    perform_system_reset();
    
    // Run matrix addition tests
    perform_matrix_addition_test();
    
    // Add small delay for waveform clarity
    repeat(5) @(posedge clock_signal);
    
    // Display final results
    $display("=== Test Summary ===");
    $display("Total Tests: %0d", total_tests);
    $display("Passed: %0d", successful_tests);
    $display("Failed: %0d", total_tests - successful_tests);
    
    if (successful_tests == total_tests) begin
      $display("All tests PASSED!");
    end else begin
      $display("Some tests FAILED!");
    end
    
    $display();
    $display("Matrix operations testbench complete!");
    $finish;
  end

  // Timeout watchdog
  initial begin
    #(CLOCK_PERIOD * 1000);
    $display("ERROR: Testbench timeout!");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Starting Matrix Operations Testbench

Performing system reset...
Reset complete

=== Test 1: Matrix Addition ===
Matrix A:
    1   2   3   4
    5   6   7   8
    9  10  11  12
   13  14  15  16
Matrix B:
    0   2   4   6
    2   4   6   8
    4   6   8  10
    6   8  10  12
Result Matrix (A + B):
    1   4   7  10
    7  10  13  16
   13  16  19  22
   19  22  25  28

Matrix addition test PASSED

=== Test Summary ===
Total Tests: 1
Passed: 1
Failed: 0
All tests PASSED!

Matrix operations testbench complete!
Process finished with return code: 0
Removing Chapter_7_examples/example_24__matrix_operations/obj_dir directory...
Chapter_7_examples/example_24__matrix_operations/obj_dir removed successfully.


0

#### Configuration Update Tasks
Tasks modifying system configuration structures by reference

In [21]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_25__configuration_update_tasks/"
files = ["system_config_manager.sv", "system_config_manager_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// system_config_manager.sv
module system_config_manager ();

  // System configuration structure
  typedef struct packed {
    logic [7:0] cpu_frequency_setting;
    logic [3:0] cache_size_level;
    logic       power_save_mode_enable;
    logic       debug_trace_enable;
  } system_config_struct;

  // Configuration registers
  system_config_struct current_system_config;
  system_config_struct backup_system_config;

  // Task to update CPU frequency configuration by reference
  task automatic update_cpu_frequency_config(
    ref system_config_struct config_to_modify,
    input logic [7:0] new_frequency_setting
  );
    $display("Updating CPU frequency from %d to %d", 
             config_to_modify.cpu_frequency_setting, new_frequency_setting);
    config_to_modify.cpu_frequency_setting = new_frequency_setting;
    $display("CPU frequency configuration updated successfully");
  endtask

  // Task to update cache configuration by reference
  task automatic update_cache_config(
    ref system_config_struct config_to_modify,
    input logic [3:0] new_cache_level
  );
    $display("Updating cache level from %d to %d", 
             config_to_modify.cache_size_level, new_cache_level);
    config_to_modify.cache_size_level = new_cache_level;
    $display("Cache configuration updated successfully");
  endtask

  // Task to update power management configuration by reference
  task automatic update_power_management_config(
    ref system_config_struct config_to_modify,
    input logic enable_power_save,
    input logic enable_debug_trace
  );
    $display("Updating power save: %b -> %b, debug trace: %b -> %b",
             config_to_modify.power_save_mode_enable, enable_power_save,
             config_to_modify.debug_trace_enable, enable_debug_trace);
    config_to_modify.power_save_mode_enable = enable_power_save;
    config_to_modify.debug_trace_enable = enable_debug_trace;
    $display("Power management configuration updated successfully");
  endtask

  // Task to backup current configuration by reference
  task automatic backup_current_config(
    ref system_config_struct source_config,
    ref system_config_struct destination_config
  );
    $display("Creating backup of current system configuration");
    destination_config = source_config;
    $display("Configuration backup completed");
  endtask

  // Display configuration status
  task automatic display_config_status(
    ref system_config_struct config_to_display,
    input string config_name
  );
    $display("=== %s Configuration Status ===", config_name);
    $display("CPU Frequency Setting: %d", 
             config_to_display.cpu_frequency_setting);
    $display("Cache Size Level: %d", config_to_display.cache_size_level);
    $display("Power Save Mode: %b", 
             config_to_display.power_save_mode_enable);
    $display("Debug Trace: %b", config_to_display.debug_trace_enable);
    $display("==============================");
  endtask

  initial begin
    $display("System Configuration Manager Initialization");
    
    // Initialize default configuration
    current_system_config.cpu_frequency_setting = 8'd100;
    current_system_config.cache_size_level = 4'd2;
    current_system_config.power_save_mode_enable = 1'b0;
    current_system_config.debug_trace_enable = 1'b1;
    
    $display("Default configuration loaded");
  end

endmodule
```

```systemverilog
// system_config_manager_testbench.sv
module config_update_testbench;

  // Instantiate the system configuration manager
  system_config_manager CONFIG_MANAGER_INSTANCE();

  initial begin
    // Setup waveform dumping
    $dumpfile("config_update_testbench.vcd");
    $dumpvars(0, config_update_testbench);
    
    $display();
    $display("Starting Configuration Update Tasks Demonstration");
    $display("================================================");
    
    // Wait for initialization
    #5;
    
    // Display initial configuration
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.current_system_config, "Initial"
    );
    
    #10;
    $display("\n--- Testing CPU Frequency Configuration Update ---");
    CONFIG_MANAGER_INSTANCE.update_cpu_frequency_config(
      CONFIG_MANAGER_INSTANCE.current_system_config, 8'd150
    );
    
    #5;
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.current_system_config, "After CPU Update"
    );
    
    #10;
    $display("\n--- Testing Cache Configuration Update ---");
    CONFIG_MANAGER_INSTANCE.update_cache_config(
      CONFIG_MANAGER_INSTANCE.current_system_config, 4'd5
    );
    
    #5;
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.current_system_config, "After Cache Update"
    );
    
    #10;
    $display("\n--- Testing Power Management Configuration Update ---");
    CONFIG_MANAGER_INSTANCE.update_power_management_config(
      CONFIG_MANAGER_INSTANCE.current_system_config, 1'b1, 1'b0
    );
    
    #5;
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.current_system_config, "After Power Update"
    );
    
    #10;
    $display("\n--- Testing Configuration Backup ---");
    CONFIG_MANAGER_INSTANCE.backup_current_config(
      CONFIG_MANAGER_INSTANCE.current_system_config,
      CONFIG_MANAGER_INSTANCE.backup_system_config
    );
    
    #5;
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.backup_system_config, "Backup"
    );
    
    #10;
    $display("\n--- Testing Multiple Configuration Updates ---");
    CONFIG_MANAGER_INSTANCE.update_cpu_frequency_config(
      CONFIG_MANAGER_INSTANCE.current_system_config, 8'd200
    );
    CONFIG_MANAGER_INSTANCE.update_cache_config(
      CONFIG_MANAGER_INSTANCE.current_system_config, 4'd7
    );
    
    #5;
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.current_system_config, "Final Current"
    );
    CONFIG_MANAGER_INSTANCE.display_config_status(
      CONFIG_MANAGER_INSTANCE.backup_system_config, "Final Backup"
    );
    
    #10;
    $display("\n================================================");
    $display("Configuration Update Tasks Demonstration Complete");
    $display();
    
    $finish;
  end

endmodule
```

Verilator Simulation Output:
System Configuration Manager Initialization
Default configuration loaded

Starting Configuration Update Tasks Demonstration
=== Initial Configuration Status ===
CPU Frequency Setting: 100
Cache Size Level:  2
Power Save Mode: 0
Debug Trace: 1

--- Testing CPU Frequency Configuration Update ---
Updating CPU frequency from 100 to 150
CPU frequency configuration updated successfully
=== After CPU Update Configuration Status ===
CPU Frequency Setting: 150
Cache Size Level:  2
Power Save Mode: 0
Debug Trace: 1

--- Testing Cache Configuration Update ---
Updating cache level from  2 to  5
Cache configuration updated successfully
=== After Cache Update Configuration Status ===
CPU Frequency Setting: 150
Cache Size Level:  5
Power Save Mode: 0
Debug Trace: 1

--- Testing Power Management Configuration Update ---
Updating power save: 0 -> 1, debug trace: 1 -> 0
Power management configuration updated successfully
=== After Power Update Configuration Status ===
CPU Fr

0

#### Buffer Management Functions
Functions managing circular buffers and memory pools

In [30]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_26__buffer_management/"
files = ["circular_buffer_manager.sv", "circular_buffer_manager_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// circular_buffer_manager.sv - Fixed Version
module circular_buffer_manager #(
  parameter BUFFER_DEPTH = 8,
  parameter DATA_WIDTH   = 8
)(
  input  logic                    clk,
  input  logic                    reset_n,
  input  logic                    write_enable,
  input  logic                    read_enable,
  input  logic [DATA_WIDTH-1:0]   write_data,
  output logic [DATA_WIDTH-1:0]   read_data,
  output logic                    buffer_full,
  output logic                    buffer_empty,
  output logic [$clog2(BUFFER_DEPTH):0] occupancy_count
);

  // Internal memory pool
  logic [DATA_WIDTH-1:0] memory_pool [BUFFER_DEPTH];
  
  // Circular buffer pointers  
  logic [$clog2(BUFFER_DEPTH)-1:0] write_pointer;
  logic [$clog2(BUFFER_DEPTH)-1:0] read_pointer;
  logic [$clog2(BUFFER_DEPTH):0]   data_count;

  // Buffer management functions
  function automatic logic is_buffer_full();
    return (data_count == BUFFER_DEPTH);
  endfunction

  function automatic logic is_buffer_empty();
    return (data_count == 0);
  endfunction

  function automatic logic [$clog2(BUFFER_DEPTH)-1:0] next_pointer(
    input logic [$clog2(BUFFER_DEPTH)-1:0] current_pointer
  );
    return (current_pointer == ($clog2(BUFFER_DEPTH)'(BUFFER_DEPTH-1))) ? 
           ($clog2(BUFFER_DEPTH)'(0)) : current_pointer + 1;
  endfunction

  // Memory pool allocation logic
  always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
      write_pointer  <= '0;
      read_pointer   <= '0;
      data_count     <= '0;
    end else begin
      // Handle simultaneous operations first
      if (write_enable && read_enable && !is_buffer_empty() && !is_buffer_full()) begin
        // Simultaneous read and write - data count stays same
        memory_pool[write_pointer] <= write_data;
        write_pointer <= next_pointer(write_pointer);
        read_pointer <= next_pointer(read_pointer);
        // data_count remains unchanged
      end else begin
        // Handle write-only operation
        if (write_enable && !is_buffer_full() && !(read_enable && !is_buffer_empty())) begin
          memory_pool[write_pointer] <= write_data;
          write_pointer <= next_pointer(write_pointer);
          data_count <= data_count + 1;
        end
        
        // Handle read-only operation  
        if (read_enable && !is_buffer_empty() && !(write_enable && !is_buffer_full())) begin
          read_pointer <= next_pointer(read_pointer);
          data_count <= data_count - 1;
        end
      end
    end
  end

  // Output assignments - combinatorial read
  assign read_data       = memory_pool[read_pointer];
  assign buffer_full     = is_buffer_full();
  assign buffer_empty    = is_buffer_empty();
  assign occupancy_count = data_count;

  initial $display("Circular buffer manager initialized with depth %0d", 
                   BUFFER_DEPTH);

endmodule
```

```systemverilog
// circular_buffer_manager_testbench.sv - Fixed Version
module buffer_manager_testbench;

  // Testbench parameters
  parameter BUFFER_DEPTH = 4;
  parameter DATA_WIDTH   = 8;
  parameter CLK_PERIOD   = 10;

  // Testbench signals
  logic                    clk;
  logic                    reset_n;
  logic                    write_enable;
  logic                    read_enable;
  logic [DATA_WIDTH-1:0]   write_data;
  logic [DATA_WIDTH-1:0]   read_data;
  logic                    buffer_full;
  logic                    buffer_empty;
  logic [$clog2(BUFFER_DEPTH):0] occupancy_count;

  // Test data storage
  logic [DATA_WIDTH-1:0] test_write_sequence [8] = 
    '{8'hAA, 8'hBB, 8'hCC, 8'hDD, 8'hEE, 8'hFF, 8'h11, 8'h22};
  logic [DATA_WIDTH-1:0] expected_read_data;
  logic [DATA_WIDTH-1:0] captured_read_data;

  // Instantiate design under test
  circular_buffer_manager #(
    .BUFFER_DEPTH(BUFFER_DEPTH),
    .DATA_WIDTH(DATA_WIDTH)
  ) buffer_manager_instance (
    .clk(clk),
    .reset_n(reset_n),
    .write_enable(write_enable),
    .read_enable(read_enable),
    .write_data(write_data),
    .read_data(read_data),
    .buffer_full(buffer_full),
    .buffer_empty(buffer_empty),
    .occupancy_count(occupancy_count)
  );

  // Clock generation
  initial begin
    clk = 0;
    forever #(CLK_PERIOD/2) clk = ~clk;
  end

  // Test stimulus and buffer management verification
  initial begin
    // Dump waves for buffer analysis
    $dumpfile("buffer_manager_testbench.vcd");
    $dumpvars(0, buffer_manager_testbench);

    $display();
    $display("=== Buffer Management Functions Test ===");
    
    // Initialize signals
    reset_n      = 0;
    write_enable = 0;
    read_enable  = 0;
    write_data   = 0;
    
    // Reset sequence
    repeat(2) @(posedge clk);
    reset_n = 1;
    @(posedge clk);
    
    $display("Initial state - Empty: %0b, Full: %0b, Count: %0d", 
             buffer_empty, buffer_full, occupancy_count);

    // Test 1: Fill circular buffer completely
    $display("\n--- Test 1: Filling circular buffer ---");
    for (int write_index = 0; write_index < BUFFER_DEPTH; write_index++) begin
      write_enable = 1;
      write_data = test_write_sequence[write_index];
      $display("Writing data: 0x%02X, Count: %0d, Full: %0b", 
               write_data, occupancy_count, buffer_full);
      @(posedge clk);
    end
    write_enable = 0;

    // Test 2: Attempt overflow write
    $display("\n--- Test 2: Testing buffer overflow protection ---");
    write_enable = 1;
    write_data = 8'hFF;
    $display("Overflow attempt - Full: %0b, Count: %0d", 
             buffer_full, occupancy_count);
    @(posedge clk);
    write_enable = 0;

    // Test 3: Read from circular buffer
    $display("\n--- Test 3: Reading from circular buffer ---");
    for (int read_index = 0; read_index < BUFFER_DEPTH; read_index++) begin
      expected_read_data = test_write_sequence[read_index];
      
      // Capture read data BEFORE applying read enable
      captured_read_data = read_data;
      
      $display("Before read - Read ptr: %0d, Write ptr: %0d, Data[0-3]: %02X %02X %02X %02X", 
               buffer_manager_instance.read_pointer, 
               buffer_manager_instance.write_pointer,
               buffer_manager_instance.memory_pool[0],
               buffer_manager_instance.memory_pool[1], 
               buffer_manager_instance.memory_pool[2],
               buffer_manager_instance.memory_pool[3]);
      
      $display("Read data: 0x%02X, Expected: 0x%02X, Match: %0b, Count: %0d", 
               captured_read_data, expected_read_data, 
               (captured_read_data == expected_read_data), occupancy_count);
      
      // Apply read enable and clock edge
      read_enable = 1;
      @(posedge clk);
      read_enable = 0;
    end

    // Test 4: Attempt underflow read
    $display("\n--- Test 4: Testing buffer underflow protection ---");
    read_enable = 1;
    $display("Underflow attempt - Empty: %0b, Count: %0d", 
             buffer_empty, occupancy_count);
    @(posedge clk);
    read_enable = 0;

    // Test 5: Simultaneous read/write operations
    $display("\n--- Test 5: Simultaneous buffer operations ---");
    // First write some data to have something to read
    write_enable = 1;
    write_data = test_write_sequence[4]; // 0xEE
    @(posedge clk);
    write_enable = 0;
    
    $display("Setup for simultaneous ops - Count: %0d", occupancy_count);
    
    for (int sim_index = 5; sim_index < 8; sim_index++) begin
      // Capture current read data before operation
      captured_read_data = read_data;
      
      // Set up simultaneous read/write
      write_enable = 1;
      read_enable  = 1;
      write_data = test_write_sequence[sim_index];
      
      $display("Simultaneous - Write: 0x%02X, Read: 0x%02X, Count before: %0d", 
               write_data, captured_read_data, occupancy_count);
      
      @(posedge clk); // Execute both operations
      
      $display("After simultaneous op - Count: %0d", occupancy_count);
      
      write_enable = 0;
      read_enable  = 0;
    end

    repeat(2) @(posedge clk);
    $display("\n=== Buffer Management Test Complete ===");
    $display();
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Circular buffer manager initialized with depth 4

=== Buffer Management Functions Test ===
Initial state - Empty: 1, Full: 0, Count: 0

--- Test 1: Filling circular buffer ---
Writing data: 0xaa, Count: 0, Full: 0
Writing data: 0xbb, Count: 1, Full: 0
Writing data: 0xcc, Count: 2, Full: 0
Writing data: 0xdd, Count: 3, Full: 0

--- Test 2: Testing buffer overflow protection ---
Overflow attempt - Full: 1, Count: 4

--- Test 3: Reading from circular buffer ---
Before read - Read ptr: 0, Write ptr: 0, Data[0-3]: aa bb cc dd
Read data: 0xaa, Expected: 0xaa, Match: 1, Count: 4
Before read - Read ptr: 1, Write ptr: 0, Data[0-3]: aa bb cc dd
Read data: 0xbb, Expected: 0xbb, Match: 1, Count: 3
Before read - Read ptr: 2, Write ptr: 0, Data[0-3]: aa bb cc dd
Read data: 0xcc, Expected: 0xcc, Match: 1, Count: 2
Before read - Read ptr: 3, Write ptr: 0, Data[0-3]: aa bb cc dd
Read data: 0xdd, Expected: 0xdd, Match: 1, Count: 1

--- Test 4: Testing buffer underflow protec

0

#### State Machine Tasks
Tasks updating complex state machine structures

In [2]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_27__state_machine_tasks/"
files = [
    "traffic_light_controller.sv",
    "traffic_light_controller_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// traffic_light_controller.sv
module traffic_light_controller (
  input  logic clock_signal,
  input  logic reset_signal,
  input  logic pedestrian_button_pressed,
  output logic red_light_active,
  output logic yellow_light_active,
  output logic green_light_active
);

  // State definitions
  typedef enum logic [2:0] {
    IDLE_STATE           = 3'b000,
    GREEN_ACTIVE_STATE   = 3'b001,
    YELLOW_ACTIVE_STATE  = 3'b010,
    RED_ACTIVE_STATE     = 3'b011,
    PEDESTRIAN_STATE     = 3'b100
  } traffic_state_type;

  // State machine variables
  traffic_state_type current_state, next_state;
  logic [3:0] timer_counter;
  logic pedestrian_crossing_requested;

  // State machine task to update complex state structure
  task update_traffic_state_machine(
    input traffic_state_type new_state,
    input logic [3:0] timer_value,
    input logic pedestrian_request
  );
    begin
      current_state = new_state;
      timer_counter = timer_value;
      pedestrian_crossing_requested = pedestrian_request;
      
      // Update output lights based on state
      case (current_state)
        IDLE_STATE: begin
          red_light_active = 1'b1;
          yellow_light_active = 1'b0;
          green_light_active = 1'b0;
        end
        GREEN_ACTIVE_STATE: begin
          red_light_active = 1'b0;
          yellow_light_active = 1'b0;
          green_light_active = 1'b1;
        end
        YELLOW_ACTIVE_STATE: begin
          red_light_active = 1'b0;
          yellow_light_active = 1'b1;
          green_light_active = 1'b0;
        end
        RED_ACTIVE_STATE, PEDESTRIAN_STATE: begin
          red_light_active = 1'b1;
          yellow_light_active = 1'b0;
          green_light_active = 1'b0;
        end
        default: begin
          red_light_active = 1'b1;
          yellow_light_active = 1'b0;
          green_light_active = 1'b0;
        end
      endcase
    end
  endtask

  // Initialize state machine
  initial begin
    update_traffic_state_machine(IDLE_STATE, 4'b0000, 1'b0);
  end

  // State machine clock process
  always @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      update_traffic_state_machine(IDLE_STATE, 4'b0000, 1'b0);
    end else begin
      // Update pedestrian request
      if (pedestrian_button_pressed) 
        pedestrian_crossing_requested = 1'b1;

      // State transitions with timer
      case (current_state)
        IDLE_STATE: begin
          update_traffic_state_machine(GREEN_ACTIVE_STATE, 4'b0000, 
                                     pedestrian_crossing_requested);
        end
        
        GREEN_ACTIVE_STATE: begin
          if (timer_counter >= 4'd5) begin
            if (pedestrian_crossing_requested)
              update_traffic_state_machine(YELLOW_ACTIVE_STATE, 4'b0000, 
                                         pedestrian_crossing_requested);
            else
              update_traffic_state_machine(GREEN_ACTIVE_STATE, 4'b0000, 
                                         pedestrian_crossing_requested);
          end else begin
            update_traffic_state_machine(GREEN_ACTIVE_STATE, 
                                       timer_counter + 1'b1, 
                                       pedestrian_crossing_requested);
          end
        end
        
        YELLOW_ACTIVE_STATE: begin
          if (timer_counter >= 4'd2) begin
            if (pedestrian_crossing_requested)
              update_traffic_state_machine(PEDESTRIAN_STATE, 4'b0000, 1'b1);
            else
              update_traffic_state_machine(RED_ACTIVE_STATE, 4'b0000, 1'b0);
          end else begin
            update_traffic_state_machine(YELLOW_ACTIVE_STATE, 
                                       timer_counter + 1'b1, 
                                       pedestrian_crossing_requested);
          end
        end
        
        RED_ACTIVE_STATE: begin
          if (timer_counter >= 4'd3) begin
            update_traffic_state_machine(GREEN_ACTIVE_STATE, 4'b0000, 1'b0);
          end else begin
            update_traffic_state_machine(RED_ACTIVE_STATE, 
                                       timer_counter + 1'b1, 
                                       pedestrian_crossing_requested);
          end
        end
        
        PEDESTRIAN_STATE: begin
          if (timer_counter >= 4'd4) begin
            update_traffic_state_machine(GREEN_ACTIVE_STATE, 4'b0000, 1'b0);
          end else begin
            update_traffic_state_machine(PEDESTRIAN_STATE, 
                                       timer_counter + 1'b1, 1'b1);
          end
        end
        
        default: begin
          update_traffic_state_machine(IDLE_STATE, 4'b0000, 1'b0);
        end
      endcase
    end
  end

endmodule
```

```systemverilog
// traffic_light_controller_testbench.sv
module traffic_light_controller_testbench;

  // Testbench signals
  logic clock_signal;
  logic reset_signal;
  logic pedestrian_button_pressed;
  logic red_light_active;
  logic yellow_light_active;
  logic green_light_active;

  // Instantiate design under test
  traffic_light_controller TRAFFIC_CONTROLLER_INSTANCE (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .pedestrian_button_pressed(pedestrian_button_pressed),
    .red_light_active(red_light_active),
    .yellow_light_active(yellow_light_active),
    .green_light_active(green_light_active)
  );

  // Task to display current traffic light state
  task display_traffic_light_status();
    begin
      $display("Time: %0t | R:%b Y:%b G:%b | State: %s | Ped: %b", 
               $time,
               red_light_active,
               yellow_light_active, 
               green_light_active,
               get_state_name(),
               pedestrian_button_pressed);
    end
  endtask

  // Function to get readable state name
  function string get_state_name();
    case ({red_light_active, yellow_light_active, green_light_active})
      3'b100: return "RED";
      3'b010: return "YELLOW";
      3'b001: return "GREEN";
      default: return "UNKNOWN";
    endcase
  endfunction

  // Task to simulate pedestrian button press
  task simulate_pedestrian_crossing_request();
    begin
      $display("\n--- Pedestrian crossing requested ---");
      pedestrian_button_pressed = 1'b1;
      #20;
      pedestrian_button_pressed = 1'b0;
    end
  endtask

  // Task to wait for specific light state
  task wait_for_light_state(input logic red, yellow, green);
    begin
      while (!(red_light_active == red && 
               yellow_light_active == yellow && 
               green_light_active == green)) begin
        #10;
      end
      $display("Reached expected light state: R:%b Y:%b G:%b", 
               red, yellow, green);
    end
  endtask

  // Clock generation - limited duration
  initial begin
    clock_signal = 0;
    repeat (200) #10 clock_signal = ~clock_signal;  // Limited clock cycles
  end

  // Test sequence
  initial begin
    // Initialize waveform dump
    $dumpfile("traffic_light_controller_testbench.vcd");
    $dumpvars(0, traffic_light_controller_testbench);

    $display("\n=== Traffic Light Controller State Machine Test ===");
    $display("Testing state machine tasks for complex state updates\n");

    // Initialize signals
    reset_signal = 1'b0;
    pedestrian_button_pressed = 1'b0;

    // Apply reset
    $display("--- Applying Reset ---");
    reset_signal = 1'b1;
    #30;
    reset_signal = 1'b0;
    #10;

    // Monitor normal traffic light sequence
    $display("\n--- Normal Traffic Light Sequence ---");
    repeat (10) begin  // Reduced from 20
      display_traffic_light_status();
      #20;
    end

    // Test pedestrian crossing during green light
    wait_for_light_state(1'b0, 1'b0, 1'b1);  // Wait for green
    simulate_pedestrian_crossing_request();
    
    $display("\n--- Monitoring Pedestrian Crossing Sequence ---");
    repeat (15) begin  // Reduced from 25
      display_traffic_light_status();
      #20;
    end

    // Test reset during operation
    $display("\n--- Testing Reset During Operation ---");
    reset_signal = 1'b1;
    #20;
    reset_signal = 1'b0;
    
    repeat (8) begin  // Reduced from 10
      display_traffic_light_status();
      #20;
    end

    $display("\n=== Test Completed Successfully ===");
    $display("State machine tasks effectively managed complex state updates");
    #50;  // Small delay before finish
    $finish;
  end

  // Monitor for unexpected states - simplified
  always @(posedge clock_signal) begin
    if (!reset_signal && $time > 100) begin  // Skip initial period
      // Check for invalid light combinations
      if ((red_light_active + yellow_light_active + green_light_active) != 1) begin
        $error("Invalid light combination detected at time %0t", $time);
        $finish;
      end
    end
  end

endmodule
```

Verilator Simulation Output:

=== Traffic Light Controller State Machine Test ===
Testing state machine tasks for complex state updates

--- Applying Reset ---

--- Normal Traffic Light Sequence ---
Time: 40 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 60 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 80 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 100 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 120 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 140 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 160 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 180 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 200 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 220 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Reached expected light state: R:0 Y:0 G:1

--- Pedestrian crossing requested ---

--- Monitoring Pedestrian Crossing Sequence ---
Time: 260 | R:0 Y:0 G:1 | State: GREEN | Ped: 0
Time: 280 | R:0 Y:1 G:0 | State: YELLOW | Ped: 0
Time: 300 | R:0 Y:1 G:0 | State: YELLOW | Ped: 0
Time: 320 | R:0 Y:1 G:0 | State: YELLOW | Ped: 0

0

## Return Statements in Functions

#### Input Validation Functions
Functions with multiple return points for different validation cases

In [2]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_28__input_validation_functions/"
files = ["input_validator_module.sv", "input_validator_module_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// input_validator_module.sv
module input_validator_module ();

  // Function to validate range with multiple return points
  function automatic logic validate_range(input logic [7:0] data_value,
                                          input logic [7:0] min_limit,
                                          input logic [7:0] max_limit);
    // Check if minimum is greater than maximum (invalid range)
    if (min_limit > max_limit) begin
      $display("ERROR: Invalid range - min (%0d) > max (%0d)", 
               min_limit, max_limit);
      return 1'b0;  // Return failure immediately
    end
    
    // Check if value is below minimum
    if (data_value < min_limit) begin
      $display("WARNING: Value %0d below minimum %0d", 
               data_value, min_limit);
      return 1'b0;  // Return failure immediately
    end
    
    // Check if value is above maximum
    if (data_value > max_limit) begin
      $display("WARNING: Value %0d above maximum %0d", 
               data_value, max_limit);
      return 1'b0;  // Return failure immediately
    end
    
    // Value is within valid range
    $display("SUCCESS: Value %0d is within range [%0d:%0d]", 
             data_value, min_limit, max_limit);
    return 1'b1;  // Return success
  endfunction

  // Function to validate password strength with multiple criteria
  function automatic logic [1:0] check_password_strength(
                                   input logic [31:0] password_length,
                                   input logic has_uppercase_char,
                                   input logic has_number_digit,
                                   input logic has_special_symbol);
    
    // Check minimum length requirement first
    if (password_length < 8) begin
      $display("WEAK: Password too short (%0d chars)", password_length);
      return 2'b00;  // Return weak immediately
    end
    
    // Check for uppercase requirement
    if (!has_uppercase_char) begin
      $display("WEAK: Missing uppercase character");
      return 2'b00;  // Return weak immediately
    end
    
    // Determine strength based on remaining criteria
    if (has_number_digit && has_special_symbol) begin
      $display("STRONG: All criteria met (len=%0d)", password_length);
      return 2'b10;  // Return strong
    end else if (has_number_digit || has_special_symbol) begin
      $display("MEDIUM: Some criteria met (len=%0d)", password_length);
      return 2'b01;  // Return medium
    end else begin
      $display("WEAK: Missing numbers and symbols");
      return 2'b00;  // Return weak
    end
  endfunction

  initial begin
    logic        validation_result;
    logic [1:0]  strength_rating;
    
    $display("=== Input Validation Functions Demo ===");
    $display();
    
    // Test range validation function
    $display("--- Range Validation Tests ---");
    validation_result = validate_range(50, 10, 100);
    validation_result = validate_range(5, 10, 100);
    validation_result = validate_range(150, 10, 100);
    validation_result = validate_range(75, 100, 50);  // Invalid range
    $display();
    
    // Test password strength function
    $display("--- Password Strength Tests ---");
    strength_rating = check_password_strength(12, 1'b1, 1'b1, 1'b1);
    strength_rating = check_password_strength(10, 1'b1, 1'b1, 1'b0);
    strength_rating = check_password_strength(8, 1'b1, 1'b0, 1'b0);
    strength_rating = check_password_strength(6, 1'b0, 1'b1, 1'b1);
    strength_rating = check_password_strength(8, 1'b0, 1'b0, 1'b0);
    $display();
  end

endmodule
```

```systemverilog
// input_validator_module_testbench.sv
module input_validator_testbench;

  // Instantiate design under test
  input_validator_module INPUT_VALIDATOR_DUT();

  // Test variables
  logic        range_check_result;
  logic [1:0]  password_strength_level;
  integer      test_counter;

  initial begin
    // Initialize VCD dump
    $dumpfile("input_validator_testbench.vcd");
    $dumpvars(0, input_validator_testbench);
    
    test_counter = 0;
    
    $display("=== Input Validation Functions Testbench ===");
    $display("Testing functions with multiple return points");
    $display();
    
    // Additional comprehensive range validation tests
    $display("--- Extended Range Validation Tests ---");
    
    test_counter++;
    $display("Test %0d: Valid boundary values", test_counter);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(10, 10, 10);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(0, 0, 255);
    #1;
    
    test_counter++;
    $display("Test %0d: Edge case testing", test_counter);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(255, 0, 255);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(128, 50, 200);
    #1;
    
    test_counter++;
    $display("Test %0d: Error condition testing", test_counter);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(100, 200, 50);
    range_check_result = INPUT_VALIDATOR_DUT.validate_range(25, 50, 40);
    #1;
    
    $display();
    
    // Additional comprehensive password strength tests
    $display("--- Extended Password Strength Tests ---");
    
    test_counter++;
    $display("Test %0d: Minimum length boundary", test_counter);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                8, 1'b1, 1'b1, 1'b1);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                7, 1'b1, 1'b1, 1'b1);
    #1;
    
    test_counter++;
    $display("Test %0d: Various combinations", test_counter);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                15, 1'b1, 1'b0, 1'b1);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                20, 1'b1, 1'b1, 1'b0);
    #1;
    
    test_counter++;
    $display("Test %0d: Weak password scenarios", test_counter);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                12, 1'b0, 1'b1, 1'b1);
    password_strength_level = INPUT_VALIDATOR_DUT.check_password_strength(
                                4, 1'b1, 1'b1, 1'b1);
    #1;
    
    $display();
    $display("=== Testbench completed successfully ===");
    $display("Demonstrated multiple return points in validation functions");
    #10;
    
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Input Validation Functions Demo ===

--- Range Validation Tests ---
SUCCESS: Value 50 is within range [10:100]
ERROR: Invalid range - min (100) > max (50)

--- Password Strength Tests ---
STRONG: All criteria met (len=12)
MEDIUM: Some criteria met (len=10)
WEAK: Missing numbers and symbols
WEAK: Password too short (6 chars)
WEAK: Missing uppercase character

=== Input Validation Functions Testbench ===
Testing functions with multiple return points

--- Extended Range Validation Tests ---
Test 1: Valid boundary values
SUCCESS: Value 10 is within range [10:10]
SUCCESS: Value 0 is within range [0:255]
Test 2: Edge case testing
SUCCESS: Value 255 is within range [0:255]
SUCCESS: Value 128 is within range [50:200]
Test 3: Error condition testing
ERROR: Invalid range - min (200) > max (50)
ERROR: Invalid range - min (50) > max (40)

--- Extended Password Strength Tests ---
Test 4: Minimum length boundary
STRONG: All criteria met (len=8)
WEAK: Password too sho

0

#### Search Functions
Functions with early returns for efficient searching algorithms

In [4]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_29__search_functions/"
files = [
    "search_algorithms_module.sv",
    "search_algorithms_module_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// search_algorithms_module.sv
module search_algorithms_module();

  // Function: Linear search with early return for efficiency
  function automatic int linear_search_with_early_return(
    input int search_array[8],
    input int target_value
  );
    for (int array_index = 0; array_index < 8; array_index++) begin
      if (search_array[array_index] == target_value) begin
        $display("Linear search: Found %0d at index %0d", 
                 target_value, array_index);
        return array_index;  // Early return when found
      end
    end
    $display("Linear search: Value %0d not found", target_value);
    return -1;  // Return -1 if not found
  endfunction

  // Function: Binary search with early return (assumes sorted array)
  function automatic int binary_search_with_early_return(
    input int sorted_array[8],
    input int target_value
  );
    int left_boundary = 0;
    int right_boundary = 7;
    int middle_index;
    
    while (left_boundary <= right_boundary) begin
      middle_index = (left_boundary + right_boundary) / 2;
      
      if (sorted_array[middle_index] == target_value) begin
        $display("Binary search: Found %0d at index %0d", 
                 target_value, middle_index);
        return middle_index;  // Early return when found
      end
      else if (sorted_array[middle_index] < target_value) begin
        left_boundary = middle_index + 1;
      end
      else begin
        right_boundary = middle_index - 1;
      end
    end
    
    $display("Binary search: Value %0d not found", target_value);
    return -1;  // Return -1 if not found
  endfunction

  // Function: Search for first occurrence with early return
  function automatic int find_first_occurrence_with_early_return(
    input int data_array[8],
    input int search_target
  );
    for (int position = 0; position < 8; position++) begin
      if (data_array[position] == search_target) begin
        $display("First occurrence: Found %0d at position %0d", 
                 search_target, position);
        return position;  // Early return on first match
      end
    end
    $display("First occurrence: Value %0d not found", search_target);
    return -1;
  endfunction

  initial begin
    $display("=== Search Functions with Early Returns Demo ===");
    $display();
  end

endmodule
```

```systemverilog
// search_algorithms_module_testbench.sv
module search_testbench_module;

  // Instantiate design under test
  search_algorithms_module SEARCH_ALGORITHM_INSTANCE();

  // Test data arrays
  int test_array[8] = '{10, 25, 33, 47, 52, 68, 74, 89};
  int sorted_test_array[8] = '{10, 25, 33, 47, 52, 68, 74, 89};
  int duplicate_array[8] = '{15, 25, 25, 25, 40, 55, 70, 85};
  
  int search_result;

  initial begin
    // Setup waveform dumping
    $dumpfile("search_testbench_module.vcd");
    $dumpvars(0, search_testbench_module);
    
    $display("=== Testing Search Functions with Early Returns ===");
    $display();
    
    // Display test arrays
    $display("Test array: %p", test_array);
    $display("Sorted array: %p", sorted_test_array);
    $display("Array with duplicates: %p", duplicate_array);
    $display();
    
    // Test linear search with early return
    $display("--- Linear Search Tests ---");
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    linear_search_with_early_return(test_array, 47);
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    linear_search_with_early_return(test_array, 99);
    $display();
    
    // Test binary search with early return
    $display("--- Binary Search Tests ---");
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    binary_search_with_early_return(sorted_test_array, 52);
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    binary_search_with_early_return(sorted_test_array, 30);
    $display();
    
    // Test first occurrence search with early return
    $display("--- First Occurrence Search Tests ---");
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    find_first_occurrence_with_early_return(
                      duplicate_array, 25);
    search_result = SEARCH_ALGORITHM_INSTANCE.
                    find_first_occurrence_with_early_return(
                      duplicate_array, 60);
    $display();
    
    $display("=== Search Functions Testing Complete ===");
    $display();
    
    #10;  // Wait before finishing
  end

endmodule
```

Verilator Simulation Output:
=== Search Functions with Early Returns Demo ===

=== Testing Search Functions with Early Returns ===

Test array: '{'ha, 'h19, 'h21, 'h2f, 'h34, 'h44, 'h4a, 'h59}
Sorted array: '{'ha, 'h19, 'h21, 'h2f, 'h34, 'h44, 'h4a, 'h59}
Array with duplicates: '{'hf, 'h19, 'h19, 'h19, 'h28, 'h37, 'h46, 'h55}

--- Linear Search Tests ---
Linear search: Found 47 at index 3
Linear search: Value 99 not found

--- Binary Search Tests ---
Binary search: Found 52 at index 4
Binary search: Value 30 not found

--- First Occurrence Search Tests ---
First occurrence: Found 25 at position 1
First occurrence: Value 60 not found

=== Search Functions Testing Complete ===
Process finished with return code: 0
Removing Chapter_7_examples/example_29__search_functions/obj_dir directory...
Chapter_7_examples/example_29__search_functions/obj_dir removed successfully.


0

#### Error Handling Functions
Functions demonstrating proper error detection and return patterns

In [12]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_30__error_handling_functions/"
files = ["error_handler_processor.sv", "error_handler_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// error_handler_processor.sv

module arithmetic_error_handler();

  // Error codes as parameters for easy access
  localparam logic [2:0] ERROR_NONE        = 3'b000;
  localparam logic [2:0] ERROR_DIVIDE_ZERO = 3'b001;
  localparam logic [2:0] ERROR_OVERFLOW    = 3'b010;
  localparam logic [2:0] ERROR_UNDERFLOW   = 3'b011;
  localparam logic [2:0] ERROR_INVALID_OP  = 3'b100;

  // Function: Safe division with error detection
  function automatic logic [15:0] safe_divide_function(
    input logic [7:0] dividend_input,
    input logic [7:0] divisor_input,
    output logic [2:0] error_status_output
  );
    logic [15:0] result_value;
    
    // Check for division by zero
    if (divisor_input == 8'h00) begin
      error_status_output = ERROR_DIVIDE_ZERO;
      result_value = 16'hFFFF;  // Error indicator value
      $display("ERROR: Division by zero detected!");
    end
    else begin
      error_status_output = ERROR_NONE;
      result_value = {8'h00, dividend_input} / {8'h00, divisor_input};
      $display("Division successful: %0d / %0d = %0d", 
               dividend_input, divisor_input, result_value);
    end
    
    return result_value;
  endfunction

  // Function: Safe addition with overflow detection
  function automatic logic [8:0] safe_add_function(
    input logic [7:0] operand_a_input,
    input logic [7:0] operand_b_input,
    output logic [2:0] error_status_output
  );
    logic [8:0] extended_result;
    
    extended_result = operand_a_input + operand_b_input;
    
    // Check for 8-bit overflow
    if (extended_result > 9'h0FF) begin
      error_status_output = ERROR_OVERFLOW;
      $display("ERROR: Addition overflow detected! %0d + %0d = %0d", 
               operand_a_input, operand_b_input, extended_result);
    end
    else begin
      error_status_output = ERROR_NONE;
      $display("Addition successful: %0d + %0d = %0d", 
               operand_a_input, operand_b_input, extended_result[7:0]);
    end
    
    return extended_result;
  endfunction

  // Function: Input validation with error return
  function automatic logic validate_input_range_function(
    input logic [7:0] data_input,
    input logic [7:0] min_boundary,
    input logic [7:0] max_boundary,
    output logic [2:0] error_status_output
  );
    logic validation_result;
    
    if (data_input < min_boundary || data_input > max_boundary) begin
      error_status_output = ERROR_INVALID_OP;
      validation_result = 1'b0;
      $display("ERROR: Input %0d outside valid range [%0d:%0d]", 
               data_input, min_boundary, max_boundary);
    end
    else begin
      error_status_output = ERROR_NONE;
      validation_result = 1'b1;
      $display("Input validation passed: %0d within [%0d:%0d]", 
               data_input, min_boundary, max_boundary);
    end
    
    return validation_result;
  endfunction

  initial begin
    $display("=== Error Handling Functions Demonstration ===");
    $display();
  end

endmodule
```

```systemverilog
// error_handler_processor_testbench.sv
module error_handler_testbench;

  // Error codes as parameters matching the design
  localparam logic [2:0] ERROR_NONE        = 3'b000;
  localparam logic [2:0] ERROR_DIVIDE_ZERO = 3'b001;
  localparam logic [2:0] ERROR_OVERFLOW    = 3'b010;
  localparam logic [2:0] ERROR_UNDERFLOW   = 3'b011;
  localparam logic [2:0] ERROR_INVALID_OP  = 3'b100;

  // Instantiate design under test
  arithmetic_error_handler ERROR_HANDLER_INSTANCE();

  // Test variables
  logic [15:0] division_result;
  logic [8:0] addition_result;
  logic validation_result;
  logic [2:0] error_status;

  // Function to convert error code to string for display
  function automatic string error_code_to_string(input logic [2:0] code);
    case (code)
      ERROR_NONE:        return "ERROR_NONE";
      ERROR_DIVIDE_ZERO: return "ERROR_DIVIDE_ZERO";
      ERROR_OVERFLOW:    return "ERROR_OVERFLOW";
      ERROR_UNDERFLOW:   return "ERROR_UNDERFLOW";
      ERROR_INVALID_OP:  return "ERROR_INVALID_OP";
      default:           return "UNKNOWN_ERROR";
    endcase
  endfunction

  initial begin
    // Dump waves
    $dumpfile("error_handler_testbench.vcd");
    $dumpvars(0, error_handler_testbench);

    $display("=== Testing Error Handling Functions ===");
    $display();

    // Test 1: Safe division - normal operation
    $display("Test 1: Normal Division");
    division_result = ERROR_HANDLER_INSTANCE.safe_divide_function(
      8'd20, 8'd4, error_status);
    $display("Result: %0d, Error: %s", division_result, 
             error_code_to_string(error_status));
    $display();

    // Test 2: Safe division - divide by zero error
    $display("Test 2: Division by Zero Error");
    division_result = ERROR_HANDLER_INSTANCE.safe_divide_function(
      8'd15, 8'd0, error_status);
    $display("Result: 0x%04X, Error: %s", division_result, 
             error_code_to_string(error_status));
    $display();

    // Test 3: Safe addition - normal operation
    $display("Test 3: Normal Addition");
    addition_result = ERROR_HANDLER_INSTANCE.safe_add_function(
      8'd100, 8'd50, error_status);
    $display("Result: %0d, Error: %s", addition_result[7:0], 
             error_code_to_string(error_status));
    $display();

    // Test 4: Safe addition - overflow error
    $display("Test 4: Addition Overflow Error");
    addition_result = ERROR_HANDLER_INSTANCE.safe_add_function(
      8'd200, 8'd100, error_status);
    $display("Result: %0d (truncated to 8-bit), Error: %s", 
             addition_result[7:0], error_code_to_string(error_status));
    $display();

    // Test 5: Input validation - valid range
    $display("Test 5: Input Validation - Valid Range");
    validation_result = ERROR_HANDLER_INSTANCE.validate_input_range_function(
      8'd75, 8'd10, 8'd100, error_status);
    $display("Valid: %b, Error: %s", validation_result, 
             error_code_to_string(error_status));
    $display();

    // Test 6: Input validation - invalid range (too low)
    $display("Test 6: Input Validation - Below Range");
    validation_result = ERROR_HANDLER_INSTANCE.validate_input_range_function(
      8'd5, 8'd10, 8'd100, error_status);
    $display("Valid: %b, Error: %s", validation_result, 
             error_code_to_string(error_status));
    $display();

    // Test 7: Input validation - invalid range (too high)
    $display("Test 7: Input Validation - Above Range");
    validation_result = ERROR_HANDLER_INSTANCE.validate_input_range_function(
      8'd150, 8'd10, 8'd100, error_status);
    $display("Valid: %b, Error: %s", validation_result, 
             error_code_to_string(error_status));
    $display();

    $display("=== Error Handling Functions Test Complete ===");
    #1;  // Wait for a time unit
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Error Handling Functions Demonstration ===

=== Testing Error Handling Functions ===

Test 1: Normal Division
Division successful: 20 / 4 = 5
Result: 5, Error: ERROR_NONE

Test 2: Division by Zero Error
ERROR: Division by zero detected!
Result: 0xffff, Error: ERROR_DIVIDE_ZERO

Test 3: Normal Addition
Addition successful: 100 + 50 = 150
Result: 150, Error: ERROR_NONE

Test 4: Addition Overflow Error
ERROR: Addition overflow detected! 200 + 100 = 300
Result: 44 (truncated to 8-bit), Error: ERROR_OVERFLOW

Test 5: Input Validation - Valid Range
Input validation passed: 75 within [10:100]
Valid: 1, Error: ERROR_NONE

Test 6: Input Validation - Below Range
ERROR: Input 5 outside valid range [10:100]
Valid: 0, Error: ERROR_INVALID_OP

Test 7: Input Validation - Above Range
ERROR: Input 150 outside valid range [10:100]
Valid: 0, Error: ERROR_INVALID_OP

=== Error Handling Functions Test Complete ===
Process finished with return code: 0
Removing Chapter_7_exam

0

#### Conditional Processing Functions
Functions with complex conditional logic and multiple exit points

In [17]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = (
    "Chapter_7_examples/example_31__conditional_processing_functions/"
)
files = ["grade_evaluator.sv", "grade_evaluator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// grade_evaluator.sv
module grade_evaluator_module;

  // Function with complex conditional logic and multiple exit points
  function automatic string determine_letter_grade(input int numeric_score);
    // Early exit for invalid scores
    if (numeric_score < 0) begin
      return "INVALID_NEGATIVE";
    end
    
    if (numeric_score > 100) begin
      return "INVALID_EXCEEDS";
    end
    
    // Complex conditional logic with multiple exit points
    if (numeric_score >= 90) begin
      if (numeric_score >= 97) begin
        return "A+";  // Exit point 1
      end else if (numeric_score >= 93) begin
        return "A";   // Exit point 2
      end else begin
        return "A-";  // Exit point 3
      end
    end else if (numeric_score >= 80) begin
      if (numeric_score >= 87) begin
        return "B+";  // Exit point 4
      end else if (numeric_score >= 83) begin
        return "B";   // Exit point 5
      end else begin
        return "B-";  // Exit point 6
      end
    end else if (numeric_score >= 70) begin
      if (numeric_score >= 77) begin
        return "C+";  // Exit point 7
      end else if (numeric_score >= 73) begin
        return "C";   // Exit point 8
      end else begin
        return "C-";  // Exit point 9
      end
    end else if (numeric_score >= 60) begin
      return "D";     // Exit point 10
    end else begin
      return "F";     // Exit point 11
    end
  endfunction

  // Function with pass/fail determination and conditional bonus
  function automatic string calculate_final_status(input int base_score,
                                                  input bit extra_credit_flag,
                                                  input int bonus_points);
    int adjusted_score;
    string letter_grade;
    
    // Early exit for invalid inputs
    if (base_score < 0 || bonus_points < 0) begin
      return "ERROR_INVALID_INPUT";
    end
    
    // Calculate adjusted score with conditional logic
    adjusted_score = base_score;
    if (extra_credit_flag) begin
      adjusted_score = adjusted_score + bonus_points;
      if (adjusted_score > 100) begin
        adjusted_score = 100;  // Cap at 100
      end
    end
    
    // Get letter grade using the other function
    letter_grade = determine_letter_grade(adjusted_score);
    
    // Multiple exit points based on performance
    if (letter_grade == "F") begin
      return "FAIL_RETAKE_REQUIRED";
    end else if (letter_grade == "D") begin
      return "PASS_BARELY_ACCEPTABLE";
    end else if (letter_grade[0] == "C") begin
      return "PASS_SATISFACTORY";
    end else if (letter_grade[0] == "B") begin
      return "PASS_GOOD_WORK";
    end else if (letter_grade[0] == "A") begin
      return "PASS_EXCELLENT";
    end else begin
      return "ERROR_UNKNOWN_GRADE";
    end
  endfunction

  initial begin
    $display("Grade Evaluator - Conditional Processing Functions Demo");
    $display("=========================================================");
    $display();
  end

endmodule
```

```systemverilog
// grade_evaluator_testbench.sv
module grade_evaluator_testbench;

  // Instantiate design under test
  grade_evaluator_module GRADE_EVALUATOR_INSTANCE();

  initial begin
    // Dump waves
    $dumpfile("grade_evaluator_testbench.vcd");
    $dumpvars(0, grade_evaluator_testbench);

    $display("Testing conditional processing functions with multiple exits");
    $display("===========================================================");
    $display();

    // Test determine_letter_grade function - various exit points
    test_letter_grade_function();
    $display();

    // Test calculate_final_status function - complex conditional logic
    test_final_status_function();
    $display();

    $display("All conditional processing function tests completed!");
    $finish;
  end

  // Test task for letter grade function
  task test_letter_grade_function();
    string result;
    
    $display("Testing determine_letter_grade function:");
    $display("---------------------------------------");
    
    // Test edge cases and invalid inputs (early exits)
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(-5);
    $display("Score: -5   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(105);
    $display("Score: 105  -> Grade: %s", result);
    
    // Test various valid scores hitting different exit points
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(98);
    $display("Score: 98   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(95);
    $display("Score: 95   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(91);
    $display("Score: 91   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(88);
    $display("Score: 88   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(84);
    $display("Score: 84   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(81);
    $display("Score: 81   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(78);
    $display("Score: 78   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(75);
    $display("Score: 75   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(72);
    $display("Score: 72   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(65);
    $display("Score: 65   -> Grade: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.determine_letter_grade(45);
    $display("Score: 45   -> Grade: %s", result);
  endtask

  // Test task for final status function
  task test_final_status_function();
    string result;
    
    $display("Testing calculate_final_status function:");
    $display("--------------------------------------");
    
    // Test invalid inputs (early exits)
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(-10, 1'b0, 0);
    $display("Base:-10, Extra:No, Bonus:0  -> Status: %s", result);
    
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(80, 1'b1, -5);
    $display("Base:80, Extra:Yes, Bonus:-5 -> Status: %s", result);
    
    // Test failing grade
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(45, 1'b0, 0);
    $display("Base:45, Extra:No, Bonus:0   -> Status: %s", result);
    
    // Test barely passing
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(62, 1'b0, 0);
    $display("Base:62, Extra:No, Bonus:0   -> Status: %s", result);
    
    // Test with extra credit helping
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(55, 1'b1, 10);
    $display("Base:55, Extra:Yes, Bonus:10 -> Status: %s", result);
    
    // Test satisfactory performance
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(76, 1'b0, 0);
    $display("Base:76, Extra:No, Bonus:0   -> Status: %s", result);
    
    // Test good performance
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(85, 1'b0, 0);
    $display("Base:85, Extra:No, Bonus:0   -> Status: %s", result);
    
    // Test excellent performance
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(94, 1'b0, 0);
    $display("Base:94, Extra:No, Bonus:0   -> Status: %s", result);
    
    // Test score capping with bonus
    result = GRADE_EVALUATOR_INSTANCE.calculate_final_status(95, 1'b1, 10);
    $display("Base:95, Extra:Yes, Bonus:10 -> Status: %s", result);
  endtask

endmodule
```

Verilator Simulation Output:

Testing determine_letter_grade function:
---------------------------------------
Score: -5   -> Grade: INVALID_NEGATIVE
Score: 105  -> Grade: INVALID_EXCEEDS
Score: 98   -> Grade: A+
Score: 95   -> Grade: A
Score: 91   -> Grade: A-
Score: 88   -> Grade: B+
Score: 84   -> Grade: B
Score: 81   -> Grade: B-
Score: 78   -> Grade: C+
Score: 75   -> Grade: C
Score: 72   -> Grade: C-
Score: 65   -> Grade: D
Score: 45   -> Grade: F

Testing calculate_final_status function:
--------------------------------------
Base:-10, Extra:No, Bonus:0  -> Status: ERROR_INVALID_INPUT
Base:80, Extra:Yes, Bonus:-5 -> Status: ERROR_INVALID_INPUT
Base:45, Extra:No, Bonus:0   -> Status: FAIL_RETAKE_REQUIRED
Base:62, Extra:No, Bonus:0   -> Status: PASS_BARELY_ACCEPTABLE
Base:55, Extra:Yes, Bonus:10 -> Status: PASS_BARELY_ACCEPTABLE
Base:76, Extra:No, Bonus:0   -> Status: PASS_SATISFACTORY
Base:85, Extra:No, Bonus:0   -> Status: PASS_GOOD_WORK
Base:94, Extra:No, Bonus:0   -> Status: P

0

## Void Functions

#### Debug and Logging Functions
Void functions for system debugging and message logging

In [19]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_32__debug_logging_functions/"
files = ["debug_logging_system.sv", "debug_logging_system_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// debug_logging_system.sv
module debug_logging_system #(
    parameter DATA_WIDTH = 8,
    parameter ENABLE_DEBUG = 1
)();
    
    // Internal signals for demonstration
    logic [DATA_WIDTH-1:0] processor_data;
    logic                  memory_valid;
    logic                  error_detected;
    logic                  system_ready;
    
    // Debug and logging void functions
    function void log_info(string message);
        $display("[INFO] [%0t] %s", $time, message);
    endfunction
    
    function void log_warning(string message);
        $display("[WARNING] [%0t] %s", $time, message);
    endfunction
    
    function void log_error(string message);
        $display("[ERROR] [%0t] %s", $time, message);
    endfunction
    
    function void debug_print(string debug_message);
        if (ENABLE_DEBUG) begin
            $display("[DEBUG] [%0t] %s", $time, debug_message);
        end
    endfunction
    
    function void dump_system_state();
        $display("========== SYSTEM STATE DUMP ==========");
        $display("Time: %0t", $time);
        $display("Processor Data: 0x%02h (%0d)", processor_data, 
                processor_data);
        $display("Memory Valid: %b", memory_valid);
        $display("Error Detected: %b", error_detected);
        $display("System Ready: %b", system_ready);
        $display("=======================================");
    endfunction
    
    function void trace_signal_change(string signal_name, 
                                     logic old_value, 
                                     logic new_value);
        if (old_value !== new_value) begin
            $display("[TRACE] [%0t] %s: %b -> %b", $time, signal_name, 
                    old_value, new_value);
        end
    endfunction
    
    function void performance_checkpoint(string checkpoint_name);
        $display("[PERF] [%0t] Checkpoint: %s", $time, checkpoint_name);
    endfunction
    
    // Simple system behavior for demonstration
    initial begin
        log_info("Debug logging system initialized");
        
        // Initialize signals
        processor_data = 8'h00;
        memory_valid = 1'b0;
        error_detected = 1'b0;
        system_ready = 1'b0;
        
        debug_print("Initial state configured");
        performance_checkpoint("System startup");
        
        // Simulate system operation
        #10;
        processor_data = 8'hAA;
        memory_valid = 1'b1;
        debug_print("Memory interface activated");
        
        #5;
        system_ready = 1'b1;
        log_info("System ready for operation");
        dump_system_state();
        
        #15;
        error_detected = 1'b1;
        log_error("Memory corruption detected!");
        trace_signal_change("error_detected", 1'b0, 1'b1);
        
        #10;
        log_warning("Attempting error recovery");
        error_detected = 1'b0;
        processor_data = 8'h55;
        debug_print("Error recovery sequence completed");
        
        performance_checkpoint("Error recovery completed");
        dump_system_state();
        
        log_info("Debug logging system test completed");
    end
    
endmodule
```

```systemverilog
// debug_logging_system_testbench.sv
module debug_logging_testbench;
    
    // Instantiate design under test with debug enabled
    debug_logging_system #(
        .DATA_WIDTH(8),
        .ENABLE_DEBUG(1)
    ) debug_system_instance();
    
    // Testbench logging functions
    function void test_log(string test_message);
        $display("[TEST] [%0t] %s", $time, test_message);
    endfunction
    
    function void test_pass(string test_name);
        $display("[PASS] [%0t] Test '%s' completed successfully", 
                $time, test_name);
    endfunction
    
    function void test_fail(string test_name, string reason);
        $display("[FAIL] [%0t] Test '%s' failed: %s", 
                $time, test_name, reason);
    endfunction
    
    function void test_section_header(string section_name);
        $display("");
        $display("============ %s ============", section_name);
    endfunction
    
    initial begin
        // Setup waveform dumping
        $dumpfile("debug_logging_testbench.vcd");
        $dumpvars(0, debug_logging_testbench);
        
        test_section_header("DEBUG LOGGING SYSTEM TEST");
        test_log("Starting comprehensive debug logging test");
        
        // Test 1: Basic logging functionality
        test_section_header("TEST 1: BASIC LOGGING");
        test_log("Verifying basic log message output");
        #5;
        test_pass("Basic logging verification");
        
        // Test 2: Debug message filtering
        test_section_header("TEST 2: DEBUG FILTERING");
        test_log("Testing debug message filtering capability");
        #20;
        test_pass("Debug filtering test");
        
        // Test 3: Error logging and tracing
        test_section_header("TEST 3: ERROR HANDLING");
        test_log("Monitoring error detection and logging");
        #25;
        test_pass("Error handling verification");
        
        // Test 4: System state dumping
        test_section_header("TEST 4: STATE DUMPING");
        test_log("Verifying system state dump functionality");
        #35;
        test_pass("State dumping test");
        
        // Test 5: Performance checkpoints
        test_section_header("TEST 5: PERFORMANCE TRACKING");
        test_log("Testing performance checkpoint logging");
        #45;
        test_pass("Performance tracking test");
        
        // Final test summary
        test_section_header("TEST SUMMARY");
        test_log("All debug and logging functions verified");
        test_pass("Complete test suite");
        
        $display("");
        $display("========================================");
        $display("Debug logging system test completed at time %0t", $time);
        $display("========================================");
        
        #10;
        $finish;
    end
    
    // Monitor for any unexpected conditions
    initial begin
        #1000;
        test_fail("Timeout test", "Simulation exceeded maximum time limit");
        $finish;
    end
    
endmodule
```

Verilator Simulation Output:

============ DEBUG LOGGING SYSTEM TEST ============
[TEST] [0] Starting comprehensive debug logging test

============ TEST 1: BASIC LOGGING ============
[TEST] [0] Verifying basic log message output
[INFO] [0] Debug logging system initialized
[DEBUG] [0] Initial state configured
[PERF] [0] Checkpoint: System startup
[PASS] [5] Test 'Basic logging verification' completed successfully

============ TEST 2: DEBUG FILTERING ============
[TEST] [5] Testing debug message filtering capability
[DEBUG] [10] Memory interface activated
[INFO] [15] System ready for operation
========== SYSTEM STATE DUMP ==========
Time: 15
Processor Data: 0xaa (170)
Memory Valid: 1
Error Detected: 0
System Ready: 1
[PASS] [25] Test 'Debug filtering test' completed successfully

============ TEST 3: ERROR HANDLING ============
[TEST] [25] Monitoring error detection and logging
[ERROR] [30] Memory corruption detected!
[TRACE] [30] error_detected: 0 -> 1
[WARNING] [40] Attempting error 

0

#### Hardware Initialization Functions
Void functions for initializing hardware components and registers

In [26]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = (
    "Chapter_7_examples/example_33__hardware_initialization_functions/"
)
files = [
    "memory_controller_with_init.sv",
    "memory_controller_with_init_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// memory_controller_with_init.sv
module memory_controller_with_init (
  input  logic        clock_signal,
  input  logic        reset_signal,
  input  logic        enable_signal,
  output logic [7:0]  status_register,
  output logic [15:0] address_register,
  output logic [31:0] data_register
);

  // Internal registers for hardware components
  logic [7:0]  control_register;
  logic [15:0] base_address_register;
  logic [31:0] cache_data_register;
  logic [3:0]  interrupt_mask_register;

  // Initialization values - computed by void functions
  logic [7:0]  init_status_value;
  logic [15:0] init_address_value;
  logic [31:0] init_data_value;

  // Void function to initialize control registers values
  function void compute_control_register_init_values();
    $display("Computing control register initialization values...");
    init_status_value = 8'h80;          // Set ready status bit
    $display("Control register values computed successfully");
  endfunction

  // Void function to initialize memory address register values
  function void compute_memory_address_init_values();
    $display("Computing memory address initialization values...");
    init_address_value = 16'h0000;      // Clear current address
    $display("Memory address values computed successfully");
  endfunction

  // Void function to initialize data path register values
  function void compute_data_path_init_values();
    $display("Computing data path initialization values...");
    init_data_value = 32'h00000000;     // Clear data register
    $display("Data path values computed successfully");
  endfunction

  // Void function to perform complete hardware initialization setup
  function void prepare_complete_hardware_initialization();
    $display("=== Preparing Complete Hardware Initialization ===");
    compute_control_register_init_values();
    compute_memory_address_init_values();
    compute_data_path_init_values();
    $display("=== Hardware Initialization Preparation Complete ===");
    $display();
  endfunction

  // Additional void function to initialize internal component registers
  function void initialize_internal_component_registers();
    $display("Initializing internal component registers...");
    // These are internal only, not mixed with sequential assignments
    control_register = 8'h00;           // Clear control register
    base_address_register = 16'h1000;   // Set base address
    cache_data_register = 32'hDEADBEEF; // Set cache test pattern
    interrupt_mask_register = 4'hF;     // Enable all interrupts
    $display("Internal component registers initialized successfully");
  endfunction

  // Hardware reset and operation logic
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      // Call void functions to prepare initialization values
      prepare_complete_hardware_initialization();
      initialize_internal_component_registers();
      
      // Apply computed initialization values to output registers
      status_register <= init_status_value;
      address_register <= init_address_value;
      data_register <= init_data_value;
    end
    else if (enable_signal) begin
      // Normal operation - set busy bit
      status_register <= status_register | 8'h01;  // Set busy bit
      address_register <= address_register + 1;    // Increment address
      data_register <= data_register + 32'h00000001; // Increment data
    end
  end

endmodule
```

```systemverilog
// memory_controller_with_init_testbench.sv
module memory_controller_initialization_testbench;

  // Testbench signals
  logic        clock_signal;
  logic        reset_signal;
  logic        enable_signal;
  logic [7:0]  status_register;
  logic [15:0] address_register;
  logic [31:0] data_register;

  // Instantiate design under test
  memory_controller_with_init MEMORY_CONTROLLER_INSTANCE (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .enable_signal(enable_signal),
    .status_register(status_register),
    .address_register(address_register),
    .data_register(data_register)
  );

  // Clock generation
  initial begin
    clock_signal = 0;
    forever #5 clock_signal = ~clock_signal;  // 100MHz clock
  end

  // Void function to initialize testbench signals
  function void initialize_testbench_signals();
    $display("Initializing testbench signals...");
    reset_signal = 1'b0;
    enable_signal = 1'b0;
    $display("Testbench signals initialized");
    $display();
  endfunction

  // Void function to apply reset sequence
  function void apply_hardware_reset_sequence();
    $display("Applying hardware reset sequence...");
    reset_signal = 1'b1;
    @(posedge clock_signal);  // Single clock cycle for reset
    reset_signal = 1'b0;
    @(posedge clock_signal);  // Wait for reset to complete
    $display("Hardware reset sequence complete");
    $display();
  endfunction

  // Void function to verify initialization values
  function void verify_hardware_initialization_values();
    $display("Verifying hardware initialization values...");
    $display("Status register: 0x%02h (expected: 0x80)", status_register);
    $display("Address register: 0x%04h (expected: 0x0000)", address_register);
    $display("Data register: 0x%08h (expected: 0x00000000)", data_register);
    
    if (status_register[7] == 1'b1) begin
      $display("Ready bit correctly set in status register");
    end else begin
      $display("Ready bit not set in status register");
    end
    $display();
  endfunction

  // Void function to test enable functionality
  function void test_enable_functionality();
    $display("Testing enable functionality...");
    enable_signal = 1'b1;
    repeat(2) @(posedge clock_signal);
    enable_signal = 1'b0;
    @(posedge clock_signal);
    
    $display("Status after enable: 0x%02h", status_register);
    if (status_register[0] == 1'b1) begin
      $display("Busy bit correctly set after enable");
    end else begin
      $display("Busy bit not set after enable");
    end
    $display();
  endfunction

  // Main test sequence
  initial begin
    // Dump waves for analysis
    $dumpfile("memory_controller_initialization_testbench.vcd");
    $dumpvars(0, memory_controller_initialization_testbench);

    $display("=== Hardware Initialization Functions Test ===");
    $display();

    // Initialize testbench
    initialize_testbench_signals();

    // Apply reset and observe initialization
    apply_hardware_reset_sequence();

    // Verify initialization worked correctly
    verify_hardware_initialization_values();

    // Test normal operation
    test_enable_functionality();

    $display("=== Test Complete ===");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
Control register values computed successfully
Computing memory address initialization values...
Memory address values computed successfully
Computing data path initialization values...
Data path values computed successfully
=== Hardware Initialization Preparation Complete ===

Initializing internal component registers...
Internal component registers initialized successfully
Hardware reset sequence complete

Verifying hardware initialization values...
Status register: 0x80 (expected: 0x80)
Address register: 0x0000 (expected: 0x0000)
Data register: 0x00000000 (expected: 0x00000000)
Ready bit correctly set in status register

Testing enable functionality...
Status after enable: 0x81
Busy bit correctly set after enable

=== Test Complete ===
Process finished with return code: 0
Removing Chapter_7_examples/example_33__hardware_initialization_functions/obj_dir directory...
Chapter_7_examples/example_33__hardware_initialization_functions/obj_dir removed successful

0

#### Data Structure Maintenance
Void functions for maintaining internal data structure consistency

In [31]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_34__data_structure_maintenance/"
files = ["memory_manager_design.sv", "memory_manager_design_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// memory_manager_design.sv
module memory_manager_design;
  
  // Internal data structures
  logic [7:0] memory_blocks [0:15];     // Memory block array
  logic [4:0] free_block_count;         // Number of free blocks (5 bits for 0-16)
  logic [15:0] allocation_bitmap;       // Bitmap for allocated blocks
  
  // Initialize memory manager
  initial begin
    initialize_memory_structures();
    
    // Demonstrate allocation and deallocation
    $display("=== Memory Manager Data Structure Maintenance ===");
    display_memory_status();
    
    // Allocate some blocks
    allocate_memory_block(4'h2);
    allocate_memory_block(4'h5);
    allocate_memory_block(4'h8);
    
    // Deallocate a block
    deallocate_memory_block(4'h5);
    
    // Try invalid operations
    allocate_memory_block(4'h2);  // Already allocated
    deallocate_memory_block(4'hF); // Invalid address
  end
  
  // Void function: Initialize all memory data structures
  function automatic void initialize_memory_structures();
    int block_index;
    
    $display("Initializing memory data structures...");
    
    // Clear all memory blocks
    for (block_index = 0; block_index < 16; block_index++) begin
      memory_blocks[block_index] = 8'h00;
    end
    
    // Reset allocation tracking
    free_block_count = 5'd16;
    allocation_bitmap = 16'h0000;
    
    // Ensure data consistency
    validate_data_consistency();
    $display("Memory structures initialized successfully");
  endfunction
  
  // Void function: Allocate a memory block and maintain consistency
  function automatic void allocate_memory_block(input logic [3:0] block_address);
    $display("\nAllocating block %0d...", block_address);
    
    // Check if block is already allocated
    if (allocation_bitmap[block_address] == 1'b1) begin
      $display("ERROR: Block %0d already allocated!", block_address);
      return;
    end
    
    // Perform allocation
    allocation_bitmap[block_address] = 1'b1;
    memory_blocks[block_address] = 8'hAA;  // Mark as allocated
    free_block_count--;
    
    // Maintain data structure consistency
    update_allocation_tracking();
    display_memory_status();
  endfunction
  
  // Void function: Deallocate a memory block and maintain consistency  
  function automatic void deallocate_memory_block(input logic [3:0] block_address);
    $display("\nDeallocating block %0d...", block_address);
    
    // Note: block_address is 4-bit, so it's always 0-15, no need to check
    // Check if block is actually allocated
    if (allocation_bitmap[block_address] == 1'b0) begin
      $display("ERROR: Block %0d not allocated!", block_address);
      return;
    end
    
    // Perform deallocation
    allocation_bitmap[block_address] = 1'b0;
    memory_blocks[block_address] = 8'h00;  // Clear block
    free_block_count++;
    
    // Maintain data structure consistency
    update_allocation_tracking();
    display_memory_status();
  endfunction
  
  // Void function: Update internal allocation tracking structures
  function automatic void update_allocation_tracking();
    // Recalculate free block count for consistency check
    logic [4:0] calculated_free_count = 0;  // 5 bits to hold 0-16
    int block_index;
    
    for (block_index = 0; block_index < 16; block_index++) begin
      if (allocation_bitmap[block_index] == 1'b0) begin
        calculated_free_count++;
      end
    end
    
    // Verify consistency
    if (calculated_free_count != free_block_count) begin
      $display("WARNING: Inconsistent free count detected!");
      $display("  Expected: %0d, Calculated: %0d", 
               free_block_count, calculated_free_count);
      free_block_count = calculated_free_count;  // Correct it
    end
    
    validate_data_consistency();
  endfunction
  
  // Void function: Validate overall data structure consistency
  function automatic void validate_data_consistency();
    int block_index;
    automatic logic consistency_ok = 1'b1;
    
    // Check memory blocks match bitmap
    for (block_index = 0; block_index < 16; block_index++) begin
      if (allocation_bitmap[block_index] == 1'b1) begin
        if (memory_blocks[block_index] != 8'hAA) begin
          $display("ERROR: Block %0d bitmap/memory mismatch!", block_index);
          consistency_ok = 1'b0;
        end
      end else begin
        if (memory_blocks[block_index] != 8'h00) begin
          $display("ERROR: Block %0d should be cleared!", block_index);
          consistency_ok = 1'b0;
        end
      end
    end
    
    if (consistency_ok) begin
      $display("Data structure consistency: PASS");
    end else begin
      $display("Data structure consistency: FAIL");
    end
  endfunction
  
  // Void function: Display current memory manager status
  function automatic void display_memory_status();
    $display("Memory Status: Free blocks = %0d/16", free_block_count);
    $display("Allocation bitmap: 0x%04h", allocation_bitmap);
  endfunction

endmodule
```

```systemverilog
// memory_manager_design_testbench.sv
module memory_manager_testbench;

  // Instantiate the design under test
  memory_manager_design MEMORY_MANAGER_INSTANCE();

  initial begin
    // Setup waveform dumping
    $dumpfile("memory_manager_testbench.vcd");
    $dumpvars(0, memory_manager_testbench);
    
    $display("=== Chapter 7 Example 34: Data Structure Maintenance ===");
    $display("Testing void functions for memory manager consistency");
    $display();
    
    // Wait for design to complete its operations
    #100;
    
    $display();
    $display("=== Additional Testbench Operations ===");
    
    // Test bulk allocation scenario
    test_bulk_memory_operations();
    
    // Test consistency recovery
    test_consistency_recovery();
    
    #10;
    $display();
    $display("Testbench completed successfully!");
    $finish;
  end

  // Task: Test bulk memory operations
  task test_bulk_memory_operations();
    int operation_count;
    
    $display("\n--- Testing Bulk Operations ---");
    
    // Simulate multiple allocations
    for (operation_count = 10; operation_count < 16; operation_count++) begin
      MEMORY_MANAGER_INSTANCE.allocate_memory_block(operation_count[3:0]);
      #1;
    end
    
    // Simulate some deallocations
    for (operation_count = 12; operation_count < 15; operation_count++) begin
      MEMORY_MANAGER_INSTANCE.deallocate_memory_block(operation_count[3:0]);
      #1;
    end
    
    $display("Bulk operations completed");
  endtask

  // Task: Test consistency recovery mechanisms
  task test_consistency_recovery();
    $display("\n--- Testing Consistency Recovery ---");
    
    // Force an inconsistency (simulate corruption)
    $display("Simulating data corruption...");
    force MEMORY_MANAGER_INSTANCE.free_block_count = 5'd20;  // Invalid value
    
    #1;
    
    // Trigger consistency check through allocation
    MEMORY_MANAGER_INSTANCE.allocate_memory_block(4'h1);
    
    // Release the force
    release MEMORY_MANAGER_INSTANCE.free_block_count;
    
    $display("Consistency recovery test completed");
  endtask

endmodule
```

Verilator Simulation Output:
Initializing memory data structures...
Data structure consistency: PASS
Memory structures initialized successfully
=== Memory Manager Data Structure Maintenance ===
Memory Status: Free blocks = 16/16
Allocation bitmap: 0x0000

Allocating block 2...
Data structure consistency: PASS
Memory Status: Free blocks = 15/16
Allocation bitmap: 0x0004

Allocating block 5...
Data structure consistency: PASS
Memory Status: Free blocks = 14/16
Allocation bitmap: 0x0024

Allocating block 8...
Data structure consistency: PASS
Memory Status: Free blocks = 13/16
Allocation bitmap: 0x0124

Deallocating block 5...
Data structure consistency: PASS
Memory Status: Free blocks = 14/16
Allocation bitmap: 0x0104

Allocating block 2...
ERROR: Block 2 already allocated!

Deallocating block 15...
ERROR: Block 15 not allocated!
=== Chapter 7 Example 34: Data Structure Maintenance ===
Testing void functions for memory manager consistency


=== Additional Testbench Operations ===

--- Tes

0

#### Performance Monitoring Functions
Void functions for collecting and updating performance metrics

#### Assertion Functions
Void functions implementing custom assertion and checking logic